In [2]:
import sys
sys.path.append('../scripts')

import requests
import zipfile
import xml.etree.ElementTree as ET
import os
import pickle
from utils import download, cleanseTransWide, convColNm, cleanSalesData, cleanCmpnyNm, catOwn, catOwn2
from selenium import webdriver
import chromedriver_autoinstaller
import numpy as np
import pandas as pd

In [3]:
chromedriver_autoinstaller.install(path="../utils/")

'/Users/choidamian/Chaebol/utils/101/chromedriver'

# Download Company Code File

In [317]:
class Api:
    
    def __init__(self, apiKey):
        
        self.apiKey = apiKey
        self.corpCode = None

    def getCorpCode(self, path):

        url = "https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=" + self.apiKey
        download(url, path)

        with zipfile.ZipFile(path, 'r') as zipObj:
            fileList = zipObj.namelist()
            target = fileList[0]
        
        loc = os.path.normpath(path)
        loc = loc.split(os.sep)
        loc = '/'.join(loc[:-1])
        
        with zipfile.ZipFile(path, 'r') as zipRef:
            zipRef.extractall(loc)

        self.corpCode = loc + '/' + target
        print(f'Corporation Code File downloaded to {self.corpCode}')

    def findCorpCode(self, corpNm):
        
        if not self.corpCode:
            print('No corporation code file found. Downloading the code file...')
            self.getCorpCode('../data/raw/opendart/corpCode/CORPCODE.zip')

        xml = ET.parse(self.corpCode)
        root = xml.getroot()

        lists = root.findall('list')

        codeList = []
        for list in lists:
            if (list.find('corp_name').text == corpNm):
                codeList.append(list.find('corp_code').text)
                
        return codeList

    def findCorpCode(self, corpNm):
        
        if not self.corpCode:
            print('No corporation code file found. Downloading the code file...')
            self.getCorpCode('../data/raw/opendart/corpCode/CORPCODE.zip')

        xml = ET.parse(self.corpCode)
        root = xml.getroot()

        lists = root.findall('list')

        codeList = []
        for list in lists:
            if (list.find('corp_name').text == corpNm):
                codeList.append(list.find('corp_code').text)
        return codeList

    def findRceptNum(self, corpNm, year):

        codeList = self.findCorpCode(corpNm)
        pblntfTy = 'j'
        pblntfDetailTy = "J004"
        pgCnt = "50"
        bgnDe = str(year) + '0501'
        endDe = str(year+1) + '0430'

        # find the firm that submitted reports to FTC whose name is <firmname>
        for corpCode in codeList:
            url = "https://opendart.fss.or.kr/api/list.json?corp_code=" + corpCode + "&crtfc_key=" + self.apiKey \
            + "&bgn_de=" + bgnDe + "&end_de=" + endDe + "&last_reprt_at=Y" + "&pblntf_ty=" + pblntfTy + \
            "&pblntf_detail_ty=" + pblntfDetailTy + "&page_count=" + pgCnt
            res = requests.get(url)
            content = res.json()
            if 'list' in content:
                break
        
        else:
            raise ValueError(f"{corpNm}: no report found for {bgnDe} ~ {endDe}")


        # find all the reports whose names contain the string '대규모기업집단현황공시[연1회공시및1/4분기용' and save as a list
        # and return rcept_no of the last report in the list
        receptNums = [report['rcept_no'] for report in content['list'] if ('대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)]' \
                        in report['report_nm'])]
        
        if (corpNm in ['아모레퍼시픽그룹', '아이엠엠인베스트먼트'] and year == 2021):
            receptNums = [receptNums[0]] # select the last report received
        
        assert len(receptNums) >= 1, f'{corpNm}: no 대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)] found for {bgnDe} ~ {endDe}'
        assert len(receptNums) <= 1, f'{corpNm}: Too many 대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)] report found'

        return receptNums[0]

    def getSharesURL(self, corpNm, year):

        rceptNum = self.findRceptNum(corpNm, year)
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + rceptNum
        driver.get(url)

        # In the navigation menu click the link that contains the string '소유지분현황'
        driver.find_element_by_partial_link_text('소유지분현황').click()

        # get the url of the '소유지분현황' page and return it
        sharesURL = driver.find_element_by_id('ifrm').get_attribute('src')
        
        # quit
        driver.quit()

        return sharesURL

    def getSharesTable(self, corpNm, year):
        
        # get shares url
        url = self.getSharesURL(corpNm, year)
        
        # parse html tables which includes string data '동일인'(the same person in Korean) with thousands delimiter ',' 
        tables = pd.read_html(url, match='동일인', thousands=',')
        
        # convert column names
        for table in tables:
            if len(table.columns) != 12:
                print(table.head())
                
            if len(table.columns) == 11 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(4, '금융회사', np.nan)
            elif len(table.columns) == 10 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(2, '동일인과의관계', '동일인측')
                table.insert(4, '동일인과의관계3', '계열회사(국내+해외)')
            elif len(table.columns) == 9 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(4, '금융회사', np.nan)
                table.insert(8, '우선주주식수', np.nan)
                table.insert(9, '우선주지분율', np.nan)
                
            table.columns = ['금융회사', '소속회사명', '동일인과의관계', '동일인과의관계2', '동일인과의관계3', '성명', \
                             '보통주주식수', '보통주지분율', '우선주주식수', '우선주지분율', '합계주식수', '합계지분율']
            
        # concat tables
        df = pd.concat(tables)
        df.reset_index(inplace=True)
        df.drop(columns=['index'], inplace=True)
        df = df[df['소속회사명'] != '소속회사명']
        
        # data cleaning
        clean(df['소속회사명'])
        clean(df['성명'])
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace(' ', '', regex=True)
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace('⑧', '', regex=True)
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace('⑨', '', regex=True)
        df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
        df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
        df['합계주식수'] = df['합계주식수'].copy().astype(float)

        # save the data in the after_cleansing folder
        if not os.path.exists(f'../data/ownership-status/raw/{year}'):
            os.makedirs(f'../data/ownership-status/raw/{year}')
        df.to_excel(f'../data/ownership-status/raw/{year}/{corpNm}.xlsx', index=False)

        return df
    
    def getSharesTableAll(self, year):
        
        dfAppnGroupSttus = pd.read_excel(f'../data/grpSumry/appnGroupSttus/appnGroupSttus{year}.xlsx')
        dfAppnGroupSttus['repreCmpny'].replace('에스케이', 'SK', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('엘지', 'LG', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('지에스', 'GS', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('한진칼', '대한항공', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('씨제이', 'CJ', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('엘에스', 'LS', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('네이버', 'NAVER', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('에이치디씨', 'HDC', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('디엘', 'DL', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('에쓰-오일', 'S-Oil', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('오씨아이', 'OCI', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('한국투자금융지주', '한국금융지주', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('디비아이엔씨', 'DB', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('에이케이홀딩스', 'AK홀딩스', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('현대해상화재보험', '현대해상', regex=True, inplace=True)
        clean(dfAppnGroupSttus)
        
        with open(f'../data/utils/gNonChaebol{year}.pkl', 'rb') as f:
            gNonChaebol = pickle.load(f)
        
        for cmpny in dfAppnGroupSttus['repreCmpny']:
            if cmpny in gNonChaebol['cmpny']:
                continue
            dfShares = api.getSharesTable(cmpny, year)
            print("="* 15 + cmpny + "=" * 15)
            print(dfShares.head())
            print(dfShares.tail())
            print(dfShares['동일인과의관계'].unique())
            print(dfShares['동일인과의관계2'].unique())
            print(dfShares['동일인과의관계3'].unique())
            print("="* 15 + cmpny + "=" * 15)        

In [323]:
dfAppnGroupSttus21 = pd.read_excel('../data/grpSumry/appnGroupSttus/appnGroupSttus2021.xlsx')
dfAppnGroupSttus21['repreCmpny'].replace('에스케이', 'SK', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('엘지', 'LG', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('지에스', 'GS', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('한진칼', '대한항공', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('씨제이', 'CJ', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('엘에스', 'LS', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('네이버', 'NAVER', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에이치디씨', 'HDC', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('디엘', 'DL', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에쓰-오일', 'S-Oil', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('오씨아이', 'OCI', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('한국투자금융지주', '한국금융지주', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('디비아이엔씨', 'DB', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에이케이홀딩스', 'AK홀딩스', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('현대해상화재보험', '현대해상', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에이치엠엠', 'HMM', regex=True, inplace=True)
clean(dfAppnGroupSttus21)
dfAppnGroupSttus21.head(50)

unityGrupNm unityGrupCode     smerNm repreCmpny  sumCmpnyCo invstmntLmtt  \
0           삼성      K1000032        이재용       삼성전자          59     상호출자제한집단   
1        현대자동차      K3000027        정의선      현대자동차          53     상호출자제한집단   
2         에스케이      K1000050        최태원         SK         148     상호출자제한집단   
3           엘지      K1000051        구광모         LG          70     상호출자제한집단   
4           롯데      K1000027        신동빈       롯데지주          86     상호출자제한집단   
5          포스코      K1000064        포스코     포스코홀딩스          33     상호출자제한집단   
6           한화      K1000082        김승연         한화          83     상호출자제한집단   
7          지에스      K3000076        허창수         GS          80     상호출자제한집단   
8        현대중공업      K3000052        정몽준     한국조선해양          33     상호출자제한집단   
9           농협      K3000091  농업협동조합중앙회  농업협동조합중앙회          58     상호출자제한집단   
10         신세계      K1000042        이명희        신세계          45     상호출자제한집단   
11         케이티      K1000070        케이티        케이티          48     상호출자제한집단   
12         씨제이      K1000055        이재현         CJ          79     상호출자제한집단   
13          한진      K1000081        조원태       대한항공          31     상호출자제한집단   
14          두산      K1000026        박정원         두산          22     상호출자제한집단   
15         엘에스      K3000062        구자홍         LS          58     상호출자제한집단   
16          부영      K2000060        이중근         부영          23     상호출자제한집단   
17         카카오      K3000137        김범수        카카오         118     상호출자제한집단   
18          DL      K1000011        이준용         DL          36     상호출자제한집단   
19        미래에셋      K3000093        박현주    미래에셋캐피탈          38     상호출자제한집단   
20       현대백화점      K1000087        정지선      현대백화점          25     상호출자제한집단   
21      금호아시아나      K1000008        박삼구       금호건설          27     상호출자제한집단   
22       에쓰-오일      K2000059      에쓰-오일      S-Oil           2     상호출자제한집단   
23        셀트리온      K3000135        서정진    셀트리온홀딩스           8     상호출자제한집단   
24      한국투자금융      K3000110        김남구     한국금융지주          30     상호출자제한집단   
25      교보생명보험      K3000065        신창재     교보생명보험          12     상호출자제한집단   
26         네이버      K3000138        이해진      NAVER          45     상호출자제한집단   
27       에이치디씨      K2000057        정몽규        HDC          29     상호출자제한집단   
28          효성      K1000086        조현준         효성          50     상호출자제한집단   
29          영풍      K1000052        장형진         영풍          27     상호출자제한집단   
30          하림      K3000121        김홍국       하림지주          55     상호출자제한집단   
31       케이티앤지      K1000010      케이티앤지      케이티앤지          10     상호출자제한집단   
32        케이씨씨      K1000001        정몽진       케이씨씨          18     상호출자제한집단   
33          넥슨      K3000119        김정주       엔엑스씨          18     상호출자제한집단   
34      대우조선해양      K3000033     대우조선해양     대우조선해양           5     상호출자제한집단   
35         넷마블      K3000162        방준혁        넷마블          23     상호출자제한집단   
36        호반건설      K3000145        김상열       호반건설          42     상호출자제한집단   
37          SM      K3000134        우오현     티케이케미칼          58     상호출자제한집단   
38          DB      K1000021        김준기         DB          21     상호출자제한집단   
39         코오롱      K1000059        이웅열        코오롱          36     상호출자제한집단   
40       한국타이어      K1000074        조양래     한국앤컴퍼니          21     공시대상기업집단   
41        대우건설      K3000030       대우건설       대우건설          15     공시대상기업집단   
42        오씨아이      K1000024        이우현        OCI          18     공시대상기업집단   
43          태영      K2000045        윤세영     티와이홀딩스          63     공시대상기업집단   
44         이랜드      K2000038        박성수      이랜드월드          33     공시대상기업집단   
45          세아      K1000041        이순형      세아홀딩스          29     공시대상기업집단   
46        중흥건설      K3000139        정창선       중흥건설          37     공시대상기업집단   
47       에이치엠엠      K3000159      에이치엠엠        HMM           4     공시대상기업집단   
48          태광      K1000060        이호진       태광산업          19     공시대상기업집단   
49          동원      K1000025        김재철   동원엔터프라이즈   

In [324]:
gNonChaebol21 = {'grp': ['포스코', '농협', '케이티', '에쓰-오일', '대우조선해양', \
                         '케이티앤지', '대우건설', '에이치엠엠', '한국지엠', '쿠팡', '한국항공우주산업'],
                 'cmpny': ['포스코홀딩스', '농업협동조합중앙회', '케이티', 'S-Oil', '대우조선해양', \
                           '케이티앤지', '대우건설', 'HMM', '한국지엠', '쿠팡', '한국항공우주산업']}

In [320]:
with open('../data/utils/gNonChaebol21.pkl', 'wb') as f:
    pickle.dump(gNonChaebol21, f)

In [325]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
for cmpny in dfAppnGroupSttus21['repreCmpny'][47:]:
    if cmpny in gNonChaebol21['cmpny']:
        continue
    df = api.getSharesTable(cmpny, 2021)
    print("="* 15 + cmpny + "=" * 15)
    print(df.head())
    print(df.tail())
    print(df['동일인과의관계'].unique())
    print(df['동일인과의관계2'].unique())
    print(df['동일인과의관계3'].unique())
    print("="* 15 + cmpny + "=" * 15)

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
===============태광산업===============
    금융회사 소속회사명 동일인과의관계 동일인과의관계2 동일인과의관계3   성명    보통주주식수  보통주지분율 우선주주식수 우선주지분율  \
0  비금융회사  태광산업    동일인측      동일인      동일인  이호진  328189.0   29.48    NaN    NaN   
1  비금융회사  태광산업    동일인측       친족   혈족2~4촌  이원준   83370.0    7.49    NaN    NaN   
2  비금융회사  태광산업    동일인측       친족   혈족2~4촌  이동준    8918.0    0.80    NaN    NaN   
3  비금융회사  태광산업    동일인측       친족   혈족2~4촌  이태준    8982.0    0.81    NaN    NaN   
4  비금융회사  태광산업    동일인측       친족   혈족5~6촌  NaN       NaN     NaN    NaN    NaN   

      합계주식수  합계지분율  
0  328189.0  29.48  
1   83370.0   7.49  
2    8918.0   0.80  
3    8982.0   0.81  
4       NaN    NaN  
     금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3      성명     보통주주식수  \
291  금융회사  예가람저축은행    동일인측         계열회사         계열회사  흥국생명보험   553018.0   
292  금융회사  예가람저축은행    동일인측       동일인측합계       동일인측합계     NaN  

===============삼천리===============
    금융회사 소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3           성명  보통주주식수 보통주지분율 우선주주식수  \
0  비금융회사   삼천리    동일인측      동일인       동일인          이만득  338390   8.34      -   
1  비금융회사   삼천리    동일인측       친족  배우자/혈족1촌          이은희   27045   0.67      -   
2  비금융회사   삼천리    동일인측       친족  배우자/혈족1촌          이은남   27045   0.67      -   
3  비금융회사   삼천리    동일인측       친족  배우자/혈족1촌  이은선YiEunSun   27045   0.67    NaN   
4  비금융회사   삼천리    동일인측       친족    혈족2~4촌          이은백  372070   9.18      -   

  우선주지분율   합계주식수 합계지분율  
0      -  338390  8.34  
1      -   27045  0.67  
2      -   27045  0.67  
3      -   27045  0.67  
4      -  372070  9.18  
     금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3           성명     보통주주식수  \
634  금융회사  아네모이제2호     동일인     기타동일인관련자     기타동일인관련자            -          -   
635  금융회사  아네모이제2호     동일인       동일인측합계       동일인측합계          NaN  465851647   
636  금융회사  아네모이제2호    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주            -          -   
637  금융회사  아네모이

===============반도홀딩스===============
    금융회사  소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3   성명   보통주주식수  보통주지분율 우선주주식수  \
0  비금융회사  반도홀딩스    동일인측      동일인       동일인  권홍사  1621395  69.61%      -   
1  비금융회사  반도홀딩스    동일인측       친족  배우자/혈족1촌  권재현   700000  30.06%      -   
2  비금융회사  반도홀딩스    동일인측       친족  배우자/혈족1촌  유성애     5869   0.25%      -   
3  비금융회사  반도홀딩스    동일인측       친족  배우자/혈족1촌  권보라     1761   0.08%      -   
4  비금융회사  반도홀딩스    동일인측       친족    혈족2~4촌    -        -       -      -   

  우선주지분율    합계주식수   합계지분율  
0      -  1621395  69.61%  
1      -   700000  30.06%  
2      -     5869   0.25%  
3      -     1761   0.08%  
4      -        -       -  
      금융회사   소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3 성명 보통주주식수   보통주지분율  \
641  비금융회사  더스타아시아    동일인측       동일인측합계       동일인측합계  -  30000   75.00%   
642  비금융회사  더스타아시아    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주  -      -        -   
643  비금융회사  더스타아시아    기타주주           기타           기타  /  10000   25.00%   
644  비금융회사  더스타아시아      총계           총계      

===============엠디엠===============
    금융회사 소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3   성명  보통주주식수  보통주지분율 우선주주식수 우선주지분율  \
0  비금융회사   엠디엠    동일인측      동일인       동일인  문주현  190000  95.00%      -      -   
1  비금융회사   엠디엠    동일인측       친족  배우자/혈족1촌  민혜정   10000   5.00%      -      -   
2  비금융회사   엠디엠    동일인측       친족    혈족2~4촌    -       -       -      -      -   
3  비금융회사   엠디엠    동일인측       친족    혈족5~6촌    -       -       -      -      -   
4  비금융회사   엠디엠    동일인측       친족    인척4촌이내    -       -       -      -      -   

    합계주식수   합계지분율  
0  190000  95.00%  
1   10000   5.00%  
2       -       -  
3       -       -  
4       -       -  
     금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3 성명    보통주주식수 보통주지분율  \
290  금융회사  한국자산캐피탈    동일인측     기타동일인관련자     기타동일인관련자  -         -      -   
291  금융회사  한국자산캐피탈    동일인측       동일인측합계       동일인측합계  /  20000000   100%   
292  금융회사  한국자산캐피탈    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주  -         -      -   
293  금융회사  한국자산캐피탈    기타주주           기타           기타  /      

# Ownership Status Data Cleansing

## 태영건설

In [491]:
df = pd.read_excel('../data/ownership-status/raw/2021/티와이홀딩스.xlsx')
df.head()

금융회사   소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3       성명    보통주주식수  보통주지분율 우선주주식수  \
0  비금융회사  티와이홀딩스    동일인측      동일인       동일인      윤세영    245424   0.61%      -   
1  비금융회사  티와이홀딩스    동일인측       친족  배우자/혈족1촌      윤석민  12827810  31.93%      -   
2  비금융회사  티와이홀딩스    동일인측       친족    인척4촌이내      이상희   1128953   2.81%      -   
3  비금융회사  티와이홀딩스    동일인측       친족      친족합계        /  14202187  35.35%      -   
4  비금융회사  티와이홀딩스    동일인측    비영리법인     비영리법인  서암윤세영재단   2830311   7.05%      -   

  우선주지분율     합계주식수   합계지분율  
0      -    245424   0.59%  
1      -  12827810  30.96%  
2      -   1128953   2.73%  
3      -  14202187  34.28%  
4      -   2830311   6.83%

In [492]:
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'][df['합계주식수'].str.contains('\D+', na=False, regex=True)]

257    " 해당사항 없음 "
Name: 합계주식수, dtype: object

In [493]:
# remove " 해당사항 없음 "
df['합계주식수'].replace('\D+', '', regex=True, inplace=True)

In [494]:
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [495]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
df.head()

금융회사   소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3       성명    보통주주식수  보통주지분율 우선주주식수  \
0  비금융회사  티와이홀딩스    동일인측      동일인       동일인      윤세영    245424   0.61%      -   
1  비금융회사  티와이홀딩스    동일인측       친족  배우자/혈족1촌      윤석민  12827810  31.93%      -   
2  비금융회사  티와이홀딩스    동일인측       친족    인척4촌이내      이상희   1128953   2.81%      -   
3  비금융회사  티와이홀딩스    동일인측       친족      친족합계        /  14202187  35.35%      -   
4  비금융회사  티와이홀딩스    동일인측    비영리법인     비영리법인  서암윤세영재단   2830311   7.05%      -   

  우선주지분율       합계주식수   합계지분율  ownCat  ownCat2  
0      -    245424.0   0.59%     0.0      0.0  
1      -  12827810.0  30.96%     NaN      NaN  
2      -   1128953.0   2.73%     NaN      NaN  
3      -  14202187.0  34.28%     0.0      0.0  
4      -   2830311.0   6.83%     NaN      0.0

In [496]:
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfShares.head()

소속회사명       주식수총계
8        티와이홀딩스  41426953.0
12  군포복합개발피에프브이   5000000.0
17       디엠씨미디어    465920.0
21      리앤에스스포츠    500000.0
26          무노스     76311.0

In [497]:
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfMrg.head()

금융회사   소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3       성명    보통주주식수  보통주지분율 우선주주식수  \
0  비금융회사  티와이홀딩스    동일인측      동일인       동일인      윤세영    245424   0.61%      -   
1  비금융회사  티와이홀딩스    동일인측       친족  배우자/혈족1촌      윤석민  12827810  31.93%      -   
2  비금융회사  티와이홀딩스    동일인측       친족    인척4촌이내      이상희   1128953   2.81%      -   
3  비금융회사  티와이홀딩스    동일인측       친족      친족합계        /  14202187  35.35%      -   
4  비금융회사  티와이홀딩스    동일인측    비영리법인     비영리법인  서암윤세영재단   2830311   7.05%      -   

  우선주지분율       합계주식수   합계지분율  ownCat  ownCat2       주식수총계    shares  
0      -    245424.0   0.59%     0.0      0.0  41426953.0  0.005924  
1      -  12827810.0  30.96%     NaN      NaN  41426953.0  0.309649  
2      -   1128953.0   2.73%     NaN      NaN  41426953.0  0.027252  
3      -  14202187.0  34.28%     0.0      0.0  41426953.0  0.342825  
4      -   2830311.0   6.83%     NaN      0.0  41426953.0  0.068321

In [498]:
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn

소속회사명  ownCat    shares
1    걸포도시개발자산과리    99.0  0.840000
3       경산에코에너지    99.0  0.500000
5   군포복합개발피에프브이    99.0  0.646000
7          네오시티    99.0  0.310000
11       디엠씨미디어    99.0  0.085852
14          무노스    99.0  0.399955
16        문고리닷컴    99.0  0.202000
18     부산바이오에너지    99.0  0.800000
19          블루원     0.0  0.122640
23         삼계개발    99.0  0.490000
25          센트로    99.0  0.300000
27      스마트미디어렙    99.0  0.500000
30   신경주역세권공영개발    99.0  0.602000
32      신경주지역개발    99.0  1.000000
34       양주동서도로    99.0  0.599997
36        에스비에스    99.0  0.600027
40    에스비에스미디어넷    99.0  0.037157
41  에스비에스미디어홀딩스     0.0  0.002001
43  에스비에스미디어홀딩스    99.0  0.294778
45    에스비에스바이아컴    99.0  0.489999
48    에스비에스에이앤티    99.0  0.003610
50     에스비에스엠앤씨    99.0  0.600000
52   에스비에스콘텐츠허브    99.0  0.252867
56         에코시티    99.0  0.600000
58       에코시티개발    99.0  0.200000
64         유니시티    99.0  0.415400
66       이너시티개발    99.0  0.600000
69         지엠파크    99.0  0.800000
71   창원복합행정타운개발    99.0  0.490000
73   천안풍세일반산업단지    99.0  0.249999
74         태영건설     0.0  0.135891
76         태영건설    99.0  0.519220
78     태영그레인터미널    99.0  0.570000
79      태영인더스트리     0.0  0.639666
81      태영인더스트리    99.0  0.056535
88     티에스케이이앤피    99.0  0.003413
90   티에스케이코퍼레이션    99.0  0.747857
93   티와이신경주주택개발    99.0  1.000000
94       티와이홀딩스     0.0  0.348749
95       티와이홀딩스    99.0  0.492375

In [499]:
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfOwn2

소속회사명  ownCat    shares
1     걸포도시개발자산과리    99.0  0.840000
3        경산에코에너지    99.0  0.500000
5    군포복합개발피에프브이    99.0  0.646000
7           네오시티    99.0  0.310000
10        디엠씨미디어     0.0  0.373626
12        디엠씨미디어    99.0  0.085852
13       리앤에스스포츠     0.0  0.300000
15           무노스     0.0  0.000026
17           무노스    99.0  0.399955
18         문고리닷컴     0.0  0.198000
20         문고리닷컴    99.0  0.202000
22      부산바이오에너지    99.0  0.800000
23           블루원     0.0  0.122640
27          삼계개발    99.0  0.490000
29           센트로    99.0  0.300000
31       스마트미디어렙    99.0  0.500000
34    신경주역세권공영개발    99.0  0.602000
36       신경주지역개발    99.0  1.000000
38        양주동서도로    99.0  0.599997
39         에스비에스     0.0  0.030818
41         에스비에스    99.0  0.600027
44     에스비에스미디어넷     0.0  0.038340
46     에스비에스미디어넷    99.0  0.037157
47   에스비에스미디어홀딩스     0.0  0.093065
49   에스비에스미디어홀딩스    99.0  0.294778
51     에스비에스바이아컴    99.0  0.489999
54     에스비에스에이앤티    99.0  0.003610
56      에스비에스엠앤씨    99.0  0.600000
57    에스비에스콘텐츠허브     0.0  0.097514
59    에스비에스콘텐츠허브    99.0  0.252867
63          에코시티    99.0  0.600000
65        에코시티개발    99.0  0.200000
71          유니시티    99.0  0.415400
73        이너시티개발    99.0  0.600000
76          지엠파크    99.0  0.800000
78    창원복합행정타운개발    99.0  0.490000
80    천안풍세일반산업단지    99.0  0.249999
81          태영건설     0.0  0.211988
83          태영건설    99.0  0.519220
85      태영그레인터미널    99.0  0.570000
86       태영인더스트리     0.0  0.639666
88       태영인더스트리    99.0  0.056535
95      티에스케이이앤피    99.0  0.003413
97    티에스케이코퍼레이션    99.0  0.747857
100   티와이신경주주택개발    99.0  1.000000
101       티와이홀딩스     0.0  0.513549
102       티와이홀딩스    99.0  0.492375

In [500]:
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfMrgd

소속회사명  ownCat       own      own2  ownername
0    걸포도시개발자산과리    99.0  0.840000  0.840000        NaN
1       경산에코에너지    99.0  0.500000  0.500000        NaN
2   군포복합개발피에프브이    99.0  0.646000  0.646000        NaN
3          네오시티    99.0  0.310000  0.310000        NaN
4        디엠씨미디어    99.0  0.085852  0.085852        NaN
5           무노스    99.0  0.399955  0.399955        NaN
6         문고리닷컴    99.0  0.202000  0.202000        NaN
7      부산바이오에너지    99.0  0.800000  0.800000        NaN
8           블루원     0.0  0.122640  0.122640        NaN
9          삼계개발    99.0  0.490000  0.490000        NaN
10          센트로    99.0  0.300000  0.300000        NaN
11      스마트미디어렙    99.0  0.500000  0.500000        NaN
12   신경주역세권공영개발    99.0  0.602000  0.602000        NaN
13      신경주지역개발    99.0  1.000000  1.000000        NaN
14       양주동서도로    99.0  0.599997  0.599997        NaN
15        에스비에스    99.0  0.600027  0.600027        NaN
16    에스비에스미디어넷    99.0  0.037157  0.037157        NaN
17  에스비에스미디어홀딩스     0.0  0.002001  0.093065        NaN
18  에스비에스미디어홀딩스    99.0  0.294778  0.294778        NaN
19    에스비에스바이아컴    99.0  0.489999  0.489999        NaN
20    에스비에스에이앤티    99.0  0.003610  0.003610        NaN
21     에스비에스엠앤씨    99.0  0.600000  0.600000        NaN
22   에스비에스콘텐츠허브    99.0  0.252867  0.252867        NaN
23         에코시티    99.0  0.600000  0.600000        NaN
24       에코시티개발    99.0  0.200000  0.200000        NaN
25         유니시티    99.0  0.415400  0.415400        NaN
26       이너시티개발    99.0  0.600000  0.600000        NaN
27         지엠파크    99.0  0.800000  0.800000        NaN
28   창원복합행정타운개발    99.0  0.490000  0.490000        NaN
29   천안풍세일반산업단지    99.0  0.249999  0.249999        NaN
30         태영건설     0.0  0.135891  0.211988        NaN
31         태영건설    99.0  0.519220  0.519220        NaN
32     태영그레인터미널    99.0  0.570000  0.570000        NaN
33      태영인더스트리     0.0  0.639666  0.639666        NaN
34      태영인더스트리    99.0  0.056535  0.056535        NaN
35     티에스케이이앤피    99.0  0.003413  0.003413        NaN
36   티에스케이코퍼레이션    99.0  0.747857  0.747857        NaN
37   티와이신경주주택개발    99.0  1.000000  1.000000        NaN
38       티와이홀딩스     0.0  0.348749  0.513549        NaN
39       티와이홀딩스    99.0  0.492375  0.492375        NaN
40       디엠씨미디어     0.0       NaN  0.373626        NaN
41      리앤에스스포츠     0.0       NaN  0.300000        NaN
42          무노스     0.0       NaN  0.000026        NaN
43        문고리닷컴     0.0       NaN  0.198000        NaN
44        에스비에스     0.0       NaN  0.030818        NaN
45    에스비에스미디어넷     0.0       NaN  0.038340        NaN
46   에스비에스콘텐츠허브     0.0       NaN  0.097514        NaN

In [501]:
dfMrgd[dfMrgd['소속회사명']== '디엠씨미디어']

소속회사명  ownCat       own      own2  ownername
4   디엠씨미디어    99.0  0.085852  0.085852        NaN
40  디엠씨미디어     0.0       NaN  0.373626        NaN

In [502]:
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn1

소속회사명  ownCat       own      own2    ownername
9    군포복합개발피에프브이     1.0  0.354000  0.354000         태영건설
14        디엠씨미디어     1.0  0.540522  0.540522        에스비에스
19       리앤에스스포츠     1.0  0.700000  0.700000      SBS미디어넷
23           무노스     1.0  0.600018  0.600018      SBS미디어넷
28         문고리닷컴     1.0  0.600000  0.600000  에스비에스네오파트너스
..           ...     ...       ...       ...          ...
244   창원복합행정타운개발     1.0  0.510000  0.510000         태영건설
248   천안풍세일반산업단지     1.0  0.750001  0.750001         태영건설
252   티와이신경주주택개발     1.0  1.000000  1.000000         태영건설
256     포천바이오에너지     1.0  0.510000  0.510000         태영건설
257     포천바이오에너지     1.0  0.490000  0.490000   티에스케이코퍼레이션

[63 rows x 5 columns]

In [503]:
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']

In [504]:
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.head()

firmname  type    own   own2 ownername
0   걸포도시개발자산과리   1.0  0.580  0.580      태영건설
1   걸포도시개발자산과리  99.0  0.840  0.840       NaN
2      경산에코에너지   1.0  0.500  0.500      태영건설
3      경산에코에너지  99.0  0.500  0.500       NaN
4  군포복합개발피에프브이   1.0  0.354  0.354      태영건설

In [505]:
dfOwn.groupby(['firmname'])['own2'].sum()

firmname
걸포도시개발자산과리     1.420000
경산에코에너지        1.000000
군포복합개발피에프브이    1.000000
네오시티           1.000000
대구남부에이엠씨       1.000000
대동산업단지         1.000000
디엠씨미디어         1.000000
리앤에스스포츠        1.000000
무노스            1.000000
문고리닷컴          1.000000
부산바이오에너지       1.400000
블루원            1.000000
블루원레저          1.000000
삼계개발           1.000000
센트로            1.000000
스마트미디어렙        1.000000
스튜디오에스         1.000000
신경주역세권공영개발     1.000000
신경주지역개발        1.754386
양주동서도로         1.000000
에스비에스          1.000000
에스비에스네오파트너스    1.000000
에스비에스디지털뉴스랩    1.000000
에스비에스미디어넷      1.000000
에스비에스미디어홀딩스    1.000000
에스비에스바이아컴      1.000000
에스비에스아이앤엠      1.000000
에스비에스에이앤티      1.000000
에스비에스엠앤씨       1.000000
에스비에스콘텐츠허브     1.000000
에스비에스플러스       1.000000
에코시스템          1.000000
에코시티           1.000000
에코시티개발         1.000000
엠시에타           1.000000
엠시에타개발         1.000000
여수엑스포환경        1.000000
여천이피에스         1.000000
유니시티           1.000000
이너시티개발         1.000000
인제스피디움         1.000000
지엠파크   

In [506]:
dfOwn[dfOwn['firmname'] == '걸포도시개발자산과리']

firmname  type   own  own2 ownername
0  걸포도시개발자산과리   1.0  0.58  0.58      태영건설
1  걸포도시개발자산과리  99.0  0.84  0.84       NaN

In [507]:
dfOwn.iloc[0, 0] = '걸포도시개발자산관리'
dfOwn.iloc[1, 0] = '걸포도시개발자산관리'
dfOwn.iloc[1, 2] = 0.42
dfOwn.iloc[1, 3] = 0.42
dfOwn[dfOwn['firmname'] == '걸포도시개발자산관리']

firmname  type   own  own2 ownername
0  걸포도시개발자산관리   1.0  0.58  0.58      태영건설
1  걸포도시개발자산관리  99.0  0.42  0.42       NaN

In [508]:
dfOwn[dfOwn['firmname'] == '신경주지역개발']

firmname  type       own      own2 ownername
35  신경주지역개발   1.0  0.754386  0.754386      태영건설
36  신경주지역개발  99.0  1.000000  1.000000       NaN

In [509]:
df[df['소속회사명'] == '신경주지역개발']

금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3    성명 보통주주식수 보통주지분율  \
214  비금융회사  신경주지역개발    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  태영건설  25800    43%   
215  비금융회사  신경주지역개발    동일인측       동일인측합계       동일인측합계   NaN  25800    43%   
216  비금융회사  신경주지역개발    기타주주           기타           기타   NaN  34200    57%   
217  비금융회사  신경주지역개발      총계           총계           총계     /  60000   100%   

    우선주주식수 우선주지분율    합계주식수 합계지분율  ownCat  ownCat2  
214      -      -  25800.0   43%     1.0      1.0  
215      -      -  25800.0   43%     NaN      NaN  
216      -      -  34200.0   57%    99.0     99.0  
217      -      -  34200.0  100%     NaN      NaN

In [510]:
dfOwn.iloc[35, 2] = 25800 / 60000
dfOwn.iloc[35, 3] = 25800 / 60000
dfOwn.iloc[36, 2] = 34200 / 60000
dfOwn.iloc[36, 3] = 34200 / 60000
dfOwn[dfOwn['firmname'] == '신경주지역개발']

firmname  type   own  own2 ownername
35  신경주지역개발   1.0  0.43  0.43      태영건설
36  신경주지역개발  99.0  0.57  0.57       NaN

In [511]:
dfOwn[dfOwn['firmname'] == '티에스케이코퍼레이션']

firmname  type       own      own2 ownername
99   티에스케이코퍼레이션   1.0  0.626071  0.626071    티와이홀딩스
100  티에스케이코퍼레이션  99.0  0.747857  0.747857       NaN

In [512]:
dfOwn.iloc[99, 2] = 358357 / 958357
dfOwn.iloc[99, 3] = 358357 / 958357
dfOwn[dfOwn['firmname'] == '티에스케이코퍼레이션']

firmname  type       own      own2 ownername
99   티에스케이코퍼레이션   1.0  0.373929  0.373929    티와이홀딩스
100  티에스케이코퍼레이션  99.0  0.747857  0.747857       NaN

In [513]:
dfOwn[dfOwn['firmname'] == '티와이신경주주택개발']

firmname  type  own  own2 ownername
102  티와이신경주주택개발   1.0  1.0   1.0      태영건설
103  티와이신경주주택개발  99.0  1.0   1.0       NaN

In [514]:
dfOwn.iloc[102, 2] = 0
dfOwn.iloc[102, 3] = 0
dfOwn[dfOwn['firmname'] == '티와이신경주주택개발']

firmname  type  own  own2 ownername
102  티와이신경주주택개발   1.0  0.0   0.0      태영건설
103  티와이신경주주택개발  99.0  1.0   1.0       NaN

In [515]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/티와이홀딩스.xlsx", index=False)

## 삼성전자

In [488]:
df = pd.read_excel('../data/ownership-status/raw/2021/삼성전자.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [489]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
미라콤아이앤씨    1.0
Name: own2, dtype: float64

In [490]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/삼성전자.xlsx", index=False)

## 에스케이

In [479]:
df = pd.read_excel('../data/ownership-status/raw/2021/SK.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [480]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
에스케이바이오텍       0.000000
에스케이에코플랜트      1.000048
에스케이텔레콤        0.999903
에스케이피아이씨글로벌    1.490000
Name: own2, dtype: float64

In [481]:
df[df['소속회사명'] == '에스케이바이오텍']

금융회사     소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3               성명  \
1051  비금융회사  에스케이바이오텍    동일인측          동일인          동일인              NaN   
1052  비금융회사  에스케이바이오텍    동일인측           친족     배우자/혈족1촌              NaN   
1053  비금융회사  에스케이바이오텍    동일인측           친족       혈족2~4촌              NaN   
1054  비금융회사  에스케이바이오텍    동일인측           친족       혈족5~6촌              NaN   
1055  비금융회사  에스케이바이오텍    동일인측           친족       인척4촌이내              NaN   
1056  비금융회사  에스케이바이오텍    동일인측           친족         친족합계                /   
1057  비금융회사  에스케이바이오텍    동일인측        비영리법인        비영리법인              NaN   
1058  비금융회사  에스케이바이오텍    동일인측        등기된임원        등기된임원              NaN   
1059  비금융회사  에스케이바이오텍    동일인측         자기주식         자기주식              NaN   
1060  비금융회사  에스케이바이오텍    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  SKPharmtecoInc.   
1061  비금융회사  에스케이바이오텍    동일인측     기타동일인관련자     기타동일인관련자              NaN   
1062  비금융회사  에스케이바이오텍    동일인측       동일인측합계       동일인측합계                /   
1063  비금융회사  에스케이바이오텍    기타주주  동일인측이아닌최대주주  동일인측이아닌최대주주              NaN   
1064  비금융회사  에스케이바이오텍    기타주주           기타           기타                /   
1065  비금융회사  에스케이바이오텍      총계           총계           총계                /   

        보통주주식수   보통주지분율 우선주주식수 우선주지분율       합계주식수    합계지분율  ownCat  ownCat2  
1051         -        -      -      -         NaN        -     0.0      0.0  
1052         -        -      -      -         NaN        -     NaN      NaN  
1053         -        -      -      -         NaN        -     NaN      NaN  
1054         -        -      -      -         NaN        -     NaN      NaN  
1055         -        -      -      -         NaN        -     NaN      NaN  
1056         -        -      -      -         NaN        -     0.0      0.0  
1057         -        -      -      -         NaN        -     NaN      0.0  
1058         -        -      -      -         NaN        -     NaN      0.0  
1059         -        -      -      -         NaN        -     NaN      0.0  
1060  22825332  100.00%      -      -  22825332.0  100.00%     1.0      1.0  
1061         -        -      -      -         NaN        -     NaN      NaN  
1062  22825332  100.00%      -      -  22825332.0  100.00%     NaN      NaN  
1063         -        -      -      -         NaN        -    99.0     99.0  
1064         -        -      -      -         NaN        -    99.0     99.0  
1065         -        -      -      -         NaN        -     NaN      NaN

In [482]:
dfOwn[dfOwn['firmname'] == '에스케이바이오텍']

firmname  type  own  own2        ownername
177  에스케이바이오텍   0.0  0.0   0.0              NaN
178  에스케이바이오텍   1.0  NaN   NaN  SKPharmtecoInc.
179  에스케이바이오텍  99.0  0.0   0.0              NaN

In [483]:
dfOwn.iloc[178, 2] = 1
dfOwn.iloc[178, 3] = 1
dfOwn[dfOwn['firmname'] == '에스케이바이오텍']

firmname  type  own  own2        ownername
177  에스케이바이오텍   0.0  0.0   0.0              NaN
178  에스케이바이오텍   1.0  1.0   1.0  SKPharmtecoInc.
179  에스케이바이오텍  99.0  0.0   0.0              NaN

In [484]:
df[df['소속회사명'] == '에스케이피아이씨글로벌']

금융회사        소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3  \
1577  비금융회사  에스케이피아이씨글로벌    동일인측          동일인          동일인   
1578  비금융회사  에스케이피아이씨글로벌    동일인측           친족     배우자/혈족1촌   
1579  비금융회사  에스케이피아이씨글로벌    동일인측           친족       혈족2~4촌   
1580  비금융회사  에스케이피아이씨글로벌    동일인측           친족       혈족5~6촌   
1581  비금융회사  에스케이피아이씨글로벌    동일인측           친족       인척4촌이내   
1582  비금융회사  에스케이피아이씨글로벌    동일인측           친족         친족합계   
1583  비금융회사  에스케이피아이씨글로벌    동일인측        비영리법인        비영리법인   
1584  비금융회사  에스케이피아이씨글로벌    동일인측        등기된임원        등기된임원   
1585  비금융회사  에스케이피아이씨글로벌    동일인측         자기주식         자기주식   
1586  비금융회사  에스케이피아이씨글로벌    동일인측  계열회사(국내+해외)  계열회사(국내+해외)   
1587  비금융회사  에스케이피아이씨글로벌    동일인측     기타동일인관련자     기타동일인관련자   
1588  비금융회사  에스케이피아이씨글로벌    동일인측       동일인측합계       동일인측합계   
1589  비금융회사  에스케이피아이씨글로벌    기타주주  동일인측이아닌최대주주  동일인측이아닌최대주주   
1590  비금융회사  에스케이피아이씨글로벌    기타주주           기타           기타   
1591  비금융회사  에스케이피아이씨글로벌      총계           총계           총계   

                                        성명   보통주주식수   보통주지분율 우선주주식수 우선주지분율  \
1577                                   NaN        -        -      -      -   
1578                                   NaN        -        -      -      -   
1579                                   NaN        -        -      -      -   
1580                                   NaN        -        -      -      -   
1581                                   NaN        -        -      -      -   
1582                                     /        -        -      -      -   
1583                                   NaN        -        -      -      -   
1584                                   NaN        -        -      -      -   
1585                                   NaN        -        -      -      -   
1586                                 에스케이씨  1020000   51.00%      -      -   
1587                                   NaN        -        -      -      -   
1588                                     /  1020000   51.00%      -      -   
1589  PetrochemicalIndustriesCompanyK.S.C.   980000   49.00%      -      -   
1590                                     /   980000   49.00%      -      -   
1591                                     /  2000000  100.00%      -      -   

          합계주식수    합계지분율  ownCat  ownCat2  
1577        NaN        -     0.0      0.0  
1578        NaN        -     NaN      NaN  
1579        NaN        -     NaN      NaN  
1580        NaN        -     NaN      NaN  
1581        NaN        -     NaN      NaN  
1582        NaN        -     0.0      0.0  
1583        NaN        -     NaN      0.0  
1584        NaN        -     NaN      0.0  
1585        NaN        -     NaN      0.0  
1586  1020000.0   51.00%     1.0      1.0  
1587        NaN        -     NaN      NaN  
1588  1020000.0   51.00%     NaN      NaN  
1589   980000.0   51.00%    99.0     99.0  
1590   980000.0   49.00%    99.0     99.0  
1591  2000000.0  100.00%     NaN      NaN

In [485]:
dfOwn[dfOwn['firmname'] == '에스케이피아이씨글로벌']

firmname  type   own  own2 ownername
284  에스케이피아이씨글로벌   0.0  0.00  0.00       NaN
285  에스케이피아이씨글로벌   1.0  0.51  0.51     에스케이씨
286  에스케이피아이씨글로벌  99.0  0.98  0.98       NaN

In [486]:
dfOwn.iloc[286, 2] = 0.49
dfOwn.iloc[286, 3] = 0.49
dfOwn[dfOwn['firmname'] == '에스케이피아이씨글로벌']

firmname  type   own  own2 ownername
284  에스케이피아이씨글로벌   0.0  0.00  0.00       NaN
285  에스케이피아이씨글로벌   1.0  0.51  0.51     에스케이씨
286  에스케이피아이씨글로벌  99.0  0.49  0.49       NaN

In [487]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/SK.xlsx", index=False)

# 현대자동차

In [663]:
df = pd.read_excel('../data/ownership-status/raw/2021/현대자동차.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [664]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
현대위아    1.0
Name: own2, dtype: float64

In [665]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/현대자동차.xlsx", index=False)

# 엘지

In [462]:
df = pd.read_excel('../data/ownership-status/raw/2021/LG.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [463]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
고운누리      0.000000
곤지암예원     0.975000
엘지유플러스    1.000057
Name: own2, dtype: float64

In [464]:
dfOwn.to_excel("../data/ownership-input/2021/LG.xlsx", index=False)

In [465]:
df[df['소속회사명'] == '미젠스토리']

금융회사  소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3      성명   보통주주식수   보통주지분율  \
311  비금융회사  미젠스토리    동일인측          동일인          동일인     NaN      NaN    0.00%   
312  비금융회사  미젠스토리    동일인측           친족     배우자/혈족1촌     NaN      NaN    0.00%   
313  비금융회사  미젠스토리    동일인측           친족       혈족2~4촌     NaN      NaN    0.00%   
314  비금융회사  미젠스토리    동일인측           친족       혈족5~6촌     NaN      NaN    0.00%   
315  비금융회사  미젠스토리    동일인측           친족       인척4촌이내     NaN      NaN    0.00%   
316  비금융회사  미젠스토리    동일인측           친족         친족합계     NaN      0.0    0.00%   
317  비금융회사  미젠스토리    동일인측        비영리법인        비영리법인     NaN      NaN    0.00%   
318  비금융회사  미젠스토리    동일인측        등기된임원        등기된임원     NaN      NaN    0.00%   
319  비금융회사  미젠스토리    동일인측         자기주식         자기주식     NaN      NaN    0.00%   
320  비금융회사  미젠스토리    동일인측         계열회사         계열회사  엘지생활건강  35000.0  100.00%   
321  비금융회사  미젠스토리    동일인측       동일인측합계       동일인측합계     NaN  35000.0  100.00%   
322  비금융회사  미젠스토리    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주     NaN      0.0    0.00%   
323  비금융회사  미젠스토리    기타주주           기타           기타     NaN      0.0    0.00%   
324  비금융회사  미젠스토리      총계           총계           총계     NaN  35000.0  100.00%   

     우선주주식수 우선주지분율    합계주식수    합계지분율  ownCat  ownCat2  
311     NaN    NaN      0.0    0.00%     0.0      0.0  
312     NaN    NaN      0.0    0.00%     NaN      NaN  
313     NaN    NaN      0.0    0.00%     NaN      NaN  
314     NaN    NaN      0.0    0.00%     NaN      NaN  
315     NaN    NaN      0.0    0.00%     NaN      NaN  
316     NaN    NaN      0.0    0.00%     0.0      0.0  
317     NaN    NaN      0.0    0.00%     NaN      0.0  
318     NaN    NaN      0.0    0.00%     NaN      0.0  
319     NaN    NaN      0.0    0.00%     NaN      0.0  
320     NaN    NaN  35000.0  100.00%     1.0      1.0  
321     NaN    NaN  35000.0  100.00%     NaN      NaN  
322     NaN    NaN      0.0    0.00%    99.0     99.0  
323     NaN    NaN      0.0    0.00%    99.0     99.0  
324     NaN    NaN  35000.0  100.00%     NaN      NaN

In [466]:
dfOwn[dfOwn['firmname'] == '미젠스토리']

firmname  type  own  own2 ownername
39    미젠스토리   0.0  0.0   0.0       NaN
40    미젠스토리   1.0  1.0   1.0    엘지생활건강
41    미젠스토리  99.0  0.0   0.0       NaN

미젠스토리에 대해 공시자료에서 중복된 데이터가 발견되므로 엑셀에서 중복된 데이터를 제거함

In [467]:
df[df['소속회사명'] == '밝은누리']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3      성명  보통주주식수   보통주지분율  \
325  비금융회사  밝은누리    동일인측          동일인          동일인     NaN     NaN    0.00%   
326  비금융회사  밝은누리    동일인측           친족     배우자/혈족1촌     NaN     NaN    0.00%   
327  비금융회사  밝은누리    동일인측           친족       혈족2~4촌     NaN     NaN    0.00%   
328  비금융회사  밝은누리    동일인측           친족       혈족5~6촌     NaN     NaN    0.00%   
329  비금융회사  밝은누리    동일인측           친족       인척4촌이내     NaN     NaN    0.00%   
330  비금융회사  밝은누리    동일인측           친족         친족합계     NaN     0.0    0.00%   
331  비금융회사  밝은누리    동일인측        비영리법인        비영리법인     NaN     NaN    0.00%   
332  비금융회사  밝은누리    동일인측        등기된임원        등기된임원     NaN     NaN    0.00%   
333  비금융회사  밝은누리    동일인측         자기주식         자기주식     NaN     NaN    0.00%   
334  비금융회사  밝은누리    동일인측         계열회사         계열회사  엘지생활건강  5000.0  100.00%   
335  비금융회사  밝은누리    동일인측       동일인측합계       동일인측합계     NaN  5000.0  100.00%   
336  비금융회사  밝은누리    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주     NaN     NaN    0.00%   
337  비금융회사  밝은누리    기타주주           기타           기타     NaN     0.0    0.00%   
338  비금융회사  밝은누리      총계           총계           총계     NaN  5000.0  100.00%   

     우선주주식수 우선주지분율   합계주식수    합계지분율  ownCat  ownCat2  
325     NaN    NaN     0.0    0.00%     0.0      0.0  
326     NaN    NaN     0.0    0.00%     NaN      NaN  
327     NaN    NaN     0.0    0.00%     NaN      NaN  
328     NaN    NaN     0.0    0.00%     NaN      NaN  
329     NaN    NaN     0.0    0.00%     NaN      NaN  
330     NaN    NaN     0.0    0.00%     0.0      0.0  
331     NaN    NaN     0.0    0.00%     NaN      0.0  
332     NaN    NaN     0.0    0.00%     NaN      0.0  
333     NaN    NaN     0.0    0.00%     NaN      0.0  
334     NaN    NaN  5000.0  100.00%     1.0      1.0  
335     NaN    NaN  5000.0  100.00%     NaN      NaN  
336     NaN    NaN     0.0    0.00%    99.0     99.0  
337     NaN    NaN     0.0    0.00%    99.0     99.0  
338     NaN    NaN  5000.0  100.00%     NaN      NaN

소속회사 밝은누리에 대해 공시자료에서 중복된 데이터가 발견되므로 엑셀에서 중복된 데이터를 제거함

In [468]:
df[df['소속회사명'] == '엘지파루크']

금융회사  소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3                  성명  \
297  비금융회사  엘지파루크    동일인측          동일인          동일인                 NaN   
298  비금융회사  엘지파루크    동일인측           친족     배우자/혈족1촌                 NaN   
299  비금융회사  엘지파루크    동일인측           친족       혈족2~4촌                 NaN   
300  비금융회사  엘지파루크    동일인측           친족       혈족5~6촌                 NaN   
301  비금융회사  엘지파루크    동일인측           친족       인척4촌이내                 NaN   
302  비금융회사  엘지파루크    동일인측           친족         친족합계                 NaN   
303  비금융회사  엘지파루크    동일인측        비영리법인        비영리법인                 NaN   
304  비금융회사  엘지파루크    동일인측        등기된임원        등기된임원                 NaN   
305  비금융회사  엘지파루크    동일인측         자기주식         자기주식                 NaN   
306  비금융회사  엘지파루크    동일인측         계열회사         계열회사              엘지생활건강   
307  비금융회사  엘지파루크    동일인측       동일인측합계       동일인측합계                 NaN   
308  비금융회사  엘지파루크    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주  FaroukSystems,Inc.   
309  비금융회사  엘지파루크    기타주주           기타           기타                 NaN   
310  비금융회사  엘지파루크      총계           총계           총계                 NaN   

     보통주주식수   보통주지분율  우선주주식수 우선주지분율   합계주식수    합계지분율  ownCat  ownCat2  
297     NaN    0.00%     NaN    NaN     0.0    0.00%     0.0      0.0  
298     NaN    0.00%     NaN    NaN     0.0    0.00%     NaN      NaN  
299     NaN    0.00%     NaN    NaN     0.0    0.00%     NaN      NaN  
300     NaN    0.00%     NaN    NaN     0.0    0.00%     NaN      NaN  
301     NaN    0.00%     NaN    NaN     0.0    0.00%     NaN      NaN  
302     0.0    0.00%     NaN    NaN     0.0    0.00%     0.0      0.0  
303     NaN    0.00%     NaN    NaN     0.0    0.00%     NaN      0.0  
304     NaN    0.00%     NaN    NaN     0.0    0.00%     NaN      0.0  
305     NaN    0.00%     NaN    NaN     0.0    0.00%     NaN      0.0  
306  2500.0   50.00%     NaN    NaN  2500.0   50.00%     1.0      1.0  
307  2500.0   50.00%     NaN    NaN  2500.0   50.00%     NaN      NaN  
308  2500.0   50.00%     NaN    NaN  2500.0   50.00%    99.0     99.0  
309     0.0    0.00%     NaN    NaN     0.0    0.00%    99.0     99.0  
310  5000.0  100.00%     NaN    NaN  5000.0  100.00%     NaN      NaN

소속회사 엘지파루크에 대해 공시자료에서 중복된 데이터가 발견되므로 엑셀에서 중복된 데이터를 제거함

In [469]:
df = pd.read_excel('../data/ownership-status/raw/2021/LG.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [470]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
고운누리      0.000000
곤지암예원     0.975000
엘지유플러스    1.000057
Name: own2, dtype: float64

In [471]:
df[df['소속회사명'] == '고운누리']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3      성명  보통주주식수   보통주지분율  \
227  비금융회사  고운누리    동일인측          동일인          동일인     NaN     NaN      NaN   
228  비금융회사  고운누리    동일인측           친족     배우자/혈족1촌     NaN     NaN      NaN   
229  비금융회사  고운누리    동일인측           친족       혈족2~4촌     NaN     NaN      NaN   
230  비금융회사  고운누리    동일인측           친족       혈족5~6촌     NaN     NaN      NaN   
231  비금융회사  고운누리    동일인측           친족       인척4촌이내     NaN     NaN      NaN   
232  비금융회사  고운누리    동일인측           친족         친족합계     NaN     0.0    0.00%   
233  비금융회사  고운누리    동일인측        비영리법인        비영리법인     NaN     NaN      NaN   
234  비금융회사  고운누리    동일인측        등기된임원        등기된임원     NaN     NaN      NaN   
235  비금융회사  고운누리    동일인측         자기주식         자기주식     NaN     NaN      NaN   
236  비금융회사  고운누리    동일인측         계열회사         계열회사  코카콜라음료  5000.0  100.00%   
237  비금융회사  고운누리    동일인측       동일인측합계       동일인측합계     NaN  5000.0  100.00%   
238  비금융회사  고운누리    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주     NaN     NaN      NaN   
239  비금융회사  고운누리    기타주주           기타           기타     NaN     NaN      NaN   
240  비금융회사  고운누리      총계           총계           총계     NaN     NaN      NaN   

     우선주주식수 우선주지분율   합계주식수    합계지분율  ownCat  ownCat2  
227     NaN    NaN     NaN      NaN     0.0      0.0  
228     NaN    NaN     NaN      NaN     NaN      NaN  
229     NaN    NaN     NaN      NaN     NaN      NaN  
230     NaN    NaN     NaN      NaN     NaN      NaN  
231     NaN    NaN     NaN      NaN     NaN      NaN  
232     NaN    NaN     0.0    0.00%     0.0      0.0  
233     NaN    NaN     NaN      NaN     NaN      0.0  
234     NaN    NaN     NaN      NaN     NaN      0.0  
235     NaN    NaN     NaN      NaN     NaN      0.0  
236     NaN    NaN  5000.0  100.00%     1.0      1.0  
237     NaN    NaN  5000.0  100.00%     NaN      NaN  
238     NaN    NaN     NaN      NaN    99.0     99.0  
239     NaN    NaN     NaN      NaN    99.0     99.0  
240     NaN    NaN     NaN      NaN     NaN      NaN

In [472]:
df.loc[240, '합계주식수'] = 5000.0

In [473]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
곤지암예원     0.975000
엘지유플러스    1.000057
Name: own2, dtype: float64

In [474]:
# 곤지암예원의 경우 오기로 추정
df[df['소속회사명']=='곤지암예원']

금융회사  소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3          성명     보통주주식수  \
915  비금융회사  곤지암예원    동일인측          동일인          동일인         NaN        NaN   
916  비금융회사  곤지암예원    동일인측           친족     배우자/혈족1촌         NaN        NaN   
917  비금융회사  곤지암예원    동일인측           친족       혈족2~4촌         NaN        NaN   
918  비금융회사  곤지암예원    동일인측           친족       혈족5~6촌         NaN        NaN   
919  비금융회사  곤지암예원    동일인측           친족       인척4촌이내         NaN        NaN   
920  비금융회사  곤지암예원    동일인측           친족         친족합계         NaN        0.0   
921  비금융회사  곤지암예원    동일인측        비영리법인        비영리법인         NaN        NaN   
922  비금융회사  곤지암예원    동일인측        등기된임원        등기된임원         석영한   120000.0   
923  비금융회사  곤지암예원    동일인측         자기주식         자기주식         NaN        NaN   
924  비금융회사  곤지암예원    동일인측         계열회사         계열회사  에스앤아이코퍼레이션  1440000.0   
925  비금융회사  곤지암예원    동일인측       동일인측합계       동일인측합계         NaN  1600000.0   
926  비금융회사  곤지암예원    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주         NaN        NaN   
927  비금융회사  곤지암예원    기타주주           기타           기타         NaN        0.0   
928  비금융회사  곤지암예원      총계           총계           총계         NaN  1600000.0   

      보통주지분율  우선주주식수 우선주지분율      합계주식수    합계지분율  ownCat  ownCat2  
915    0.00%     NaN    NaN        0.0    0.00%     0.0      0.0  
916    0.00%     NaN    NaN        0.0    0.00%     NaN      NaN  
917    0.00%     NaN    NaN        0.0    0.00%     NaN      NaN  
918    0.00%     NaN    NaN        0.0    0.00%     NaN      NaN  
919    0.00%     NaN    NaN        0.0    0.00%     NaN      NaN  
920    0.00%     NaN    NaN        0.0    0.00%     0.0      0.0  
921    0.00%     NaN    NaN        0.0    0.00%     NaN      0.0  
922   10.00%     NaN    NaN   120000.0   10.00%     NaN      0.0  
923    0.00%     NaN    NaN        0.0    0.00%     NaN      0.0  
924   90.00%     NaN    NaN  1440000.0   90.00%     1.0      1.0  
925  100.00%     NaN    NaN  1600000.0  100.00%     NaN      NaN  
926    0.00%     NaN    NaN        0.0    0.00%    99.0     99.0  
927    0.00%     NaN    NaN        0.0    0.00%    99.0     99.0  
928  100.00%     NaN    NaN  1600000.0  100.00%     NaN      NaN

In [475]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/LG.xlsx", index=False)

# 롯데지주

In [452]:
df = pd.read_excel('../data/ownership-status/raw/2021/롯데지주.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [453]:
df.head(50)

금융회사 소속회사명 동일인과의관계 동일인과의관계2 동일인과의관계3              성명    보통주주식수  보통주지분율  \
0   비금융회사  롯데지주    동일인측      동일인      동일인             신동빈  13683202  13.04%   
1   비금융회사  롯데지주    동일인측       친족   혈족2~4촌             신영자   3434230   3.27%   
2   비금융회사  롯데지주    동일인측       친족   혈족2~4촌             신동주    983029   0.94%   
3   비금융회사  롯데지주    동일인측       친족   혈족2~4촌             장선윤       801   0.00%   
4   비금융회사  롯데지주    동일인측       친족   혈족2~4촌             장정안     77037   0.07%   
5   비금융회사  롯데지주    동일인측       친족   혈족2~4촌             신유미     42254   0.04%   
6   비금융회사  롯데지주    동일인측       친족   혈족5~6촌             조현욱       151   0.00%   
7   비금융회사  롯데지주    동일인측       친족   혈족5~6촌             신영우        34   0.00%   
8   비금융회사  롯데지주    동일인측       친족   혈족5~6촌             신복남       320   0.00%   
9   비금융회사  롯데지주    동일인측       친족   인척4촌이내             조용완         5   0.00%   
10  비금융회사  롯데지주    동일인측       친족   인척4촌이내             박희청       320   0.00%   
11  비금융회사  롯데지주    동일인측       친족   인척4촌이내           오고야요이      1792   0.00%   
12  비금융회사  롯데지주    동일인측       친족     친족합계            친족합계   4539973   4.33%   
13  비금융회사  롯데지주    동일인측    비영리법인      NaN         재롯데문화재단     90869   0.09%   
14  비금융회사  롯데지주    동일인측    비영리법인      NaN         재롯데장학재단   3394589   3.24%   
15  비금융회사  롯데지주    동일인측    비영리법인      NaN  사회복지법인롯데삼동복지재단     54807   0.05%   
16  비금융회사  롯데지주    동일인측    등기된임원      NaN             김이훈        63   0.00%   
17  비금융회사  롯데지주    동일인측    등기된임원      NaN             남창희       109   0.00%   
18  비금융회사  롯데지주    동일인측    등기된임원      NaN             송용덕      1400   0.00%   
19  비금융회사  롯데지주    동일인측    등기된임원      NaN             강희태       127   0.00%   
20  비금융회사  롯데지주    동일인측    등기된임원      NaN             류민열       195   0.00%   
21  비금융회사  롯데지주    동일인측    등기된임원      NaN             김익수       100   0.00%   
22  비금융회사  롯데지주    동일인측    등기된임원      NaN             오일근        44   0.00%   
23  비금융회사  롯데지주    동일인측    등기된임원      NaN             이봉철       190   0.00%   
24  비금융회사  롯데지주    동일인측    등기된임원      NaN             이동진       411   0.00%   
25  비금융회사  롯데지주    동일인측    등기된임원      NaN             유형주        81   0.00%   
26  비금융회사  롯데지주    동일인측    등기된임원      NaN             류제돈       481   0.00%   
27  비금융회사  롯데지주    동일인측    등기된임원      NaN             서광식         8   0.00%   
28  비금융회사  롯데지주    동일인측    등기된임원      NaN             이영구         9   0.00%   
29  비금융회사  롯데지주    동일인측    등기된임원      NaN             장학영       100   0.00%   
30  비금융회사  롯데지주    동일인측    등기된임원      NaN             정찬우      4400   0.00%   
31  비금융회사  롯데지주    동일인측    등기된임원      NaN             김민아       325   0.00%   
32  비금융회사  롯데지주    동일인측    등기된임원      NaN             황용석        18   0.00%   
33  비금융회사  롯데지주    동일인측    등기된임원      NaN             기원규        70   0.00%   
34  비금융회사  롯데지주    동일인측    등기된임원      NaN             김현옥       263   0.00%   
35  비금융회사  롯데지주    동일인측    등기된임원      NaN             김원재       177   0.00%   
36  비금융회사  롯데지주    동일인측    등기된임원      NaN             추광식       365   0.00%   
37  비금융회사  롯데지주    동일인측    등기된임원      NaN             이형규        57   0.00%   
38  비금융회사  롯데지주    동일인측    등기된임원      NaN             정부옥       400   0.00%   
39  비금융회사  롯데지주    동일인측    등기된임원      NaN             정경문       600   0.00%   
40  비금융회사  롯데지주    동일인측    등기된임원      NaN             하영수       710   0.00%   
41  비금융회사  롯데지주    동일인측    등기된임원      NaN             이재옥        40   0.00%   
42  비금융회사  롯데지주    동일인측    등기된임원      NaN             김상수        91   0.00%   
43  비금융회사  롯데지주    동일인측    등기된임원      NaN             최홍훈       378   0.00%   
44  비금융회사  롯데지주    동일인측    등기된임원      NaN             권오승       230   0.00%   
45  비금융회사  롯데지주    동일인측    등기된임원      NaN             심명섭       200   0.00%   
46  비금융회사  롯데지주    동일인측    등기된임원      NaN             이종환        25   0.00%   
47  비금융회사  롯데지주    동일인측    등기된임원      NaN             이승국        17   0.00%   
48  비금융회사  롯데지주    동일인측    등기된임원      NaN             김경봉        22   0.00%   
49  비금융회사  롯데지주    동일인측    등기된임원      NaN             손

In [454]:
df['동일인과의관계2'].unique()

array(['동일인', '친족', '비영리법인', '등기된임원', '자기주식', '계열회사', '해외계열', '동일인측합계',
       '기타', '롯데지주㈜합계', '㈜호텔롯데합계', '롯데면세점제주㈜합계', '㈜부산롯데호텔합계', '롯데물산㈜합계',
       '스마일위드㈜합계', '롯데디에프리테일㈜합계', '롯데제과㈜합계', '롯데칠성음료㈜합계', '롯데푸드㈜합계',
       '동일인측이아닌최다주주', '롯데네슬레코리아㈜합계', '롯데지알에스㈜합계', '㈜롯데아사히주류합계', '㈜충북소주합계',
       '엠제이에이와인㈜합계', '씨에이치음료㈜합계', '㈜백학음료합계', '스위트위드㈜합계', '산청음료㈜합계',
       '푸드위드㈜합계', '롯데쇼핑㈜합계', '롯데역사㈜합계', '㈜코리아세븐합계', '에프알엘코리아㈜합계',
       '㈜우리홈쇼핑합계', '롯데김해개발㈜합계', '롯데수원역쇼핑타운㈜합계', '롯데지에프알㈜합계',
       '롯데송도쇼핑타운㈜합계', '한국에스티엘㈜합계', '씨에스유통㈜합계', '롯데하이마트㈜합계', '롯데인천개발㈜합계',
       '롯데인천타운㈜합계', '롯데타운동탄㈜합계', '롯데울산개발㈜합계', '롯데컬처웍스㈜합계', '롯데디엠씨개발㈜합계',
       '롯데케미칼㈜합계', '㈜케이피켐텍합계', '롯데엠시시㈜합계', '삼박엘에프티㈜합계', '데크항공㈜합계',
       '롯데미쓰이화학㈜합계', '롯데베르살리스엘라스토머스㈜합계', '롯데정밀화학㈜합계', '한덕화학㈜합계',
       '롯데지에스화학㈜합계', '한국후지필름㈜합계', '롯데알미늄㈜합계', '캐논코리아비즈니스솔루션㈜합계',
       '㈜엔젤위드합계', '롯데건설㈜합계', '롯데멤버스㈜합계', '롯데상사㈜합계', '롯데글로벌로지스㈜합계',
       '롯데부산신항로지스㈜합계', '롯데렌탈㈜합계', '제이지산업㈜합계', '롯데오토케어㈜합계', '㈜그린카합계',
       '㈜이지고합계', '서울복합물류프로젝트금융투자㈜합계',

In [455]:
df['동일인과의관계2'] = df['동일인과의관계2'].replace("^[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]+㈜합계$", '총계', regex=True)
df['동일인과의관계2'].unique()

array(['동일인', '친족', '비영리법인', '등기된임원', '자기주식', '계열회사', '해외계열', '동일인측합계',
       '기타', '총계', '㈜호텔롯데합계', '㈜부산롯데호텔합계', '동일인측이아닌최다주주', '㈜롯데아사히주류합계',
       '㈜충북소주합계', '㈜백학음료합계', '㈜코리아세븐합계', '㈜우리홈쇼핑합계', '㈜케이피켐텍합계',
       '㈜엔젤위드합계', '㈜그린카합계', '㈜이지고합계', '㈜대홍기획합계', '㈜모비쟆미디어합계', '㈜엠허브합계',
       '㈜롯데자이언츠합계', '㈜씨텍합계', '㈜에스디제이합계', '유한책임회사성암허심합계'], dtype=object)

In [456]:
df['동일인과의관계2'] = df['동일인과의관계2'].replace("^㈜[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]+합계$", '총계', regex=True)
df['동일인과의관계2'].unique()

array(['동일인', '친족', '비영리법인', '등기된임원', '자기주식', '계열회사', '해외계열', '동일인측합계',
       '기타', '총계', '동일인측이아닌최다주주', '유한책임회사성암허심합계'], dtype=object)

In [457]:
df['동일인과의관계2'] = df['동일인과의관계2'].replace("^유한책임회사[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]+합계$", '총계', regex=True)
df['동일인과의관계2'].unique()

array(['동일인', '친족', '비영리법인', '등기된임원', '자기주식', '계열회사', '해외계열', '동일인측합계',
       '기타', '총계', '동일인측이아닌최다주주'], dtype=object)

In [458]:
df['동일인과의관계2'] = df['동일인과의관계2'].replace("해외계열", '계열회사', regex=True)

In [459]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
롯데캐피탈    1.0
Name: own2, dtype: float64

In [461]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/롯데.xlsx", index=False)

# 한화

In [445]:
df = pd.read_excel('../data/ownership-status/raw/2021/한화.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [446]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
아쿠아플라넷       0.00000
원오원글로벌       0.03825
한화호텔앤드리조트    1.00000
Name: own2, dtype: float64

In [447]:
df[df['소속회사명'] == '아쿠아플라넷']

금융회사   소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3         성명  보통주주식수  \
1457  비금융회사  아쿠아플라넷    동일인측          동일인          동일인        NaN     NaN   
1458  비금융회사  아쿠아플라넷    동일인측           친족     배우자/혈족1촌        NaN     NaN   
1459  비금융회사  아쿠아플라넷    동일인측           친족     배우자/혈족1촌        NaN     NaN   
1460  비금융회사  아쿠아플라넷    동일인측           친족     배우자/혈족1촌        NaN     NaN   
1461  비금융회사  아쿠아플라넷    동일인측           친족     배우자/혈족1촌        NaN     NaN   
1462  비금융회사  아쿠아플라넷    동일인측           친족       혈족2~4촌        NaN     NaN   
1463  비금융회사  아쿠아플라넷    동일인측           친족       혈족5~6촌        NaN     NaN   
1464  비금융회사  아쿠아플라넷    동일인측           친족       인척4촌이내        NaN     NaN   
1465  비금융회사  아쿠아플라넷    동일인측           친족         친족합계        NaN       -   
1466  비금융회사  아쿠아플라넷    동일인측        비영리법인        비영리법인        NaN     NaN   
1467  비금융회사  아쿠아플라넷    동일인측        등기된임원        등기된임원        NaN     NaN   
1468  비금융회사  아쿠아플라넷    동일인측        등기된임원        등기된임원        NaN     NaN   
1469  비금융회사  아쿠아플라넷    동일인측        등기된임원        등기된임원        NaN     NaN   
1470  비금융회사  아쿠아플라넷    동일인측        등기된임원        등기된임원        NaN     NaN   
1471  비금융회사  아쿠아플라넷    동일인측         자기주식         자기주식        NaN     NaN   
1472  비금융회사  아쿠아플라넷    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  한화호텔앤드리조트  100000   
1473  비금융회사  아쿠아플라넷    동일인측  계열회사(국내+해외)  계열회사(국내+해외)        NaN     NaN   
1474  비금융회사  아쿠아플라넷    동일인측  계열회사(국내+해외)  계열회사(국내+해외)        NaN     NaN   
1475  비금융회사  아쿠아플라넷    동일인측     기타동일인관련자     기타동일인관련자        NaN     NaN   
1476  비금융회사  아쿠아플라넷    동일인측       동일인측합계       동일인측합계        NaN     NaN   
1477  비금융회사  아쿠아플라넷    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주        NaN     NaN   
1478  비금융회사  아쿠아플라넷    기타주주           기타           기타        NaN     NaN   
1479  비금융회사  아쿠아플라넷      총계           총계           총계        NaN     NaN   

      보통주지분율 우선주주식수 우선주지분율     합계주식수   합계지분율  ownCat  ownCat2  
1457     NaN    NaN    NaN       NaN     NaN     0.0      0.0  
1458       -    NaN    NaN       NaN       -     NaN      NaN  
1459       -    NaN    NaN       NaN       -     NaN      NaN  
1460       -    NaN    NaN       NaN       -     NaN      NaN  
1461       -    NaN    NaN       NaN       -     NaN      NaN  
1462       -    NaN    NaN       NaN       -     NaN      NaN  
1463       -    NaN    NaN       NaN       -     NaN      NaN  
1464       -    NaN    NaN       NaN       -     NaN      NaN  
1465       -      -    NaN       NaN       -     0.0      0.0  
1466       -    NaN    NaN       NaN       -     NaN      0.0  
1467       -    NaN    NaN       NaN       -     NaN      0.0  
1468       -    NaN    NaN       NaN       -     NaN      0.0  
1469       -    NaN    NaN       NaN       -     NaN      0.0  
1470       -    NaN    NaN       NaN       -     NaN      0.0  
1471       -    NaN    NaN       NaN       -     NaN      0.0  
1472  100.00    NaN    NaN  100000.0  100.00     1.0      1.0  
1473     NaN    NaN    NaN       NaN     NaN     1.0      1.0  
1474     NaN    NaN    NaN       NaN     NaN     1.0      1.0  
1475     NaN    NaN    NaN       NaN     NaN     NaN      NaN  
1476     NaN    NaN    NaN       NaN     NaN     NaN      NaN  
1477     NaN    NaN    NaN       NaN     NaN    99.0     99.0  
1478     NaN    NaN    NaN       NaN     NaN    99.0     99.0  
1479  100.00      -    NaN       NaN  100.00     NaN      NaN

In [448]:
df.loc[1479, '합계주식수'] = 100000.0

In [449]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
원오원글로벌       0.03825
한화호텔앤드리조트    1.00000
Name: own2, dtype: float64

In [450]:
df[df['소속회사명'] == '원오원글로벌']

금융회사   소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3              성명  보통주주식수  \
849  비금융회사  원오원글로벌    동일인측          동일인          동일인             NaN     NaN   
850  비금융회사  원오원글로벌    동일인측           친족     배우자/혈족1촌             NaN     NaN   
851  비금융회사  원오원글로벌    동일인측           친족       혈족2~4촌             NaN     NaN   
852  비금융회사  원오원글로벌    동일인측           친족       혈족5~6촌             NaN     NaN   
853  비금융회사  원오원글로벌    동일인측           친족       인척4촌이내             NaN     NaN   
854  비금융회사  원오원글로벌    동일인측           친족         친족합계             NaN       -   
855  비금융회사  원오원글로벌    동일인측        비영리법인        비영리법인             NaN     NaN   
856  비금융회사  원오원글로벌    동일인측        등기된임원        등기된임원             NaN     NaN   
857  비금융회사  원오원글로벌    동일인측         자기주식         자기주식             NaN     NaN   
858  비금융회사  원오원글로벌    동일인측         계열회사         계열회사             NaN     NaN   
859  비금융회사  원오원글로벌    동일인측     기타동일인관련자     기타동일인관련자  에이치글로벌파트너스합자조합  179500   
860  비금융회사  원오원글로벌    동일인측       동일인측합계       동일인측합계             NaN  179500   
861  비금융회사  원오원글로벌    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주             NaN     NaN   
862  비금융회사  원오원글로벌    기타주주           기타           기타             NaN    7139   
863  비금융회사  원오원글로벌      총계           총계           총계             NaN  186639   

     보통주지분율 우선주주식수 우선주지분율     합계주식수   합계지분율  ownCat  ownCat2  
849       -    NaN    NaN       NaN       -     0.0      0.0  
850       -    NaN    NaN       NaN       -     NaN      NaN  
851       -    NaN    NaN       NaN       -     NaN      NaN  
852       -    NaN    NaN       NaN       -     NaN      NaN  
853       -    NaN    NaN       NaN       -     NaN      NaN  
854       -      -    NaN       NaN       -     0.0      0.0  
855       -    NaN    NaN       NaN       -     NaN      0.0  
856       -    NaN    NaN       NaN       -     NaN      0.0  
857       -    NaN    NaN       NaN       -     NaN      0.0  
858       -    NaN    NaN       NaN       -     1.0      1.0  
859   96.17    NaN    NaN  179500.0   96.17     NaN      NaN  
860   96.17      -    NaN  179500.0   96.17     NaN      NaN  
861       -    NaN    NaN       NaN       -    99.0     99.0  
862    3.83    NaN    NaN    7139.0    3.83    99.0     99.0  
863  100.00      -    NaN  186639.0  100.00     NaN      NaN

In [451]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/한화.xlsx", index=False)

# 지에스

In [651]:
df = pd.read_excel('../data/ownership-status/raw/2021/GS.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [655]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
파르나스호텔    0.324444
Name: own2, dtype: float64

In [656]:
df

금융회사     소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3                 성명  \
0    비금융회사       지에스    동일인측          동일인          동일인                허창수   
1    비금융회사       지에스    동일인측           친족     배우자/혈족1촌  HuhAliceYoonyoung   
2    비금융회사       지에스    동일인측           친족     배우자/혈족1촌                허윤홍   
3    비금융회사       지에스    동일인측           친족       혈족2~4촌                허광수   
4    비금융회사       지에스    동일인측           친족       혈족2~4촌                허남각   
..     ...       ...     ...          ...          ...                ...   
491  비금융회사      프로케어    동일인측       동일인측합계       동일인측합계                NaN   
492  비금융회사      프로케어      총계          NaN          NaN                NaN   
493   금융회사  지베스코자산운용    동일인측  계열회사(국내+해외)  계열회사(국내+해외)               지베스코   
494   금융회사  지베스코자산운용    동일인측       동일인측합계       동일인측합계                NaN   
495   금융회사  지베스코자산운용      총계          NaN          NaN                NaN   

      보통주주식수   보통주지분율  우선주주식수 우선주지분율      합계주식수    합계지분율  ownCat  ownCat2  
0    4417695    4.75%     NaN  0.00%  4417695.0    4.66%     0.0      0.0  
1     323917    0.35%     NaN  0.00%   323917.0    0.34%     NaN      NaN  
2     494888    0.53%     NaN  0.00%   494888.0    0.52%     NaN      NaN  
3    2033983    2.19%     NaN  0.00%  2033983.0    2.15%     NaN      NaN  
4    1956670    2.11%     NaN  0.00%  1956670.0    2.07%     NaN      NaN  
..       ...      ...     ...    ...        ...      ...     ...      ...  
491   100000  100.00%     NaN  0.00%   100000.0  100.00%     NaN      NaN  
492   100000  100.00%     NaN  0.00%   100000.0  100.00%     NaN      NaN  
493   920000  100.00%     NaN  0.00%   920000.0  100.00%     1.0      1.0  
494   920000  100.00%     NaN  0.00%   920000.0  100.00%     NaN      NaN  
495   920000  100.00%     NaN  0.00%   920000.0  100.00%     NaN      NaN  

[496 rows x 14 columns]

In [657]:
dfOwn

firmname  type   own  own2 ownername
0       가승개발   1.0  0.45  0.45        승산
1       가승개발  99.0  0.55  0.55       NaN
2      강남메트로   1.0  0.39  0.39     지에스건설
3      강남메트로   1.0  0.04  0.04    지에스네오텍
4      강남메트로  99.0  0.57  0.57       NaN
..       ...   ...   ...   ...       ...
130  향산2도시개발   1.0  0.50  0.50     지에스건설
131  향산2도시개발  99.0  0.50  0.50       NaN
132    후레쉬미트   1.0  0.51  0.51    지에스리테일
133    후레쉬미트  99.0  0.49  0.49       NaN
134    후레쉬서브   1.0  1.00  1.00    지에스리테일

[135 rows x 5 columns]

In [658]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/GS.xlsx", index=False)

# 현대중공업

In [659]:
df = pd.read_excel('../data/ownership-status/raw/2021/한국조선해양.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [660]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
태백풍력발전           1.000000
현대일렉트릭앤에너지시스템    1.000009
Name: own2, dtype: float64

In [662]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/현대중공업.xlsx", index=False)

# 신세계

In [430]:
df = pd.read_excel('../data/ownership-status/raw/2021/신세계.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [431]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
신세계       0.0
인천신세계    10.0
Name: own2, dtype: float64

In [432]:
df[df['소속회사명'] == '신세계']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3   성명   보통주주식수 보통주지분율 우선주주식수  \
0   비금융회사   신세계    동일인측          동일인          동일인  이명희   984518  10.00      -   
1   비금융회사   신세계    동일인측           친족     배우자/혈족1촌  정유경  1827521  18.56      -   
2   비금융회사   신세계    동일인측           친족       혈족2~4촌    -        -      -      -   
3   비금융회사   신세계    동일인측           친족       혈족5~6촌    -        -      -      -   
4   비금융회사   신세계    동일인측           친족       인척4촌이내    -        -      -      -   
5   비금융회사   신세계    동일인측           친족         친족합계    /  1827521  18.56      -   
6   비금융회사   신세계    동일인측        비영리법인        비영리법인    -        -      -      -   
7   비금융회사   신세계    동일인측        등기된임원        등기된임원  차정호      560   0.00      -   
8   비금융회사   신세계    동일인측        등기된임원        등기된임원  박주형      345   0.00      -   
9   비금융회사   신세계    동일인측         자기주식         자기주식  신세계     8774   0.09      -   
10  비금융회사   신세계    동일인측  계열회사(국내+해외)  계열회사(국내+해외)    -        -      -      -   
11  비금융회사   신세계    동일인측     기타동일인관련자     기타동일인관련자    -        -      -      -   
12  비금융회사   신세계    동일인측       동일인측합계       동일인측합계    /  2821718  28.66      -   
13  비금융회사   신세계    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주    -        -      -      -   
14  비금융회사   신세계    기타주주           기타           기타    /        -      -      -   
15  비금융회사   신세계      총계           총계           총계    /  9845181   100%      -   

   우선주지분율      합계주식수  합계지분율  ownCat  ownCat2  
0       -   984518.0  10.00     0.0      0.0  
1       -  1827521.0  18.56     NaN      NaN  
2       -        NaN      -     NaN      NaN  
3       -        NaN      -     NaN      NaN  
4       -        NaN      -     NaN      NaN  
5       -  1827521.0  18.56     0.0      0.0  
6       -        NaN      -     NaN      0.0  
7       -      560.0      -     NaN      0.0  
8       -      345.0      -     NaN      0.0  
9       -     8774.0    NaN     NaN      0.0  
10      -        NaN      -     1.0      1.0  
11      -        NaN      -     NaN      NaN  
12      -  2821718.0  28.66     NaN      NaN  
13      -        NaN      -    99.0     99.0  
14      -        NaN      -    99.0     99.0  
15   100%        NaN   100%     NaN      NaN

In [433]:
df.loc[15, '합계주식수'] = 9845181

In [434]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
신세계       0.286609
인천신세계    10.000000
Name: own2, dtype: float64

In [435]:
df[df['소속회사명'] == '인천신세계']

금융회사  소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3                     성명  \
46  비금융회사  인천신세계    동일인측          동일인          동일인                    이명희   
47  비금융회사  인천신세계    동일인측           친족     배우자/혈족1촌                      -   
48  비금융회사  인천신세계    동일인측           친족       혈족2~4촌                      -   
49  비금융회사  인천신세계    동일인측           친족       혈족5~6촌                      -   
50  비금융회사  인천신세계    동일인측           친족       인척4촌이내                      -   
51  비금융회사  인천신세계    동일인측           친족         친족합계                      /   
52  비금융회사  인천신세계    동일인측        비영리법인        비영리법인                      -   
53  비금융회사  인천신세계    동일인측        등기된임원        등기된임원                      -   
54  비금융회사  인천신세계    동일인측         자기주식         자기주식                      -   
55  비금융회사  인천신세계    동일인측  계열회사(국내+해외)  계열회사(국내+해외)                    신세계   
56  비금융회사  인천신세계    동일인측     기타동일인관련자     기타동일인관련자                      -   
57  비금융회사  인천신세계    동일인측       동일인측합계       동일인측합계                      /   
58  비금융회사  인천신세계    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주                      -   
59  비금융회사  인천신세계    기타주주           기타           기타  RECOSongdoPrivateLtd.   
60  비금융회사  인천신세계      총계           총계           총계                      /   

     보통주주식수 보통주지분율 우선주주식수 우선주지분율      합계주식수  합계지분율  ownCat  ownCat2  
46        -      -      -      -        NaN      -     0.0      0.0  
47        -      -      -      -        NaN      -     NaN      NaN  
48        -      -      -      -        NaN      -     NaN      NaN  
49        -      -      -      -        NaN      -     NaN      NaN  
50        -      -      -      -        NaN      -     NaN      NaN  
51        -      -      -      -        NaN      -     0.0      0.0  
52        -      -      -      -        NaN      -     NaN      0.0  
53        -      -      -      -        NaN      -     NaN      0.0  
54        -      -      -      -        NaN      -     NaN      0.0  
55  1494000  90.00      -      -  1494000.0  90.00     1.0      1.0  
56        -      -      -      -        NaN      -     NaN      NaN  
57  1494000  90.00      -      -  1494000.0  90.00     NaN      NaN  
58        -      -      -      -        NaN      -    99.0     99.0  
59   166000  10.00      -      -   166000.0  10.00    99.0     99.0  
60  1660000   100%      -      -   166000.0   100%     NaN      NaN

In [436]:
df.loc[60, '합계주식수'] = 1660000

In [437]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
신세계    0.286609
Name: own2, dtype: float64

In [438]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/신세계.xlsx", index=False)

# 씨제이

In [426]:
df = pd.read_excel('../data/ownership-status/raw/2021/CJ.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [427]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]


firmname
씨제이프레시웨이    0.890495
프레시원광주      1.000000
Name: own2, dtype: float64

In [428]:
df[df['소속회사명'] == '씨제이프레시웨이']

금융회사     소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3         성명    보통주주식수  \
462  비금융회사  씨제이프레시웨이    동일인측          동일인          동일인        이재현     70000   
463  비금융회사  씨제이프레시웨이    동일인측           친족     배우자/혈족1촌          -       NaN   
464  비금융회사  씨제이프레시웨이    동일인측           친족     배우자/혈족1촌          -       NaN   
465  비금융회사  씨제이프레시웨이    동일인측           친족       혈족2~4촌          -       NaN   
466  비금융회사  씨제이프레시웨이    동일인측           친족       혈족2~4촌          -       NaN   
467  비금융회사  씨제이프레시웨이    동일인측           친족       혈족5~6촌          -       NaN   
468  비금융회사  씨제이프레시웨이    동일인측           친족       혈족5~6촌          -       NaN   
469  비금융회사  씨제이프레시웨이    동일인측           친족       인척4촌이내          -       NaN   
470  비금융회사  씨제이프레시웨이    동일인측           친족       인척4촌이내          -       NaN   
471  비금융회사  씨제이프레시웨이    동일인측           친족         친족합계          -         -   
472  비금융회사  씨제이프레시웨이    동일인측        비영리법인        비영리법인          -       NaN   
473  비금융회사  씨제이프레시웨이    동일인측        비영리법인        비영리법인          -       NaN   
474  비금융회사  씨제이프레시웨이    동일인측        등기된임원        등기된임원        강연중      2000   
475  비금융회사  씨제이프레시웨이    동일인측        등기된임원        등기된임원          -       NaN   
476  비금융회사  씨제이프레시웨이    동일인측        등기된임원        등기된임원          -       NaN   
477  비금융회사  씨제이프레시웨이    동일인측         자기주식         자기주식          -       NaN   
478  비금융회사  씨제이프레시웨이    동일인측  계열회사(국내+해외)  계열회사(국내+해외)        씨제이   5592220   
479  비금융회사  씨제이프레시웨이    동일인측     기타동일인관련자     기타동일인관련자  EFGBANKAG   1300000   
480  비금융회사  씨제이프레시웨이    동일인측       동일인측합계       동일인측합계          -   6964220   
481  비금융회사  씨제이프레시웨이    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주          -       NaN   
482  비금융회사  씨제이프레시웨이    기타주주           기타           기타          -   4907366   
483  비금융회사  씨제이프레시웨이      총계           총계           총계          -  11871586   

     보통주지분율 우선주주식수  우선주지분율       합계주식수   합계지분율  ownCat  ownCat2  
462    0.59    NaN     0.0     70000.0    0.59     0.0      0.0  
463    0.00    NaN     0.0         NaN    0.00     NaN      NaN  
464    0.00    NaN     0.0         NaN    0.00     NaN      NaN  
465    0.00    NaN     0.0         NaN    0.00     NaN      NaN  
466    0.00    NaN     0.0         NaN    0.00     NaN      NaN  
467    0.00    NaN     0.0         NaN    0.00     NaN      NaN  
468    0.00    NaN     0.0         NaN    0.00     NaN      NaN  
469    0.00    NaN     0.0         NaN    0.00     NaN      NaN  
470    0.00    NaN     0.0         NaN    0.00     NaN      NaN  
471    0.00      -     0.0         NaN    0.00     0.0      0.0  
472    0.00    NaN     0.0         NaN    0.00     NaN      0.0  
473    0.00    NaN     0.0         NaN    0.00     NaN      0.0  
474    0.02    NaN     0.0      2000.0    0.02     NaN      0.0  
475    0.00    NaN     0.0         NaN    0.00     NaN      0.0  
476    0.00    NaN     0.0         NaN    0.00     NaN      0.0  
477    0.00    NaN     0.0         NaN    0.00     NaN      0.0  
478   47.11    NaN     0.0   5592220.0   47.11     1.0      1.0  
479   10.95    NaN     0.0   1300000.0   10.95     NaN      NaN  
480   58.66      -     0.0   6964220.0   58.66     NaN      NaN  
481    0.00    NaN     0.0         NaN    0.00    99.0     99.0  
482   41.34    NaN     0.0   4907366.0   41.34    99.0     99.0  
483  100.00    NaN     0.0  11871586.0  100.00     NaN      NaN

In [429]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/CJ.xlsx", index=False)

# 한진

In [420]:
df = pd.read_excel('../data/ownership-status/raw/2021/대한항공.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [421]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
대한항공        1.000088
세계혼재항공화물    2.000000
Name: own2, dtype: float64

In [422]:
df[df['소속회사명'] == '세계혼재항공화물'] 

금융회사     소속회사명 동일인과의관계 동일인과의관계2 동일인과의관계3   성명  보통주주식수 보통주지분율 우선주주식수  \
367  비금융회사  세계혼재항공화물    동일인측       친족   혈족2~4촌  이상영   30000    30%      -   
368  비금융회사  세계혼재항공화물    동일인측       친족   인척4촌이내  손희종   30000    30%      -   
369  비금융회사  세계혼재항공화물    동일인측       친족     친족합계    /   60000    60%      -   
370  비금융회사  세계혼재항공화물    동일인측    등기된임원    등기된임원  김태익   30000    30%      -   
371  비금융회사  세계혼재항공화물    동일인측    등기된임원    등기된임원  조영우   10000    10%      -   
372  비금융회사  세계혼재항공화물    동일인측    등기된임원    등기된임원    /  100000   100%      -   
373  비금융회사  세계혼재항공화물      총계       총계       총계    /  100000   100%      -   

    우선주지분율     합계주식수 합계지분율  ownCat  ownCat2  
367      -   30000.0   30%     NaN      NaN  
368      -   30000.0   30%     NaN      NaN  
369      -   60000.0   60%     0.0      0.0  
370      -   30000.0   30%     NaN      0.0  
371      -   10000.0   10%     NaN      0.0  
372      -  100000.0  100%     NaN      0.0  
373      -  100000.0  100%     NaN      NaN

In [423]:
df.loc[372, '합계주식수'] = 0

In [424]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
대한항공    1.000088
Name: own2, dtype: float64

In [425]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/한진.xlsx", index=False)

# 두산

In [416]:
df = pd.read_excel('../data/ownership-status/raw/2021/두산.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [417]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [418]:
dfOwn

firmname  type       own      own2 ownername
0     네오트랜스   0.0  0.000000  0.000000       NaN
1     네오트랜스   1.0  0.428600  0.428600      두산건설
2     네오트랜스  99.0  0.571400  0.571400       NaN
3        두산   0.0  0.369954  0.615105       NaN
4        두산   1.0       NaN       NaN         -
..      ...   ...       ...       ...       ...
71   이큐브솔루션   1.0  0.600000  0.600000   두산인프라코어
72   이큐브솔루션  99.0  0.400000  0.400000       NaN
73       한컴   0.0  0.000000  0.000000       NaN
74       한컴   1.0  1.000000  1.000000       오리콤
75       한컴  99.0  0.000000  0.000000       NaN

[76 rows x 5 columns]

In [419]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/두산.xlsx", index=False)

# 엘에스

In [410]:
df = pd.read_excel('../data/ownership-status/raw/2021/LS.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [411]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
지엘마린    0.0
Name: own2, dtype: float64

In [412]:
df[df['소속회사명'] == '지엘마린']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3    성명    보통주주식수 보통주지분율  \
340  비금융회사  지엘마린    동일인측          동일인          동일인     -         -      -   
341  비금융회사  지엘마린    동일인측           친족     배우자/혈족1촌     -         -      -   
342  비금융회사  지엘마린    동일인측           친족     배우자/혈족1촌     -         -      -   
343  비금융회사  지엘마린    동일인측           친족       혈족2~4촌     -         -      -   
344  비금융회사  지엘마린    동일인측           친족       혈족2~4촌     -         -      -   
345  비금융회사  지엘마린    동일인측           친족       혈족5~6촌     -         -      -   
346  비금융회사  지엘마린    동일인측           친족       혈족5~6촌     -         -      -   
347  비금융회사  지엘마린    동일인측           친족       인척4촌이내     -         -      -   
348  비금융회사  지엘마린    동일인측           친족       인척4촌이내     -         -      -   
349  비금융회사  지엘마린    동일인측           친족         친족합계     -         -      -   
350  비금융회사  지엘마린    동일인측        비영리법인        비영리법인     -         -      -   
351  비금융회사  지엘마린    동일인측        등기된임원        등기된임원     -         -      -   
352  비금융회사  지엘마린    동일인측         자기주식         자기주식     -         -      -   
353  비금융회사  지엘마린    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  LS전선  33202000   100%   
354  비금융회사  지엘마린    동일인측     기타동일인관련자     기타동일인관련자     -         -      -   
355  비금융회사  지엘마린    동일인측       동일인측합계       동일인측합계  LS전선  33202000   100%   
356  비금융회사  지엘마린    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주     -         -      -   
357  비금융회사  지엘마린    기타주주           기타           기타     -         -      -   
358  비금융회사  지엘마린      총계           총계           총계     /  33202000   100%   

    우선주주식수 우선주지분율       합계주식수 합계지분율  ownCat  ownCat2  
340      -      -         NaN     _     0.0      0.0  
341      -      -         NaN     _     NaN      NaN  
342      -      -         NaN     _     NaN      NaN  
343      -      -         NaN     _     NaN      NaN  
344      -      -         NaN     _     NaN      NaN  
345      -      -         NaN     _     NaN      NaN  
346      -      -         NaN     _     NaN      NaN  
347      -      -         NaN     _     NaN      NaN  
348      -      -         NaN     _     NaN      NaN  
349      -      -         NaN     _     0.0      0.0  
350      -      -         NaN     _     NaN      0.0  
351      -      -         NaN     _     NaN      0.0  
352      -      -         NaN     _     NaN      0.0  
353      -      -  33202000.0  100%     1.0      1.0  
354      -      -         NaN     _     NaN      NaN  
355      -      -  33202000.0  100%     NaN      NaN  
356      -      -         NaN     _    99.0     99.0  
357      -      -         NaN     _    99.0     99.0  
358      -   100%         NaN  100%     NaN      NaN

In [413]:
df.loc[358, '합계주식수'] = 33202000

In [414]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [415]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/LS.xlsx", index=False)

# 부영

In [406]:
df = pd.read_excel('../data/ownership-status/raw/2021/부영.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [407]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [408]:
dfOwn

firmname  type       own      own2 ownername
0      광영토건   0.0  0.511611  1.000000       NaN
1      광영토건   1.0       NaN       NaN         -
2      광영토건  99.0  0.000000  0.000000       NaN
3    남광건설산업   0.0  1.000000  1.000000       NaN
4    남광건설산업   1.0       NaN       NaN         -
..      ...   ...       ...       ...       ...
64   천원종합개발   1.0  0.995495  0.995495      부영주택
65   천원종합개발  99.0  0.004505  0.004505       NaN
66    한라일보사   0.0  0.489999  0.489999       NaN
67    한라일보사   1.0       NaN       NaN         -
68    한라일보사  99.0  0.510001  0.510001       NaN

[69 rows x 5 columns]

In [409]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/부영.xlsx", index=False)

# 카카오

In [400]:
df = pd.read_excel('../data/ownership-status/raw/2021/카카오.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [401]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
손노리          1.000000
스타쉽엔터테인먼트    1.000000
카카오          1.153696
Name: own2, dtype: float64

In [402]:
df[df['소속회사명'] == '카카오']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3   성명     보통주주식수  보통주지분율  \
0   비금융회사   카카오    동일인측          동일인          동일인  김범수   59124547   13.32   
1   비금융회사   카카오    동일인측           친족     배우자/혈족1촌  형미선     282000    0.06   
2   비금융회사   카카오    동일인측           친족     배우자/혈족1촌  김상빈     270000    0.06   
3   비금융회사   카카오    동일인측           친족     배우자/혈족1촌  김예빈     268000    0.06   
4   비금융회사   카카오    동일인측           친족       혈족2~4촌  김명희      90000    0.02   
..    ...   ...     ...          ...          ...  ...        ...     ...   
71  비금융회사   카카오    동일인측     기타동일인관련자     기타동일인관련자    -          -       -   
72  비금융회사   카카오    동일인측       동일인측합계       동일인측합계    /  120172504   27.08   
73  비금융회사   카카오    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주    -          -       -   
74  비금융회사   카카오    기타주주           기타           기타    /  323636801   72.92   
75  비금융회사   카카오      총계           총계           총계    /  443809305  100.00   

   우선주주식수 우선주지분율        합계주식수   합계지분율  ownCat  ownCat2  
0       -      -   59124547.0   13.32     0.0      0.0  
1       -      -     282000.0    0.06     NaN      NaN  
2       -      -     270000.0    0.06     NaN      NaN  
3       -      -     268000.0    0.06     NaN      NaN  
4       -      -      90000.0    0.02     NaN      NaN  
..    ...    ...          ...     ...     ...      ...  
71      -      -          NaN       -     NaN      NaN  
72      -      -   61047957.0   27.08     NaN      NaN  
73      -      -          NaN       -    99.0     99.0  
74      -      -  323636801.0   72.92    99.0     99.0  
75      -      -  384684758.0  100.00     NaN      NaN  

[76 rows x 14 columns]

In [403]:
df.loc[75, '합계주식수'] = 443809305

In [404]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
손노리          1.0
스타쉽엔터테인먼트    1.0
Name: own2, dtype: float64

In [405]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/카카오.xlsx", index=False)

# DL

In [397]:
df = pd.read_excel('../data/ownership-status/raw/2021/DL.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [398]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
디엘건설    1.000573
Name: own2, dtype: float64

In [399]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/DL.xlsx", index=False)

# 미래에셋

In [390]:
df = pd.read_excel('../data/ownership-status/raw/2021/미래에셋캐피탈.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [391]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
미래에셋생명보험            1.000002
미래에셋증권글로벌제이호사모투자    0.000000
서울공항리무진             1.000000
수원학교사랑              0.850000
신성장동력그린퓨처사모투자       0.000000
Name: own2, dtype: float64

In [392]:
df[df['소속회사명'] == '수원학교사랑']

금융회사   소속회사명 동일인과의관계     동일인과의관계2 동일인과의관계3          성명   보통주주식수 보통주지분율  \
370  비금융회사  수원학교사랑    동일인측  계열회사(국내+해외)      NaN    미래에셋생명보험      NaN    NaN   
371  비금융회사  수원학교사랑    동일인측       동일인측합계      NaN           /      NaN    NaN   
372  비금융회사  수원학교사랑    기타주주           기타      NaN      덕동종합건설  1323956    17%   
373  비금융회사  수원학교사랑    기타주주           기타      NaN  에이치디에스자산관리  2647911    34%   
374  비금융회사  수원학교사랑    기타주주           기타      NaN        영동건설  1323956    17%   
375  비금융회사  수원학교사랑    기타주주           기타      NaN        한동건설  2647911    34%   
376  비금융회사  수원학교사랑      총계           총계      NaN           /  7943734   100%   

       우선주주식수 우선주지분율       합계주식수 합계지분율  ownCat  ownCat2  
370  45014493   100%  45014493.0  100%     1.0      1.0  
371  45014493   100%  45014493.0  100%     NaN      NaN  
372       NaN    NaN         NaN   NaN    99.0     99.0  
373       NaN    NaN         NaN   NaN    99.0     99.0  
374       NaN    NaN         NaN   NaN    99.0     99.0  
375       NaN    NaN         NaN   NaN    99.0     99.0  
376  45014493   100%  52958227.0  100%     NaN      NaN

In [393]:
df.loc[372, '합계주식수'] = 1323956
df.loc[373, '합계주식수'] = 2647911
df.loc[374, '합계주식수'] = 1323956
df.loc[375, '합계주식수'] = 2647911

In [394]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
미래에셋생명보험            1.000002
미래에셋증권글로벌제이호사모투자    0.000000
서울공항리무진             1.000000
신성장동력그린퓨처사모투자       0.000000
Name: own2, dtype: float64

In [395]:
df[df['소속회사명'] == '신성장동력그린퓨처사모투자']

금융회사          소속회사명 동일인과의관계     동일인과의관계2 동일인과의관계3          성명 보통주주식수  \
308  금융회사  신성장동력그린퓨처사모투자    동일인측  계열회사(국내+해외)      NaN      미래에셋증권      -   
309  금융회사  신성장동력그린퓨처사모투자    동일인측       동일인측합계      NaN           /      -   
310  금융회사  신성장동력그린퓨처사모투자    기타주주  동일인측이아닌최다주주      NaN      한국산업은행      -   
311  금융회사  신성장동력그린퓨처사모투자    기타주주           기타      NaN  화이텍인베스트먼트외      -   
312  금융회사  신성장동력그린퓨처사모투자      총계           총계      NaN           /      -   

    보통주지분율 우선주주식수 우선주지분율  합계주식수 합계지분율  ownCat  ownCat2  
308      -      -      -    NaN     -     1.0      1.0  
309      -      -      -    NaN     -     NaN      NaN  
310      -      -      -    NaN     -    99.0     99.0  
311      -      -      -    NaN     -    99.0     99.0  
312      -      -      -    NaN     -     NaN      NaN

In [396]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/미래에셋.xlsx", index=False)

# 현대백화점

In [386]:
df = pd.read_excel('../data/ownership-status/raw/2021/현대백화점.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [388]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
한섬라이프앤    1.0
Name: own2, dtype: float64

In [389]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/현대백화점.xlsx", index=False)

# 금호아시아나

In [382]:
df = pd.read_excel('../data/ownership-status/raw/2021/금호건설.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [383]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [384]:
dfOwn

firmname  type       own      own2 ownername
0      금호건설   0.0  0.003352  0.032614       NaN
1      금호건설   1.0  0.442107  0.442107      금호고속
2      금호건설  99.0  0.525279  0.525279       NaN
3      금호고속   0.0  0.688887  0.917037       NaN
4      금호고속   1.0  0.020741  0.020741      케이에이
..      ...   ...       ...       ...       ...
75     케이에프  99.0  0.000000  0.000000       NaN
76      케이오   0.0  0.000000  1.000000       NaN
77      케이오  99.0  0.000000  0.000000       NaN
78      케이지   0.0  0.000000  1.000000       NaN
79      케이지  99.0  0.000000  0.000000       NaN

[80 rows x 5 columns]

In [385]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/금호아시아나.xlsx", index=False)

# 셀트리온

In [378]:
df = pd.read_excel('../data/ownership-status/raw/2021/셀트리온홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [379]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
셀트리온홀딩스    1.0
Name: own2, dtype: float64

In [380]:
dfOwn

firmname  type       own      own2    ownername
0          셀트리온   0.0  0.001431  0.014254          NaN
1          셀트리온   1.0  0.200107  0.200107      셀트리온홀딩스
2          셀트리온   1.0  0.021158  0.021158     셀트리온스킨큐어
3          셀트리온  99.0  0.764481  0.764481          NaN
4      셀트리온스킨큐어   0.0  0.742452  0.828185          NaN
5      셀트리온스킨큐어  99.0  0.171815  0.171815          NaN
6    셀트리온엔터테인먼트   1.0  1.000000  1.000000      셀트리온홀딩스
7    셀트리온엔터테인먼트  99.0  0.000000  0.000000          NaN
8        셀트리온제약   0.0  0.000149  0.002757          NaN
9        셀트리온제약   1.0  0.549327  0.549327         셀트리온
10       셀트리온제약  99.0  0.447916  0.447916          NaN
11     셀트리온헬스케어   0.0  0.113945  0.135126          NaN
12     셀트리온헬스케어   1.0  0.013851  0.013851     셀트리온스킨큐어
13     셀트리온헬스케어   1.0  0.242964  0.242964  셀트리온헬스케어홀딩스
14     셀트리온헬스케어  99.0  0.608059  0.608059          NaN
15  셀트리온헬스케어홀딩스   0.0  1.000000  1.000000          NaN
16  셀트리온헬스케어홀딩스  99.0  0.000000  0.000000          NaN
17      셀트리온홀딩스   0.0  0.961140  0.977092          NaN
18      셀트리온홀딩스  99.0  0.022908  0.022908          NaN
19       티에스이엔씨   0.0  1.000000  1.000000          NaN
20       티에스이엔씨  99.0  0.000000  0.000000          NaN

In [381]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/셀트리온.xlsx", index=False)

# 한국투자금융

In [372]:
df = pd.read_excel('../data/ownership-status/raw/2021/한국금융지주.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [373]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
이큐파트너스제1호환경서비스사모투자            1.000000
케이엔반도체투자                      1.000000
한국투자글로벌제약산업육성사모투자             1.000000
한국투자온기업그린모빌리티제1호기업재무안정사모투자    0.898182
Name: own2, dtype: float64

In [374]:
df[df['소속회사명'] == '한국투자온기업그린모빌리티제1호기업재무안정사모투자']

금융회사                       소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3  \
306  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측          동일인          동일인   
307  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측           친족     배우자/혈족1촌   
308  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측           친족       혈족2~4촌   
309  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측           친족       혈족5~6촌   
310  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측           친족       인척4촌이내   
311  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측           친족         친족합계   
312  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측        비영리법인        비영리법인   
313  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측        등기된임원        등기된임원   
314  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측         자기주식         자기주식   
315  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측  계열회사(국내+해외)  계열회사(국내+해외)   
316  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측          NaN          NaN   
317  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측     기타동일인관련자     기타동일인관련자   
318  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    동일인측       동일인측합계       동일인측합계   
319  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주   
320  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자    기타주주           기타           기타   
321  금융회사  한국투자온기업그린모빌리티제1호기업재무안정사모투자      총계           총계           총계   

              성명       보통주주식수  보통주지분율 우선주주식수 우선주지분율         합계주식수   합계지분율  \
306            -            -       -      -      -           NaN       -   
307            -            -       -      -      -           NaN       -   
308            -            -       -      -      -           NaN       -   
309            -            -       -      -      -           NaN       -   
310            -            -       -      -      -           NaN       -   
311            /            -       -      -      -           NaN       -   
312            -            -       -      -      -           NaN       -   
313            -            -       -      -      -           NaN       -   
314            -            -       -      -      -           NaN       -   
315       한국투자증권  10000000000   18.18      -      -  1.000000e+10   18.18   
316  한국투자프라이빗에쿼티   5600000000   10.18      -      -  5.600000e+09   10.18   
317            -            -       -      -      -           NaN       -   
318            /  15600000000   28.36      -      -  1.560000e+10   28.36   
319     한국자산관리공사  23000000000   41.82      -      -  2.300000e+10   41.82   
320            /  16400000000   29.82      -      -  1.640000e+10   29.82   
321            /  55000000000  100.00      -      -  5.500000e+10  100.00   

     ownCat  ownCat2  
306     0.0      0.0  
307     NaN      NaN  
308     NaN      NaN  
309     NaN      NaN  
310     NaN      NaN  
311     0.0      0.0  
312     NaN      0.0  
313     NaN      0.0  
314     NaN      0.0  
315     1.0      1.0  
316     NaN      NaN  
317     NaN      NaN  
318     NaN      NaN  
319    99.0     99.0  
320    99.0     99.0  
321     NaN      NaN

In [375]:
df.loc[316, '동일인과의관계2'] = '계열회사(국내+해외)'

In [376]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
이큐파트너스제1호환경서비스사모투자    1.0
케이엔반도체투자              1.0
한국투자글로벌제약산업육성사모투자     1.0
Name: own2, dtype: float64

In [377]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/한국투자금융.xlsx", index=False)

# 교보생명보험

In [368]:
df = pd.read_excel('../data/ownership-status/raw/2021/교보생명보험.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [369]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [370]:
dfOwn

firmname  type       own      own2  ownername
0   교보라이프플래닛생명보험   0.0  0.000000  0.000000        NaN
1   교보라이프플래닛생명보험   1.0  1.000000  1.000000     교보생명보험
2   교보라이프플래닛생명보험  99.0  0.000000  0.000000        NaN
3          교보리얼코   0.0  0.000000  0.000000        NaN
4          교보리얼코   1.0  1.000000  1.000000     교보생명보험
5          교보리얼코  99.0  0.000000  0.000000        NaN
6           교보문고   0.0  0.000000  0.000000        NaN
7           교보문고   1.0  1.000000  1.000000     교보생명보험
8           교보문고  99.0  0.000000  0.000000        NaN
9         교보생명보험   0.0  0.369047  0.369060        NaN
10        교보생명보험   1.0       NaN       NaN          -
11        교보생명보험  99.0  0.630940  0.630940        NaN
12      교보악사자산운용   0.0  0.000000  0.000000        NaN
13      교보악사자산운용   1.0  0.500000  0.500000     교보생명보험
14      교보악사자산운용  99.0  0.500000  0.500000        NaN
15        교보자산신탁   0.0  0.000000  0.000000        NaN
16        교보자산신탁   1.0  1.000000  1.000000     교보생명보험
17        교보자산신탁  99.0  0.000000  0.000000        NaN
18        교보정보통신   0.0  0.000000  0.000000        NaN
19        교보정보통신   1.0  1.000000  1.000000     교보생명보험
20        교보정보통신  99.0  0.000000  0.000000        NaN
21          교보증권   0.0  0.000079  0.017149        NaN
22          교보증권   1.0  0.730647  0.730647     교보생명보험
23          교보증권  99.0  0.252204  0.252204        NaN
24        교보핫트랙스   0.0  0.000000  0.000000        NaN
25        교보핫트랙스   1.0  1.000000  1.000000       교보문고
26        교보핫트랙스  99.0  0.000000  0.000000        NaN
27     제일종합관리서비스   0.0  0.000000  0.000000        NaN
28     제일종합관리서비스   1.0  1.000000  1.000000      교보리얼코
29     제일종합관리서비스  99.0  0.000000  0.000000        NaN
30      케이씨에이서비스   0.0  0.000000  0.000000        NaN
31      케이씨에이서비스   1.0  0.666667  0.666667     교보생명보험
32      케이씨에이서비스   1.0  0.333333  0.333333  케이씨에이손해사정
33      케이씨에이서비스  99.0  0.000000  0.000000        NaN
34     케이씨에이손해사정   0.0  0.000000  0.000000        NaN
35     케이씨에이손해사정   1.0  1.000000  1.000000     교보생명보험
36     케이씨에이손해사정  99.0  0.000000  0.000000        NaN

In [371]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/교보생명보험.xlsx", index=False)

# 네이버

In [365]:
df = pd.read_excel('../data/ownership-status/raw/2021/NAVER.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [366]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
네이버    0.999999
Name: own2, dtype: float64

In [367]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/NAVER.xlsx", index=False)

# HDC

In [361]:
df = pd.read_excel('../data/ownership-status/raw/2021/HDC.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [362]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
에이치디씨현대산업개발    1.0
Name: own2, dtype: float64

In [363]:
dfOwn

firmname  type       own      own2    ownername
0   광명문화복합단지자산관리   0.0  0.000000  0.000000          NaN
1   광명문화복합단지자산관리   1.0  0.600000  0.600000  에이치디씨현대산업개발
2   광명문화복합단지자산관리  99.0  0.400000  0.400000          NaN
3          미래비아이   0.0  0.000000  0.000000          NaN
4          미래비아이   1.0  0.644494  0.644494       부동산일일사
..           ...   ...       ...       ...          ...
82        통영에코파워   1.0  1.000000  1.000000        에이치디씨
83        통영에코파워  99.0  0.000000  0.000000          NaN
84       호텔에이치디씨   0.0  0.000000  0.000000          NaN
85       호텔에이치디씨   1.0  1.000000  1.000000  에이치디씨현대산업개발
86       호텔에이치디씨  99.0  0.000000  0.000000          NaN

[87 rows x 5 columns]

In [364]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/HDC.xlsx", index=False)

# 효성

In [358]:
df = pd.read_excel('../data/ownership-status/raw/2021/효성.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [359]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
효성티앤에스    1.0
Name: own2, dtype: float64

In [360]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/효성.xlsx", index=False)

# 영풍

In [354]:
df = pd.read_excel('../data/ownership-status/raw/2021/영풍.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [355]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [356]:
dfOwn

firmname  type       own      own2 ownername
0      고려아연   0.0  0.053914  0.184120       NaN
1      고려아연   1.0  0.274870  0.274870        영풍
2      고려아연   1.0  0.015639  0.015639      영풍정밀
3      고려아연   1.0  0.009889  0.009889      유미개발
4      고려아연   1.0  0.001125  0.001125      테라닉스
..      ...   ...       ...       ...       ...
93    클린코리아  99.0  0.000000  0.000000       NaN
94     테라닉스   0.0  0.005357  0.029196       NaN
95     테라닉스   1.0  0.500935  0.500935    코리아써키트
96     테라닉스   1.0  0.416238  0.416238        영풍
97     테라닉스  99.0  0.053631  0.053631       NaN

[98 rows x 5 columns]

In [357]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/영풍.xlsx", index=False)

# 하림

In [349]:
df = pd.read_excel('../data/ownership-status/raw/2021/하림지주.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [350]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [351]:
dfOwn

firmname  type  own  own2 ownername
0          경우   0.0  0.8   0.8       NaN
1          경우   1.0  0.2   0.2        올품
2          경우   1.0  NaN   NaN         -
3          경우  99.0  0.0   0.0       NaN
4        글라이드   0.0  0.0   0.0       NaN
..        ...   ...  ...   ...       ...
217      한국썸벧  99.0  0.0   0.0       NaN
218  한국인베스트먼트   0.0  0.0   0.0       NaN
219  한국인베스트먼트   1.0  1.0   1.0        올품
220  한국인베스트먼트   1.0  NaN   NaN         -
221  한국인베스트먼트  99.0  0.0   0.0       NaN

[222 rows x 5 columns]

In [352]:
dfOwn.firmname.unique()

array(['경우', '글라이드', '농업회사법인늘푸른', '농업회사법인봉산지피', '농업회사법인봉화', '농업회사법인비앤피',
       '농업회사법인선진한마을', '농업회사법인순우리한우', '농업회사법인에이치비씨', '농업회사법인익산',
       '농업회사법인주원산오리', '농업회사법인팜스코바이오인티', '농업회사법인팜엔코', '농업회사법인피그앤그린',
       '농업회사법인하이포크스마트팜', '동림', '디디에프엔비', '맥시칸', '선진', '선진브릿지랩', '선진에프에스',
       '선진팜', '선진햄', '세티', '싱그린에프에스', '애그리로보텍', '에버미라클', '에이치에스푸드',
       '에코캐피탈', '엑셀로이콰인알앤디센터', '엔디', '엔바이콘', '엔에스쇼핑', '엔에스홈쇼핑미디어센터', '올품',
       '제이에이치제이', '제일사료', '지포레', '참트레이딩', '청운농업회사법인', '팜스코', '팜스코푸드',
       '팬오션', '포스에스엠', '하림', '하림산업', '하림애니멀클리닉', '하림엠에스', '하림유통', '하림지주',
       '하림펫푸드', '하림푸드', '한강씨엠', '한국썸벧', '한국인베스트먼트'], dtype=object)

In [353]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/하림.xlsx", index=False)

# 케이씨씨

In [344]:
df = pd.read_excel('../data/ownership-status/raw/2021/케이씨씨.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [345]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
-       0.0
금강레저    1.0
Name: own2, dtype: float64

In [346]:
df = df[~(df['소속회사명'] == '-')]

In [347]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
금강레저    1.0
Name: own2, dtype: float64

In [348]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/케이씨씨.xlsx", index=False)

# 엔엑스씨

In [338]:
df = pd.read_excel('../data/ownership-status/raw/2021/엔엑스씨.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [339]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
엔엑스씨    1.674918
Name: own2, dtype: float64

In [340]:
df[df['소속회사명'] == '엔엑스씨']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3           성명   보통주주식수  \
0  비금융회사  엔엑스씨    동일인측          동일인          동일인          김정주  1963000   
1  비금융회사  엔엑스씨    동일인측           친족     배우자/혈족1촌          유정현   856000   
2  비금융회사  엔엑스씨    동일인측           친족     배우자/혈족1촌          김정민    19750   
3  비금융회사  엔엑스씨    동일인측           친족     배우자/혈족1촌          김정윤    19750   
4  비금융회사  엔엑스씨    동일인측           친족         친족합계            /  2858500   
5  비금융회사  엔엑스씨    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  유한책임회사와이즈키즈    50000   
6  비금융회사  엔엑스씨    동일인측       동일인측합계       동일인측합계            /  2908500   
7  비금융회사  엔엑스씨      총계           총계           총계            /  2908500   

    보통주지분율 우선주주식수 우선주지분율      합계주식수    합계지분율  ownCat  ownCat2  
0   67.49%      -      -  1963000.0   67.49%     0.0      0.0  
1   29.43%      -      -   856000.0   29.43%     NaN      NaN  
2    0.68%      -      -    19750.0    0.68%     NaN      NaN  
3    0.68%      -      -    19750.0    0.68%     NaN      NaN  
4   98.28%      -      -  2858500.0   98.28%     0.0      0.0  
5    1.72%      -      -    50000.0    1.72%     1.0      1.0  
6  100.00%      -      -  2908500.0  100.00%     NaN      NaN  
7  100.00%      -      -  2908500.0  100.00%     NaN      NaN

In [341]:
df.loc[4, '합계주식수'] = 2858500.0 - 1963000.0

In [342]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [343]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/엔엑스씨.xlsx", index=False)

# 넷마블

In [332]:
df = pd.read_excel('../data/ownership-status/raw/2021/넷마블.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [333]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
넷마블엔투    1.0
잼팟       1.0
Name: own2, dtype: float64

In [334]:
df[df['소속회사명'] == '넷마블에프앤씨']

금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3       성명    보통주주식수  \
153  비금융회사  넷마블에프앤씨    동일인측          동일인          동일인        -         -   
154  비금융회사  넷마블에프앤씨    동일인측           친족     배우자/혈족1촌        -         -   
155  비금융회사  넷마블에프앤씨    동일인측           친족       혈족2~4촌        -         -   
156  비금융회사  넷마블에프앤씨    동일인측           친족       혈족5~6촌        -         -   
157  비금융회사  넷마블에프앤씨    동일인측           친족       인척4촌이내        -         -   
158  비금융회사  넷마블에프앤씨    동일인측           친족         친족합계        -         -   
159  비금융회사  넷마블에프앤씨    동일인측        비영리법인        비영리법인        -         -   
160  비금융회사  넷마블에프앤씨    동일인측        등기된임원        등기된임원      서우원    252335   
161  비금융회사  넷마블에프앤씨    동일인측        등기된임원        등기된임원      정철호    102755   
162  비금융회사  넷마블에프앤씨    동일인측         자기주식         자기주식  넷마블에프앤씨    263005   
163  비금융회사  넷마블에프앤씨    동일인측  계열회사(국내+해외)  계열회사(국내+해외)      넷마블  10260155   
164  비금융회사  넷마블에프앤씨    동일인측     기타동일인관련자     기타동일인관련자        -         -   
165  비금융회사  넷마블에프앤씨    동일인측       동일인측합계       동일인측합계        -  10878250   
166  비금융회사  넷마블에프앤씨      기타  동일인측이아닌최다주주  동일인측이아닌최다주주        -         -   
167  비금융회사  넷마블에프앤씨      기타           기타           기타        -   1499280   
168  비금융회사  넷마블에프앤씨      총계           총계           총계      NaN  12377530   

     보통주지분율 우선주주식수 우선주지분율       합계주식수   합계지분율  ownCat  ownCat2  
153       -      -      -         NaN       -     0.0      0.0  
154       -      -      -         NaN       -     NaN      NaN  
155       -      -      -         NaN       -     NaN      NaN  
156       -      -      -         NaN       -     NaN      NaN  
157       -      -      -         NaN       -     NaN      NaN  
158       -      -      -         NaN       -     0.0      0.0  
159       -      -      -         NaN       -     NaN      0.0  
160    2.04      -      -    252335.0    2.04     NaN      0.0  
161    0.83    NaN    NaN    102755.0    0.83     NaN      0.0  
162    2.12      -      -    263005.0    2.12     NaN      0.0  
163   82.89      -      -  10260155.0   82.89     1.0      1.0  
164       -      -      -         NaN       -     NaN      NaN  
165   87.89      -      -  10878250.0   87.89     NaN      NaN  
166       -      -      -         NaN       -    99.0     99.0  
167   12.11      -      -   1499280.0   12.11    99.0     99.0  
168  100.00      -      -  12377530.0  100.00     NaN      NaN

In [335]:
def catOwn(row):
    if row['동일인과의관계2'] == '동일인':
        return 0
    elif row['동일인과의관계2'] == '친족' and row['동일인과의관계3'] == '친족합계':
        return 0
    elif row['동일인과의관계2'] in ['계열회사(국내+해외)', '계열회사', '계열회사(국내)', '해외계열']:
        return 1
    elif row['동일인과의관계'] in ['기타주주', '기타']:
        return 99

def catOwn2(row):
    if row['동일인과의관계2'] in ['동일인', '비영리법인', '등기된임원', '자기주식']:
        return 0
    elif row['동일인과의관계2'] == '친족' and row['동일인과의관계3'] == '친족합계':
        return 0
    elif row['동일인과의관계2'] in ['계열회사(국내+해외)', '계열회사', '계열회사(국내)', '해외계열']:
        return 1
    elif row['동일인과의관계'] in ['기타주주', '기타']:
        return 99

In [336]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
넷마블엔투    1.0
잼팟       1.0
Name: own2, dtype: float64

In [337]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/넷마블.xlsx", index=False)

# 호반건설

In [329]:
df = pd.read_excel('../data/ownership-status/raw/2021/호반건설.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [330]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
마륵파크     1.0
호반서서울    1.0
Name: own2, dtype: float64

In [331]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/호반건설.xlsx", index=False)

# SM

In [326]:
df = pd.read_excel('../data/ownership-status/raw/2021/티케이케미칼.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [327]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼환기업      1.0
에스엠중공업    1.0
Name: own2, dtype: float64

In [328]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/SM.xlsx", index=False)

# DB

In [323]:
df = pd.read_excel('../data/ownership-status/raw/2021/DB.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [324]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
DB금융투자    1.0
Name: own2, dtype: float64

In [325]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/DB.xlsx", index=False)

# 코오롱

In [319]:
df = pd.read_excel('../data/ownership-status/raw/2021/코오롱.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [320]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [321]:
dfOwn

firmname  type       own      own2 ownername
0      그린나래   1.0  1.000000  1.000000    코오롱글로텍
1    네이처브리지   1.0  1.000000  1.000000    코오롱글로벌
2   더블유파트너스   0.0  1.000000  1.000000       NaN
3      덕평랜드   1.0  0.510000  0.510000    코오롱글로벌
4      덕평랜드  99.0  0.490000  0.490000       NaN
..      ...   ...       ...       ...       ...
60     테크비전   1.0  1.000000  1.000000    코오롱글로벌
61    퍼플아이오   1.0  0.649995  0.649995  코오롱인더스트리
62    퍼플아이오  99.0  0.350005  0.350005       NaN
63   한중문화타운   1.0  0.700000  0.700000    코오롱글로벌
64   한중문화타운  99.0  0.300000  0.300000       NaN

[65 rows x 5 columns]

In [322]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/코오롱.xlsx", index=False)

# 한국타이어

In [307]:
df = pd.read_excel('../data/ownership-status/raw/2021/한국앤컴퍼니.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [308]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
농업회사법인장지       0.100000
신양월드레저         1.200000
한국타이어앤테크놀로지    1.056669
Name: own2, dtype: float64

In [309]:
df[df['소속회사명']=='농업회사법인장지']

금융회사     소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3      성명  보통주주식수 보통주지분율 우선주주식수  \
110  비금융회사  농업회사법인장지    동일인측       친족  배우자/혈족1촌     조현식    9000    45%      -   
111  비금융회사  농업회사법인장지    동일인측       친족  배우자/혈족1촌     조현범    9000    45%      -   
112  비금융회사  농업회사법인장지    동일인측     친족합계      친족합계    친족합계   18000    90%      -   
113  비금융회사  농업회사법인장지    동일인측   동일인측합계    동일인측합계  동일인측합계   18000    90%      -   
114  비금융회사  농업회사법인장지    기타주주       기타        기타       -    2000    10%      -   
115  비금융회사  농업회사법인장지      총계       총계        총계      총계   20000   100%      -   

    우선주지분율    합계주식수 합계지분율  ownCat  ownCat2  
110      -   9000.0   45%     NaN      NaN  
111      -   9000.0   45%     NaN      NaN  
112      -  18000.0   90%     NaN      NaN  
113      -  18000.0   90%     NaN      NaN  
114      -   2000.0   10%    99.0     99.0  
115      -  20000.0  100%     NaN      NaN

In [310]:
df.loc[112, '동일인과의관계2'] = '친족'

In [311]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
신양월드레저         1.200000
한국타이어앤테크놀로지    1.056669
Name: own2, dtype: float64

In [312]:
df[df['소속회사명']=='신양월드레저']

금융회사   소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3      성명  보통주주식수 보통주지분율 우선주주식수  \
72  비금융회사  신양월드레저    동일인측      동일인       동일인     조양래   40000    20%      -   
73  비금융회사  신양월드레저    동일인측       친족  배우자/혈족1촌     조현식   80000    40%      -   
74  비금융회사  신양월드레저    동일인측       친족  배우자/혈족1촌     조현범   80000    40%      -   
75  비금융회사  신양월드레저    동일인측       친족      친족합계    친족합계  200000   100%      -   
76  비금융회사  신양월드레저    동일인측   동일인측합계    동일인측합계  동일인측합계  200000   100%      -   
77  비금융회사  신양월드레저      총계       총계        총계      총계  200000   100%      -   

   우선주지분율     합계주식수 합계지분율  ownCat  ownCat2  
72      -   40000.0   20%     0.0      0.0  
73      -   80000.0   40%     NaN      NaN  
74      -   80000.0   40%     NaN      NaN  
75      -  200000.0  100%     0.0      0.0  
76      -  200000.0  100%     NaN      NaN  
77      -  200000.0  100%     NaN      NaN

In [313]:
df.loc[75, '합계주식수'] = 80000.0 * 2

In [314]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
한국타이어앤테크놀로지    1.056669
Name: own2, dtype: float64

In [315]:
df[df['소속회사명']=='한국타이어앤테크놀로지']

금융회사        소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3           성명     보통주주식수  \
17  비금융회사  한국타이어앤테크놀로지    동일인측      동일인       동일인          조양래    7019903   
18  비금융회사  한국타이어앤테크놀로지    동일인측       친족  배우자/혈족1촌          홍문자       9119   
19  비금융회사  한국타이어앤테크놀로지    동일인측       친족  배우자/혈족1촌          조현범    2561241   
20  비금융회사  한국타이어앤테크놀로지    동일인측       친족  배우자/혈족1촌          조현식     799244   
21  비금융회사  한국타이어앤테크놀로지    동일인측       친족  배우자/혈족1촌          조희원     884239   
22  비금융회사  한국타이어앤테크놀로지    동일인측       친족  배우자/혈족1촌          조희경    3366870   
23  비금융회사  한국타이어앤테크놀로지    동일인측       친족    혈족2~4촌          조재형      15397   
24  비금융회사  한국타이어앤테크놀로지    동일인측       친족    혈족2~4촌          조유빈      15351   
25  비금융회사  한국타이어앤테크놀로지    동일인측       친족    혈족2~4촌          조재민      15302   
26  비금융회사  한국타이어앤테크놀로지    동일인측       친족    혈족2~4촌          조재완      15447   
27  비금융회사  한국타이어앤테크놀로지    동일인측       친족    인척4촌이내          노정호      26121   
28  비금융회사  한국타이어앤테크놀로지    동일인측       친족    인척4촌이내          김명자      26057   
29  비금융회사  한국타이어앤테크놀로지    동일인측       친족      친족합계         친족합계   14754291   
30  비금융회사  한국타이어앤테크놀로지    동일인측    등기된임원     등기된임원          이수일       5263   
31  비금융회사  한국타이어앤테크놀로지    동일인측     자기주식      자기주식  한국타이어앤테크놀로지    1863928   
32  비금융회사  한국타이어앤테크놀로지    동일인측     계열회사      계열회사       한국앤컴퍼니   37995959   
33  비금융회사  한국타이어앤테크놀로지    동일인측     계열회사      계열회사       신양관광개발     793522   
34  비금융회사  한국타이어앤테크놀로지    동일인측   동일인측합계    동일인측합계       동일인측합계   55412963   
35  비금융회사  한국타이어앤테크놀로지    기타주주       기타        기타            -   68462106   
36  비금융회사  한국타이어앤테크놀로지      총계       총계        총계           총계  123875069   

     보통주지분율 우선주주식수 우선주지분율        합계주식수    합계지분율  ownCat  ownCat2  
17    5.67%      -      -    7019903.0    5.67%     0.0      0.0  
18    0.01%      -      -       9119.0    0.01%     NaN      NaN  
19    2.07%      -      -    2561241.0    2.07%     NaN      NaN  
20    0.65%      -      -     799244.0    0.65%     NaN      NaN  
21    0.71%      -      -     884239.0    0.71%     NaN      NaN  
22    2.72%      -      -    3366870.0    2.72%     NaN      NaN  
23    0.01%      -      -      15397.0    0.01%     NaN      NaN  
24    0.01%      -      -      15351.0    0.01%     NaN      NaN  
25    0.01%      -      -      15302.0    0.01%     NaN      NaN  
26    0.01%      -      -      15447.0    0.01%     NaN      NaN  
27    0.02%      -      -      26121.0    0.02%     NaN      NaN  
28    0.02%      -      -      26057.0    0.02%     NaN      NaN  
29   11.91%      -      -   14754291.0   11.91%     0.0      0.0  
30    0.00%      -      -       5263.0    0.00%     NaN      0.0  
31    1.50%      -      -    1863928.0    1.50%     NaN      0.0  
32   30.67%      -      -   37995959.0   30.67%     1.0      1.0  
33    0.64%      -      -     793522.0    0.64%     1.0      1.0  
34   44.73%      -      -   55412963.0   44.73%     NaN      NaN  
35   55.27%      -      -   68462106.0   55.27%    99.0     99.0  
36  100.00%      -      -  123875069.0  100.00%     NaN      NaN

In [316]:
df.loc[29, '합계주식수'] = 14754291.0 - 7019903.0

In [317]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [318]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/한국타이어.xlsx", index=False)

# 오씨아이

In [303]:
df = pd.read_excel('../data/ownership-status/raw/2021/OCI.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [304]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [305]:
dfOwn

firmname  type       own      own2 ownername
0         디씨알이   0.0  0.000000  0.000000       NaN
1         디씨알이   1.0  1.000000  1.000000      오씨아이
2         디씨알이  99.0  0.000000  0.000000       NaN
3       비앤오바이오   0.0  0.000000  0.000000       NaN
4       비앤오바이오   1.0  0.500000  0.500000       OCI
5       비앤오바이오  99.0  0.500000  0.500000       NaN
6     에스지씨그린파워   0.0  0.000000  0.000000       NaN
7     에스지씨그린파워   1.0  0.950034  0.950034   에스지씨에너지
8     에스지씨그린파워  99.0  0.049966  0.049966       NaN
9    에스지씨디벨롭먼트   0.0  0.000000  0.000000       NaN
10   에스지씨디벨롭먼트   1.0  1.000000  1.000000   에스지씨에너지
11   에스지씨디벨롭먼트  99.0  0.000000  0.000000       NaN
12     에스지씨솔루션   0.0  0.000000  0.000000       NaN
13     에스지씨솔루션   1.0  1.000000  1.000000   에스지씨에너지
14     에스지씨솔루션  99.0  0.000000  0.000000       NaN
15     에스지씨에너지   0.0  0.477712  0.486507       NaN
16     에스지씨에너지   1.0  0.055803  0.055803       유니드
17     에스지씨에너지   1.0  0.031911  0.031911  SGC이테크건설
18     에스지씨에너지  99.0  0.425779  0.425779       NaN
19   에스지씨이테크건설   0.0  0.110163  0.163706       NaN
20   에스지씨이테크건설   1.0  0.311930  0.311930   에스지씨에너지
21   에스지씨이테크건설   1.0  0.025005  0.025005       유니드
22   에스지씨이테크건설  99.0  0.499360  0.499360       NaN
23        오씨아이   0.0  0.201382  0.226458       NaN
24        오씨아이   1.0  0.004193  0.004193       유니드
25        오씨아이  99.0  0.769349  0.769349       NaN
26      오씨아이드림   0.0  0.000000  0.000000       NaN
27      오씨아이드림   1.0  1.000000  1.000000      오씨아이
28      오씨아이드림  99.0  0.000000  0.000000       NaN
29    오씨아이스페셜티   0.0  0.000000  0.000162       NaN
30    오씨아이스페셜티   1.0  0.780704  0.780704      오씨아이
31    오씨아이스페셜티  99.0  0.219135  0.219135       NaN
32     오씨아이에스이   0.0  0.000000  0.000000       NaN
33     오씨아이에스이   1.0  1.000000  1.000000      오씨아이
34     오씨아이에스이  99.0  0.000000  0.000000       NaN
35    오씨아이정보통신   0.0  0.000000  0.000000       NaN
36    오씨아이정보통신   1.0  1.000000  1.000000      오씨아이
37    오씨아이정보통신  99.0  0.000000  0.000000       NaN
38      오씨아이파워   0.0  0.000000  0.000000       NaN
39      오씨아이파워   1.0  1.000000  1.000000      오씨아이
40      오씨아이파워  99.0  0.000000  0.000000       NaN
41      오씨아이페로   0.0  0.000000  0.000000       NaN
42      오씨아이페로   1.0  0.500000  0.500000      오씨아이
43      오씨아이페로  99.0  0.500000  0.500000       NaN
44         유니드   0.0  0.212794  0.230927       NaN
45         유니드   1.0  0.250567  0.250567  유니드글로벌상사
46         유니드  99.0  0.518507  0.518507       NaN
47    유니드글로벌상사   0.0  1.000000  1.000000       NaN
48    유니드글로벌상사  99.0  0.000000  0.000000       NaN
49      유니드엘이디   0.0  0.106971  0.111149       NaN
50      유니드엘이디   1.0  0.690007  0.690007       유니드
51      유니드엘이디   1.0  0.170439  0.170439  유니드글로벌상사
52      유니드엘이디  99.0  0.028405  0.028405       NaN
53  행복도시태양광발전소   0.0  0.000000  0.000000       NaN
54  행복도시태양광발전소   1.0  0.400000  0.400000      오씨아이
55  행복도시태양광발전소  99.0  0.600000  0.600000       NaN

In [306]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/OCI.xlsx", index=False)

# 이랜드

In [299]:
df = pd.read_excel('../data/ownership-status/raw/2021/이랜드월드.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [300]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [301]:
dfOwn

firmname  type  own  own2             ownername
0    농업회사법인맛누리   0.0  0.0   0.1                   NaN
1    농업회사법인맛누리   1.0  0.9   0.9                이랜드리테일
2    농업회사법인맛누리  99.0  0.0   0.0                   NaN
3   농업회사법인켄싱턴팜   0.0  0.0   0.1                   NaN
4   농업회사법인켄싱턴팜   1.0  0.9   0.9                 이랜드파크
..         ...   ...  ...   ...                   ...
95       켄싱턴월드   1.0  1.0   1.0                 이랜드파크
96       켄싱턴월드  99.0  0.0   0.0                   NaN
97       켄싱턴평창   0.0  0.0   0.0                   NaN
98       켄싱턴평창   1.0  1.0   1.0  MicornesiaResort,Inc
99       켄싱턴평창  99.0  0.0   0.0                   NaN

[100 rows x 5 columns]

In [302]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/이랜드.xlsx", index=False)

# 세아

In [279]:
df = pd.read_excel('../data/ownership-status/raw/2021/세아홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [280]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [281]:
dfOwn

firmname  type       own      own2 ownername
0              동아스틸   0.0  0.000000  0.000000       NaN
1              동아스틸   1.0  0.976732  0.976732  에스에스아이케이
2              동아스틸   1.0       NaN       NaN         -
3              동아스틸  99.0  0.023268  0.023268       NaN
4             브이엔티지   0.0  0.000000  0.089684       NaN
..              ...   ...       ...       ...       ...
107            이앤비원  99.0  0.000000  0.000000       NaN
108  탈리다쿰구,이사야에이치피피   0.0  0.000000  0.000000       NaN
109  탈리다쿰구,이사야에이치피피   1.0  0.841031  0.841031     에이치피피
110  탈리다쿰구,이사야에이치피피   1.0       NaN       NaN         -
111  탈리다쿰구,이사야에이치피피  99.0  0.158969  0.158969       NaN

[112 rows x 5 columns]

In [291]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/세아.xlsx", index=False)

# 중흥건설

In [287]:
df = pd.read_excel('../data/ownership-status/raw/2021/중흥건설.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [288]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
세흥건설    0.861557
Name: own2, dtype: float64

In [290]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/중흥건설.xlsx", index=False)

# 태광

In [518]:
df = pd.read_excel('../data/ownership-status/raw/2021/태광산업.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [519]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
대신흥국제일호사모투자    0.000000
티시스            1.042299
Name: own2, dtype: float64

In [520]:
# 오기로 추정됨
df[df['소속회사명'] == '대신흥국제일호사모투자']

금융회사        소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3           성명  보통주주식수  \
246  금융회사  대신흥국제일호사모투자    동일인측          동일인          동일인          NaN     NaN   
247  금융회사  대신흥국제일호사모투자    동일인측           친족     배우자/혈족1촌          NaN     NaN   
248  금융회사  대신흥국제일호사모투자    동일인측           친족       혈족2~4촌          NaN     NaN   
249  금융회사  대신흥국제일호사모투자    동일인측           친족       혈족5~6촌          NaN     NaN   
250  금융회사  대신흥국제일호사모투자    동일인측           친족       인척4촌이내          NaN     NaN   
251  금융회사  대신흥국제일호사모투자    동일인측           친족         친족합계          NaN     NaN   
252  금융회사  대신흥국제일호사모투자    동일인측        비영리법인        비영리법인          NaN     NaN   
253  금융회사  대신흥국제일호사모투자    동일인측        등기된임원        등기된임원          NaN     NaN   
254  금융회사  대신흥국제일호사모투자    동일인측         자기주식         자기주식          NaN     NaN   
255  금융회사  대신흥국제일호사모투자    동일인측         계열회사         계열회사       흥국생명보험     NaN   
256  금융회사  대신흥국제일호사모투자    동일인측         계열회사         계열회사       흥국자산운용     NaN   
257  금융회사  대신흥국제일호사모투자    동일인측         계열회사         계열회사     흥국화재해상보험     NaN   
258  금융회사  대신흥국제일호사모투자    동일인측       동일인측합계       동일인측합계          NaN     NaN   
259  금융회사  대신흥국제일호사모투자    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주       국민연금공단     NaN   
260  금융회사  대신흥국제일호사모투자    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주         대신증권     NaN   
261  금융회사  대신흥국제일호사모투자    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주  사립학교교직원연금공단     NaN   
262  금융회사  대신흥국제일호사모투자    기타주주           기타           기타          NaN     NaN   
263  금융회사  대신흥국제일호사모투자      총계           총계           총계          NaN     NaN   

     보통주지분율 우선주주식수 우선주지분율  합계주식수  합계지분율  ownCat  ownCat2  
246     NaN    NaN    NaN    NaN    NaN     0.0      0.0  
247     NaN    NaN    NaN    NaN    NaN     NaN      NaN  
248     NaN    NaN    NaN    NaN    NaN     NaN      NaN  
249     NaN    NaN    NaN    NaN    NaN     NaN      NaN  
250     NaN    NaN    NaN    NaN    NaN     NaN      NaN  
251     NaN    NaN    NaN    NaN    NaN     0.0      0.0  
252     NaN    NaN    NaN    NaN    NaN     NaN      0.0  
253     NaN    NaN    NaN    NaN    NaN     NaN      0.0  
254     NaN    NaN    NaN    NaN    NaN     NaN      0.0  
255     NaN    NaN    NaN    NaN    NaN     1.0      1.0  
256     NaN    NaN    NaN    NaN    NaN     1.0      1.0  
257     NaN    NaN    NaN    NaN    NaN     1.0      1.0  
258     NaN    NaN    NaN    NaN    NaN     NaN      NaN  
259     NaN    NaN    NaN    NaN    NaN    99.0     99.0  
260     NaN    NaN    NaN    NaN    NaN    99.0     99.0  
261     NaN    NaN    NaN    NaN    NaN    99.0     99.0  
262     NaN    NaN    NaN    NaN    NaN    99.0     99.0  
263     NaN    NaN    NaN    NaN    NaN     NaN      NaN

In [521]:
df[df['소속회사명'] == '티시스']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3          성명     보통주주식수  \
117  비금융회사   티시스    동일인측          동일인          동일인         이호진   400000.0   
118  비금융회사   티시스    동일인측           친족     배우자/혈족1촌         이현준  1068586.0   
119  비금융회사   티시스    동일인측           친족     배우자/혈족1촌         이현나    52263.0   
120  비금융회사   티시스    동일인측           친족     배우자/혈족1촌         신유나    52263.0   
121  비금융회사   티시스    동일인측           친족       혈족2~4촌         이원준   197000.0   
122  비금융회사   티시스    동일인측           친족       혈족5~6촌         NaN        NaN   
123  비금융회사   티시스    동일인측           친족       인척4촌이내         NaN        NaN   
124  비금융회사   티시스    동일인측           친족         친족합계         NaN  1770112.0   
125  비금융회사   티시스    동일인측        비영리법인        비영리법인  학교법인일주세화학원   321567.0   
126  비금융회사   티시스    동일인측        등기된임원        등기된임원         NaN        NaN   
127  비금융회사   티시스    동일인측         자기주식         자기주식         NaN        NaN   
128  비금융회사   티시스    동일인측         계열회사         계열회사        태광산업  4380950.0   
129  비금융회사   티시스    동일인측         계열회사         계열회사        대한화섬  2983800.0   
130  비금융회사   티시스    동일인측       동일인측합계       동일인측합계         NaN  9456429.0   
131  비금융회사   티시스    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주         NaN        NaN   
132  비금융회사   티시스    기타주주           기타           기타         NaN        NaN   
133  비금융회사   티시스      총계           총계           총계         NaN  9456429.0   

     보통주지분율 우선주주식수 우선주지분율      합계주식수   합계지분율  ownCat  ownCat2  
117    4.23      -      -   400000.0    4.23     0.0      0.0  
118   11.30      -      -  1068586.0   11.30     NaN      NaN  
119    0.55      -      -    52263.0    0.55     NaN      NaN  
120    0.55      -      -    52263.0    0.55     NaN      NaN  
121    2.08      -      -   197000.0    2.08     NaN      NaN  
122     NaN    NaN    NaN        NaN     NaN     NaN      NaN  
123     NaN    NaN    NaN        NaN     NaN     NaN      NaN  
124   18.72      -      -  1770112.0   18.72     0.0      0.0  
125    3.40      -      -   321567.0    3.40     NaN      0.0  
126     NaN    NaN    NaN        NaN     NaN     NaN      0.0  
127     NaN    NaN    NaN        NaN     NaN     NaN      0.0  
128   46.33      -      -  4380950.0   46.33     1.0      1.0  
129   31.55      -      -  2983800.0   31.55     1.0      1.0  
130  100.00      -      -  9456429.0  100.00     NaN      NaN  
131     NaN    NaN    NaN        NaN     NaN    99.0     99.0  
132     NaN    NaN    NaN        NaN     NaN    99.0     99.0  
133  100.00      -      -  9456429.0  100.00     NaN      NaN

In [522]:
df.loc[124, '합계주식수'] = 1770112.0 - 400000.0

In [523]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
대신흥국제일호사모투자    0.0
Name: own2, dtype: float64

In [524]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/태광.xlsx", index=False)

# 동원

In [516]:
df = pd.read_excel('../data/ownership-status/raw/2021/동원엔터프라이즈.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [517]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
동원산업    0.994912
Name: own2, dtype: float64

In [525]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/동원.xlsx", index=False)

# 한라

In [526]:
df = pd.read_excel('../data/ownership-status/raw/2021/한라홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [527]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [528]:
dfOwn

firmname  type       own      own2 ownername
0     로터스프라이빗에쿼티   0.0  1.000000  1.000000       NaN
1             만도   0.0  0.000070  0.002943       NaN
2             만도   1.0  0.302473  0.302473     한라홀딩스
3             만도  99.0  0.694585  0.694585       NaN
4          만도브로제   1.0  0.500000  0.500000        만도
5          만도브로제  99.0  0.500000  0.500000       NaN
6       만도차이나홀딩스   1.0  1.000000  1.000000        만도
7     만도헬라일렉트로닉스   1.0  1.000000  1.000000        만도
8        목포신항만운영   1.0  1.000000  1.000000        한라
9   배곧신도시지역특성화타운   1.0  0.950000  0.950000        한라
10  배곧신도시지역특성화타운  99.0  0.050000  0.050000       NaN
11            위코   1.0  1.000000  1.000000     한라홀딩스
12        제이제이한라   1.0  1.000000  1.000000     한라홀딩스
13            한라   0.0  0.138588  0.179754       NaN
14            한라   1.0  0.333045  0.333045     한라홀딩스
15            한라  99.0  0.487201  0.487201       NaN
16        한라리츠운용   1.0  0.901000  0.901000     한라홀딩스
17        한라리츠운용  99.0  0.099000  0.099000       NaN
18        한라세라지오   1.0  1.000000  1.000000        한라
19        한라오엠에스   1.0  1.000000  1.000000        한라
20        한라지엘에스   1.0  1.000000  1.000000        한라
21         한라홀딩스   0.0  0.243442  0.277261       NaN
22         한라홀딩스  99.0  0.722739  0.722739       NaN

In [529]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/한라.xlsx", index=False)

# 아모레퍼시픽

In [530]:
df = pd.read_excel('../data/ownership-status/raw/2021/아모레퍼시픽그룹.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [531]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
이니스프리        1.0
코어테크놀로지    800.2
Name: own2, dtype: float64

In [532]:
df[df['소속회사명'] == '코어테크놀로지']

금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3      성명  보통주주식수  보통주지분율  \
186  비금융회사  코어테크놀로지    동일인측          동일인          동일인       -       -       -   
187  비금융회사  코어테크놀로지    동일인측           친족     배우자/혈족1촌       -       -       -   
188  비금융회사  코어테크놀로지    동일인측           친족       혈족2~4촌       -       -       -   
189  비금융회사  코어테크놀로지    동일인측           친족       혈족5~6촌       -       -       -   
190  비금융회사  코어테크놀로지    동일인측           친족       인척4촌이내       -       -       -   
191  비금융회사  코어테크놀로지    동일인측           친족         친족합계       -       -       -   
192  비금융회사  코어테크놀로지    동일인측        비영리법인        비영리법인       -       -       -   
193  비금융회사  코어테크놀로지    동일인측        등기된임원        등기된임원       -       -       -   
194  비금융회사  코어테크놀로지    동일인측         자기주식         자기주식       -       -       -   
195  비금융회사  코어테크놀로지    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  아모레퍼시픽  200000   80.00   
196  비금융회사  코어테크놀로지    동일인측     기타동일인관련자     기타동일인관련자       -       -       -   
197  비금융회사  코어테크놀로지    동일인측       동일인측합계       동일인측합계       /  200000   80.00   
198  비금융회사  코어테크놀로지    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주       -       -       -   
199  비금융회사  코어테크놀로지    기타주주           기타           기타       -   50000   20.00   
200  비금융회사  코어테크놀로지      총계           총계           총계       /  250000  100.00   

    우선주주식수 우선주지분율     합계주식수   합계지분율  ownCat  ownCat2  
186      -      -       NaN       -     0.0      0.0  
187      -      -       NaN       -     NaN      NaN  
188      -      -       NaN       -     NaN      NaN  
189      -      -       NaN       -     NaN      NaN  
190      -      -       NaN       -     NaN      NaN  
191      -      -       NaN       -     0.0      0.0  
192      -      -       NaN       -     NaN      0.0  
193      -      -       NaN       -     NaN      0.0  
194      -      -       NaN       -     NaN      0.0  
195      -      -  200000.0    8000     1.0      1.0  
196      -      -       NaN       -     NaN      NaN  
197      -      -  200000.0    8000     NaN      NaN  
198      -      -       NaN       -    99.0     99.0  
199      -      -      50.0   20.00    99.0     99.0  
200      -      -     250.0  100.00     NaN      NaN

In [535]:
df.loc[199, '합계주식수'] = 50000
df.loc[200, '합계주식수'] = 250000

In [536]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
이니스프리    1.0
Name: own2, dtype: float64

In [537]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/아모레퍼시픽.xlsx", index=False)

# 아이엠엠인베스트먼트

In [539]:
df = pd.read_excel('../data/ownership-status/raw/2021/아이엠엠인베스트먼트.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [540]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
농업회사법인팜에이트          0.521200
아이엠엠인베스트먼트          1.000000
아이엠엠인프라제4호사모투자      1.000000
아이엠엠인프라제5의1호사모투자    1.000000
오하임아이엔티             0.583097
이미인                 0.761971
Name: own2, dtype: float64

In [541]:
df[df['소속회사명'] == '농업회사법인팜에이트']

금융회사       소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3             성명  \
1954  비금융회사  농업회사법인팜에이트    동일인측          동일인          동일인              -   
1955  비금융회사  농업회사법인팜에이트    동일인측           친족     배우자/혈족1촌              -   
1956  비금융회사  농업회사법인팜에이트    동일인측           친족     배우자/혈족1촌              -   
1957  비금융회사  농업회사법인팜에이트    동일인측           친족       혈족2~4촌              -   
1958  비금융회사  농업회사법인팜에이트    동일인측           친족       혈족2~4촌              -   
1959  비금융회사  농업회사법인팜에이트    동일인측           친족       혈족5~6촌              -   
1960  비금융회사  농업회사법인팜에이트    동일인측           친족       혈족5~6촌              -   
1961  비금융회사  농업회사법인팜에이트    동일인측           친족       인척4촌이내              -   
1962  비금융회사  농업회사법인팜에이트    동일인측           친족       인척4촌이내              -   
1963  비금융회사  농업회사법인팜에이트    동일인측           친족         친족합계              /   
1964  비금융회사  농업회사법인팜에이트    동일인측        비영리법인        비영리법인              -   
1965  비금융회사  농업회사법인팜에이트    동일인측        비영리법인        비영리법인              -   
1966  비금융회사  농업회사법인팜에이트    동일인측        등기된임원        등기된임원            박종위   
1967  비금융회사  농업회사법인팜에이트    동일인측        등기된임원        등기된임원            강대현   
1968  비금융회사  농업회사법인팜에이트    동일인측        등기된임원        등기된임원            김양수   
1969  비금융회사  농업회사법인팜에이트    동일인측         자기주식         자기주식              -   
1970  비금융회사  농업회사법인팜에이트    동일인측  계열회사(국내+해외)  계열회사(국내+해외)              -   
1971  비금융회사  농업회사법인팜에이트    동일인측     기타동일인관련자     기타동일인관련자  2014IMMAG벤처펀드   
1972  비금융회사  농업회사법인팜에이트    동일인측     기타동일인관련자     기타동일인관련자              -   
1973  비금융회사  농업회사법인팜에이트    동일인측       동일인측합계       동일인측합계              /   
1974  비금융회사  농업회사법인팜에이트    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주              -   
1975  비금융회사  농업회사법인팜에이트    기타주주           기타           기타              /   
1976  비금융회사  농업회사법인팜에이트      총계           총계           총계              /   

      보통주주식수   보통주지분율 우선주주식수 우선주지분율     합계주식수    합계지분율  ownCat  ownCat2  
1954       -        -      -      -       NaN        -     0.0      0.0  
1955       -        -      -      -       NaN        -     NaN      NaN  
1956       -        -      -      -       NaN        -     NaN      NaN  
1957       -        -      -      -       NaN        -     NaN      NaN  
1958       -        -      -      -       NaN        -     NaN      NaN  
1959       -        -      -      -       NaN        -     NaN      NaN  
1960       -        -      -      -       NaN        -     NaN      NaN  
1961       -        -      -      -       NaN        -     NaN      NaN  
1962       -        -      -      -       NaN        -     NaN      NaN  
1963       -        -      -      -       NaN        -     0.0      0.0  
1964       -        -      -      -       NaN        -     NaN      0.0  
1965       -        -      -      -       NaN        -     NaN      0.0  
1966   23400    3.97%      -      -   23400.0    3.97%     NaN      0.0  
1967   16800    2.85%      -      -   16800.0    2.85%     NaN      0.0  
1968    1210    0.21%      -      -    1210.0    0.21%     NaN      0.0  
1969       -        -      -      -       NaN        -     NaN      0.0  
1970       -        -      -      -       NaN        -     1.0      1.0  
1971  282066   47.88%      -      -  282066.0   47.88%     NaN      NaN  
1972       -        -      -      -       NaN        -     NaN      NaN  
1973  323476   54.91%      -      -  323476.0   54.91%     NaN      NaN  
1974       -        -      -      -       NaN        -    99.0     99.0  
1975  265634   45.09%      -      -  265634.0   45.09%    99.0     99.0  
1976  589110  100.00%      -      -  589110.0  100.00%     NaN      NaN

In [543]:
df[df['소속회사명'] == '오하임아이엔티']

금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3                 성명  \
1865  비금융회사  오하임아이엔티    동일인측          동일인          동일인                  -   
1866  비금융회사  오하임아이엔티    동일인측           친족     배우자/혈족1촌                  -   
1867  비금융회사  오하임아이엔티    동일인측           친족     배우자/혈족1촌                  -   
1868  비금융회사  오하임아이엔티    동일인측           친족       혈족2~4촌                  -   
1869  비금융회사  오하임아이엔티    동일인측           친족       혈족2~4촌                  -   
1870  비금융회사  오하임아이엔티    동일인측           친족       혈족5~6촌                  -   
1871  비금융회사  오하임아이엔티    동일인측           친족       혈족5~6촌                  -   
1872  비금융회사  오하임아이엔티    동일인측           친족       인척4촌이내                  -   
1873  비금융회사  오하임아이엔티    동일인측           친족       인척4촌이내                  -   
1874  비금융회사  오하임아이엔티    동일인측           친족         친족합계                  /   
1875  비금융회사  오하임아이엔티    동일인측        비영리법인        비영리법인                  -   
1876  비금융회사  오하임아이엔티    동일인측        비영리법인        비영리법인                  -   
1877  비금융회사  오하임아이엔티    동일인측        등기된임원        등기된임원                서정일   
1878  비금융회사  오하임아이엔티    동일인측        등기된임원        등기된임원                  -   
1879  비금융회사  오하임아이엔티    동일인측         자기주식         자기주식                  -   
1880  비금융회사  오하임아이엔티    동일인측  계열회사(국내+해외)  계열회사(국내+해외)                  -   
1881  비금융회사  오하임아이엔티    동일인측     기타동일인관련자     기타동일인관련자     2014IMMICT벤처펀드   
1882  비금융회사  오하임아이엔티    동일인측     기타동일인관련자     기타동일인관련자  2014성장사다리-IMM벤처펀드   
1883  비금융회사  오하임아이엔티    동일인측     기타동일인관련자     기타동일인관련자  2015IMMDESIGN벤처펀드   
1884  비금융회사  오하임아이엔티    동일인측       동일인측합계       동일인측합계                  /   
1885  비금융회사  오하임아이엔티    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주                  -   
1886  비금융회사  오하임아이엔티    기타주주           기타           기타                  /   
1887  비금융회사  오하임아이엔티      총계           총계           총계                  /   

        보통주주식수   보통주지분율 우선주주식수 우선주지분율       합계주식수    합계지분율  ownCat  ownCat2  
1865         -        -      -      -         NaN        -     0.0      0.0  
1866         -        -      -      -         NaN        -     NaN      NaN  
1867         -        -      -      -         NaN        -     NaN      NaN  
1868         -        -      -      -         NaN        -     NaN      NaN  
1869         -        -      -      -         NaN        -     NaN      NaN  
1870         -        -      -      -         NaN        -     NaN      NaN  
1871         -        -      -      -         NaN        -     NaN      NaN  
1872         -        -      -      -         NaN        -     NaN      NaN  
1873         -        -      -      -         NaN        -     NaN      NaN  
1874         -        -      -      -         NaN        -     0.0      0.0  
1875         -        -      -      -         NaN        -     NaN      0.0  
1876         -        -      -      -         NaN        -     NaN      0.0  
1877   1441798    7.30%      -      -   1441798.0    7.30%     NaN      0.0  
1878         -        -      -      -         NaN        -     NaN      0.0  
1879         -        -      -      -         NaN        -     NaN      0.0  
1880         -        -      -      -         NaN        -     1.0      1.0  
1881    538062    2.72%      -      -    538062.0    2.72%     NaN      NaN  
1882    538062    2.72%      -      -    538062.0    2.72%     NaN      NaN  
1883   7156754   36.24%      -      -   7156754.0   36.24%     NaN      NaN  
1884   9674676   48.99%      -      -   9674676.0   48.99%     NaN      NaN  
1885         -        -      -      -         NaN        -    99.0     99.0  
1886  10073015   51.01%      -      -  10073015.0   51.01%    99.0     99.0  
1887  19747691  100.00%      -      -  19747691.0  100.00%     NaN      NaN

In [544]:
df[df['소속회사명'] == '이미인']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3   성명  보통주주식수   보통주지분율  \
1888  비금융회사   이미인    동일인측          동일인          동일인    -       -        -   
1889  비금융회사   이미인    동일인측           친족     배우자/혈족1촌    -       -        -   
1890  비금융회사   이미인    동일인측           친족     배우자/혈족1촌    -       -        -   
1891  비금융회사   이미인    동일인측           친족       혈족2~4촌    -       -        -   
1892  비금융회사   이미인    동일인측           친족       혈족2~4촌    -       -        -   
1893  비금융회사   이미인    동일인측           친족       혈족5~6촌    -       -        -   
1894  비금융회사   이미인    동일인측           친족       혈족5~6촌    -       -        -   
1895  비금융회사   이미인    동일인측           친족       인척4촌이내    -       -        -   
1896  비금융회사   이미인    동일인측           친족       인척4촌이내    -       -        -   
1897  비금융회사   이미인    동일인측           친족         친족합계    /       -        -   
1898  비금융회사   이미인    동일인측        비영리법인        비영리법인    -       -        -   
1899  비금융회사   이미인    동일인측        비영리법인        비영리법인    -       -        -   
1900  비금융회사   이미인    동일인측        등기된임원        등기된임원  김주원    4757   23.83%   
1901  비금융회사   이미인    동일인측        등기된임원        등기된임원    -       -        -   
1902  비금융회사   이미인    동일인측         자기주식         자기주식  이미인     339    1.70%   
1903  비금융회사   이미인    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  볼라레  10.036   50.27%   
1904  비금융회사   이미인    동일인측     기타동일인관련자     기타동일인관련자    -       -        -   
1905  비금융회사   이미인    동일인측     기타동일인관련자     기타동일인관련자    -       -        -   
1906  비금융회사   이미인    동일인측       동일인측합계       동일인측합계    -   15132   75.80%   
1907  비금융회사   이미인    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주    -       -        -   
1908  비금융회사   이미인    기타주주           기타           기타    -    4833   24.20%   
1909  비금융회사   이미인      총계           총계           총계    -  19.965  100.00%   

     우선주주식수 우선주지분율      합계주식수    합계지분율  ownCat  ownCat2  
1888      -      -        NaN        -     0.0      0.0  
1889      -      -        NaN        -     NaN      NaN  
1890      -      -        NaN        -     NaN      NaN  
1891      -      -        NaN        -     NaN      NaN  
1892      -      -        NaN        -     NaN      NaN  
1893      -      -        NaN        -     NaN      NaN  
1894      -      -        NaN        -     NaN      NaN  
1895      -      -        NaN        -     NaN      NaN  
1896      -      -        NaN        -     NaN      NaN  
1897      -      -        NaN        -     0.0      0.0  
1898      -      -        NaN        -     NaN      0.0  
1899      -      -        NaN        -     NaN      0.0  
1900      -      -      4.757   23.83%     NaN      0.0  
1901      -      -        NaN        -     NaN      0.0  
1902      -      -    339.000    1.70%     NaN      0.0  
1903      -      -  10036.000   50.27%     1.0      1.0  
1904      -      -        NaN        -     NaN      NaN  
1905      -      -        NaN        -     NaN      NaN  
1906      -      -  15132.000   75.80%     NaN      NaN  
1907      -      -        NaN        -    99.0     99.0  
1908      -      -   4833.000   24.20%    99.0     99.0  
1909      -      -  19965.000  100.00%     NaN      NaN

In [547]:
df.loc[1900, '합계주식수'] = 4757

In [548]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
농업회사법인팜에이트          0.521200
아이엠엠인베스트먼트          1.000000
아이엠엠인프라제4호사모투자      1.000000
아이엠엠인프라제5의1호사모투자    1.000000
오하임아이엔티             0.583097
이미인                 1.000000
Name: own2, dtype: float64

In [549]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/IMM인베스트먼트.xlsx", index=False)

# 삼천리

In [550]:
df = pd.read_excel('../data/ownership-status/raw/2021/삼천리.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [551]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼천리    0.961165
Name: own2, dtype: float64

In [552]:
df[df['소속회사명'] == '삼천리']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3                       성명  \
0   비금융회사   삼천리    동일인측          동일인          동일인                      이만득   
1   비금융회사   삼천리    동일인측           친족     배우자/혈족1촌                      이은희   
2   비금융회사   삼천리    동일인측           친족     배우자/혈족1촌                      이은남   
3   비금융회사   삼천리    동일인측           친족     배우자/혈족1촌              이은선YiEunSun   
4   비금융회사   삼천리    동일인측           친족       혈족2~4촌                      이은백   
5   비금융회사   삼천리    동일인측           친족       혈족5~6촌                        -   
6   비금융회사   삼천리    동일인측           친족       인척4촌이내                        -   
7   비금융회사   삼천리    동일인측           친족         친족합계                        /   
8   비금융회사   삼천리    동일인측        비영리법인        비영리법인                      NaN   
9   비금융회사   삼천리    동일인측        등기된임원        등기된임원                      길형도   
10  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      김상현   
11  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      김원중   
12  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      김정태   
13  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      김태석   
14  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      권현명   
15  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      박무철   
16  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      손원현   
17  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      안민호   
18  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      유상덕   
19  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      유재권   
20  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      유태봉   
21  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      윤양노   
22  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      이성욱   
23  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      이찬의   
24  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      정희돈   
25  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      조성용   
26  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      하찬호   
27  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      현운식   
28  비금융회사   삼천리    동일인측        등기된임원        등기된임원                      홍창우   
29  비금융회사   삼천리    동일인측        등기된임원        등기된임원  유로버트용욱YooRobertYongWook   
30  비금융회사   삼천리    동일인측         자기주식         자기주식                      삼천리   
31  비금융회사   삼천리    동일인측  계열회사(국내+해외)  계열회사(국내+해외)                        -   
32  비금융회사   삼천리    동일인측     기타동일인관련자     기타동일인관련자                      유혜숙   
33  비금융회사   삼천리    동일인측       동일인측합계       동일인측합계                        /   
34  비금융회사   삼천리    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주                        -   
35  비금융회사   삼천리    기타주주           기타           기타                        /   
36  비금융회사   삼천리      총계           총계           총계                        /   

     보통주주식수  보통주지분율 우선주주식수 우선주지분율      합계주식수   합계지분율  ownCat  ownCat2  
0    338390    8.34      -      -   338390.0    8.34     0.0      0.0  
1     27045    0.67      -      -    27045.0    0.67     NaN      NaN  
2     27045    0.67      -      -    27045.0    0.67     NaN      NaN  
3     27045    0.67    NaN      -    27045.0    0.67     NaN      NaN  
4    372070    9.18      -      -   372070.0    9.18     NaN      NaN  
5         -       -      -      -        NaN       -     NaN      NaN  
6         -     NaN      -      -        NaN     NaN     NaN      NaN  
7    453205   11.18      -      -   453205.0   11.18     0.0      0.0  
8       NaN     NaN      -      -        NaN     NaN     NaN      0.0  
9       250    0.01      -      -      250.0    0.01     NaN      0.0  
10       80    0.00      -      -       80.0    0.00     NaN      0.0  
11       67    0.00      -      -       67.0    0.00     NaN      0.0  
12      100    0.00      -      -      100.0    0.00     NaN      0.0  
13       53    0.00      -      

In [553]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/삼천리.xlsx", index=False)

# 금호석유화학

In [554]:
df = pd.read_excel('../data/ownership-status/raw/2021/금호석유화학.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [555]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [556]:
dfOwn

firmname  type       own      own2  ownername
0     강원학교태양광   1.0  1.000000  1.000000  코리아에너지발전소
1      금호개발상사   1.0  1.000000  1.000000    금호피앤비화학
2       금호리조트   1.0  0.667221  0.667221     금호석유화학
3       금호리조트   1.0  0.332779  0.332779    금호피앤비화학
4     금호미쓰이화학   1.0  0.500000  0.500000     금호석유화학
5     금호미쓰이화학  99.0  0.500000  0.500000        NaN
6      금호석유화학   0.0  0.226999  0.394760        NaN
7      금호석유화학  99.0  0.605240  0.605240        NaN
8       금호티앤엘   1.0  1.000000  1.000000     금호석유화학
9       금호폴리켐   1.0  0.500000  0.500000     금호석유화학
10      금호폴리켐  99.0  0.500000  0.500000        NaN
11    금호피앤비화학   1.0  1.000000  1.000000     금호석유화학
12     디앤케이켐텍   1.0  0.500000  0.500000    금호피앤비화학
13     디앤케이켐텍  99.0  0.500000  0.500000        NaN
14      여수페트로   1.0  0.222000  0.222000     금호석유화학
15      여수페트로   1.0  0.202000  0.202000    금호피앤비화학
16      여수페트로  99.0  0.576000  0.576000        NaN
17   영광백수풍력발전   1.0  0.510000  0.510000     금호석유화학
18   영광백수풍력발전  99.0  0.490000  0.490000        NaN
19      지노모터스   0.0  1.000000  1.000000        NaN
20       지노무역   0.0  1.000000  1.000000        NaN
21       철도솔라   1.0  0.999439  0.999439  코리아에너지발전소
22       철도솔라  99.0  0.000561  0.000561        NaN
23  코리아에너지발전소   1.0  0.961282  0.961282     금호석유화학
24  코리아에너지발전소  99.0  0.038718  0.038718        NaN

In [557]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/IMM인베스트먼트.xlsx", index=False)

# 다우키움

In [558]:
df = pd.read_excel('../data/ownership-status/raw/2021/키움증권.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [559]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
포스코우리이아이지글로벌사모투자    1.0
한국전력우리스프랏글로벌사모투자    1.0
Name: own2, dtype: float64

In [560]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/다우키움.xlsx", index=False)

# 장금상선

In [561]:
df = pd.read_excel('../data/ownership-status/raw/2021/장금상선.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [562]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
장금상선    0.179348
Name: own2, dtype: float64

In [563]:
df[df['소속회사명'] == '장금상선']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3               성명  보통주주식수  \
0   비금융회사  장금상선    동일인측          동일인          동일인              정태순   62000   
1   비금융회사  장금상선    동일인측           친족         배우자/                -       -   
2   비금융회사  장금상선    동일인측           친족         혈족1촌                -       -   
3   비금융회사  장금상선    동일인측           친족       혈족2~4촌                -       -   
4   비금융회사  장금상선    동일인측           친족       혈족2~4촌                -       -   
5   비금융회사  장금상선    동일인측           친족       혈족5~6촌                -       -   
6   비금융회사  장금상선    동일인측           친족       혈족5~6촌                -       -   
7   비금융회사  장금상선    동일인측           친족       인척4촌이내                -       -   
8   비금융회사  장금상선    동일인측           친족       인척4촌이내                -       -   
9   비금융회사  장금상선    동일인측           친족         친족합계                /       -   
10  비금융회사  장금상선    동일인측        비영리법인        비영리법인                -       -   
11  비금융회사  장금상선    동일인측        비영리법인        비영리법인                -       -   
12  비금융회사  장금상선    동일인측        등기된임원        등기된임원                -       -   
13  비금융회사  장금상선    동일인측        등기된임원        등기된임원                -       -   
14  비금융회사  장금상선    동일인측         자기주식         자기주식                -       -   
15  비금융회사  장금상선    동일인측  계열회사(국내+해외)  계열회사(국내+해외)            국양로지텍    4000   
16  비금융회사  장금상선    동일인측  계열회사(국내+해외)  계열회사(국내+해외)                -       -   
17  비금융회사  장금상선    동일인측     기타동일인관련자     기타동일인관련자  SINOKOR.CO.,LTD  302000   
18  비금융회사  장금상선    동일인측       동일인측합계       동일인측합계                /  368000   
19  비금융회사  장금상선    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주                -       -   
20  비금융회사  장금상선    기타주주           기타           기타                /       -   
21  비금융회사  장금상선      총계           총계           총계                /  368000   

     보통주지분율 우선주주식수 우선주지분율     합계주식수    합계지분율  ownCat  ownCat2  
0    16.85%      -      -   62000.0   16.85%     0.0      0.0  
1         -      -      -       NaN        -     NaN      NaN  
2         -      -      -       NaN        -     NaN      NaN  
3         -      -      -       NaN        -     NaN      NaN  
4         -      -      -       NaN        -     NaN      NaN  
5         -      -      -       NaN        -     NaN      NaN  
6         -      -      -       NaN        -     NaN      NaN  
7         -      -      -       NaN        -     NaN      NaN  
8         -      -      -       NaN        -     NaN      NaN  
9         -      -      -       NaN        -     0.0      0.0  
10        -      -      -       NaN        -     NaN      0.0  
11        -      -      -       NaN        -     NaN      0.0  
12        -      -      -       NaN        -     NaN      0.0  
13        -      -      -       NaN        -     NaN      0.0  
14        -      -      -       NaN        -     NaN      0.0  
15    1.08%      -      -    4000.0    1.08%     1.0      1.0  
16        -      -      -       NaN        -     1.0      1.0  
17   82.07%      -      -  302000.0   82.07%     NaN      NaN  
18  100.00%      -  0.00%  368000.0  100.00%     NaN      NaN  
19        -      -      -       NaN        -    99.0     99.0  
20        -      -      -       NaN        -    99.0     99.0  
21  100.00%      -  0.00%  368000.0  100.00%     NaN      NaN

In [564]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/장금상선.xlsx", index=False)

# 동국제강

In [566]:
df = pd.read_excel('../data/ownership-status/raw/2021/동국제강.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [567]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
인터지스웅동센터    0.1009
Name: own2, dtype: float64

In [568]:
df[df['소속회사명'] == '인터지스웅동센터']

금융회사     소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3    성명  보통주주식수 보통주지분율  \
59  비금융회사  인터지스웅동센터    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  인터지스  720000  90.00   
60  비금융회사  인터지스웅동센터    동일인측       동일인측합계       동일인측합계     /  720000  90.00   
61  비금융회사  인터지스웅동센터    기타주주           기타           기타     /   80000  10.00   
62  비금융회사  인터지스웅동센터      총계           총계           총계     /  800000    100   

   우선주주식수 우선주지분율     합계주식수  합계지분율  ownCat  ownCat2  
59      -      -     720.0  90.00     1.0      1.0  
60      -      -     720.0  90.00     NaN      NaN  
61      -      -   80000.0  10.00    99.0     99.0  
62      -      -  800000.0    100     NaN      NaN

In [569]:
df.loc[59, '합계주식수'] = 720000

In [570]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [571]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/동국제강.xlsx", index=False)

# 애경

In [572]:
df = pd.read_excel('../data/ownership-status/raw/2021/AK홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [573]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
서림          2.000000
에이케이에스앤디    0.999739
Name: own2, dtype: float64

In [574]:
df[df['소속회사명'] == '서림']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3           성명  보통주주식수  \
685  비금융회사    서림    동일인측          동일인          동일인            -       -   
686  비금융회사    서림    동일인측           친족     배우자/혈족1촌            -       -   
687  비금융회사    서림    동일인측           친족     배우자/혈족1촌            -       -   
688  비금융회사    서림    동일인측           친족       혈족2~4촌            -       -   
689  비금융회사    서림    동일인측           친족       혈족2~4촌            -       -   
690  비금융회사    서림    동일인측           친족       혈족5~6촌            -       -   
691  비금융회사    서림    동일인측           친족       혈족5~6촌            -       -   
692  비금융회사    서림    동일인측           친족       인척4촌이내            -       -   
693  비금융회사    서림    동일인측           친족       인척4촌이내            -       -   
694  비금융회사    서림    동일인측           친족         친족합계            /       -   
695  비금융회사    서림    동일인측        비영리법인        비영리법인            -       -   
696  비금융회사    서림    동일인측        비영리법인        비영리법인            -       -   
697  비금융회사    서림    동일인측        등기된임원        등기된임원            -       -   
698  비금융회사    서림    동일인측        등기된임원        등기된임원            -       -   
699  비금융회사    서림    동일인측         자기주식         자기주식            -       -   
700  비금융회사    서림    동일인측  계열회사(국내+해외)  계열회사(국내+해외)         애경개발  780600   
701  비금융회사    서림    동일인측  계열회사(국내+해외)  계열회사(국내+해외)     기타동일인관련자       -   
702  비금융회사    서림    동일인측       동일인측합계       동일인측합계            /  780600   
703  비금융회사    서림    동일인측         기타주주         기타주주  동일인측이아닌최다주주       -   
704  비금융회사    서림      기타            /            /            -       -   
705  비금융회사    서림      기타           총계           총계            /  780600   
706  비금융회사    서림     NaN          NaN          NaN          NaN     NaN   

    보통주지분율 우선주주식수 우선주지분율     합계주식수 합계지분율  ownCat  ownCat2  
685      -      -      -       NaN     -     0.0      0.0  
686      -      -      -       NaN     -     NaN      NaN  
687      -      -      -       NaN     -     NaN      NaN  
688      -      -      -       NaN     -     NaN      NaN  
689      -      -      -       NaN     -     NaN      NaN  
690      -      -      -       NaN     -     NaN      NaN  
691      -      -      -       NaN     -     NaN      NaN  
692      -      -      -       NaN     -     NaN      NaN  
693      -      -      -       NaN     -     NaN      NaN  
694      -      -      -       NaN     -     0.0      0.0  
695      -      -      -       NaN     -     NaN      0.0  
696      -      -      -       NaN     -     NaN      0.0  
697      -      -      -       NaN     -     NaN      0.0  
698      -      -      -       NaN     -     NaN      0.0  
699      -      -      -       NaN     -     NaN      0.0  
700   100%      -      -  780600.0  100%     1.0      1.0  
701      -      -      -       NaN     -     1.0      1.0  
702   100%      -      -  780600.0  100%     NaN      NaN  
703      -      -      -       NaN     -     NaN      NaN  
704      -      -      -       NaN   NaN    99.0     99.0  
705   100%      -   100%  780600.0  100%    99.0     99.0  
706    NaN    NaN    NaN       NaN   NaN     NaN      NaN

In [575]:
df.loc[705, '동일인과의관계'] = '총계'

In [576]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
에이케이에스앤디    0.999739
Name: own2, dtype: float64

In [577]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/애경.xlsx", index=False)

# 반도홀딩스

In [578]:
df = pd.read_excel('../data/ownership-status/raw/2021/반도홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [579]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
반도레저    0.1
Name: own2, dtype: float64

In [580]:
df[df['소속회사명'] == '반도레저']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3   성명  보통주주식수   보통주지분율  \
185  비금융회사  반도레저    동일인측          동일인          동일인  권홍사   20000   10.00%   
186  비금융회사  반도레저    동일인측           친족     배우자/혈족1촌  권재현  159800   79.90%   
187  비금융회사  반도레저    동일인측           친족     배우자/혈족1촌  유성애   20000   10.00%   
188  비금융회사  반도레저    동일인측           친족     배우자/혈족1촌  신동철     200    0.10%   
189  비금융회사  반도레저    동일인측           친족       혈족2~4촌    -       -        -   
190  비금융회사  반도레저    동일인측           친족       혈족2~4촌    -       -        -   
191  비금융회사  반도레저    동일인측           친족       혈족5~6촌    -       -        -   
192  비금융회사  반도레저    동일인측           친족       혈족5~6촌    -       -        -   
193  비금융회사  반도레저    동일인측           친족       인척4촌이내    -       -        -   
194  비금융회사  반도레저    동일인측           친족       인척4촌이내    -       -        -   
195  비금융회사  반도레저    동일인측           친족         친족합계    -       -        -   
196  비금융회사  반도레저    동일인측        비영리법인        비영리법인    -       -        -   
197  비금융회사  반도레저    동일인측        비영리법인        비영리법인    -       -        -   
198  비금융회사  반도레저    동일인측        등기된임원        등기된임원    -       -        -   
199  비금융회사  반도레저    동일인측        등기된임원        등기된임원    -       -        -   
200  비금융회사  반도레저    동일인측         자기주식         자기주식    -       -        -   
201  비금융회사  반도레저    동일인측  계열회사(국내+해외)  계열회사(국내+해외)    -       -        -   
202  비금융회사  반도레저    동일인측  계열회사(국내+해외)  계열회사(국내+해외)    -       -        -   
203  비금융회사  반도레저    동일인측     기타동일인관련자     기타동일인관련자    -       -        -   
204  비금융회사  반도레저    동일인측       동일인측합계       동일인측합계    -  180000   90.00%   
205  비금융회사  반도레저    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주    -       -        -   
206  비금융회사  반도레저    기타주주           기타           기타    -       -        -   
207  비금융회사  반도레저      총계           총계           총계    /  200000  100.00%   

    우선주주식수 우선주지분율     합계주식수    합계지분율  ownCat  ownCat2  
185      -      -   20000.0   10.00%     0.0      0.0  
186      -      -  159800.0   79.90%     NaN      NaN  
187      -      -   20000.0   10.00%     NaN      NaN  
188      -      -     200.0    0.10%     NaN      NaN  
189      -      -       NaN        -     NaN      NaN  
190      -      -       NaN        -     NaN      NaN  
191      -      -       NaN        -     NaN      NaN  
192      -      -       NaN        -     NaN      NaN  
193      -      -       NaN        -     NaN      NaN  
194      -      -       NaN        -     NaN      NaN  
195      -      -       NaN        -     0.0      0.0  
196      -      -       NaN        -     NaN      0.0  
197      -      -       NaN        -     NaN      0.0  
198      -      -       NaN        -     NaN      0.0  
199      -      -       NaN        -     NaN      0.0  
200      -      -       NaN        -     NaN      0.0  
201      -      -       NaN        -     1.0      1.0  
202      -      -       NaN        -     1.0      1.0  
203      -      -       NaN        -     NaN      NaN  
204      -      -  180000.0   90.00%     NaN      NaN  
205      -      -       NaN        -    99.0     99.0  
206      -      -       NaN        -    99.0     99.0  
207      -      -  200000.0  100.00%     NaN      NaN

In [581]:
df.loc[195, '합계주식수'] = 180000.0

In [582]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [583]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/반도홀딩스.xlsx", index=False)

# 유진

In [585]:
df = pd.read_excel('../data/ownership-status/raw/2021/유진기업.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [586]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
동양      0.941753
천안기업    1.108728
클린메디    2.000000
Name: own2, dtype: float64

In [587]:
df[df['소속회사명']=='동양']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3      성명     보통주주식수  보통주지분율  \
43  비금융회사    동양    동일인측          동일인          동일인       -          -       -   
44  비금융회사    동양    동일인측           친족     배우자/혈족1촌       -          -       -   
45  비금융회사    동양    동일인측           친족       혈족2~4촌       -          -       -   
46  비금융회사    동양    동일인측           친족       혈족5~6촌       -          -       -   
47  비금융회사    동양    동일인측           친족       인척4촌이내       -          -       -   
48  비금융회사    동양    동일인측           친족         친족합계       /          -       -   
49  비금융회사    동양    동일인측        비영리법인        비영리법인       -          -       -   
50  비금융회사    동양    동일인측        등기된임원        등기된임원     정진학      50000    0.02   
51  비금융회사    동양    동일인측         자기주식         자기주식      동양   40512000   16.97   
52  비금융회사    동양    동일인측  계열회사(국내+해외)  계열회사(국내+해외)    유진기업   56768592   23.78   
53  비금융회사    동양    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  유진투자증권   11438000    4.79   
54  비금융회사    동양    동일인측  계열회사(국내+해외)  계열회사(국내+해외)    현대개발    3469117    1.45   
55  비금융회사    동양    동일인측     기타동일인관련자     기타동일인관련자       -          -       -   
56  비금융회사    동양    동일인측       동일인측합계       동일인측합계       /  126200241   52.87   
57  비금융회사    동양    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주       -          -       -   
58  비금융회사    동양    기타주주           기타           기타       -  112483822   47.13   
59  비금융회사    동양      총계           총계           총계       /  238684063  100.00   

     우선주주식수  우선주지분율        합계주식수    합계지분율  ownCat  ownCat2  
43        -       -          NaN        -     0.0      0.0  
44        -       -          NaN        -     NaN      NaN  
45        -       -          NaN        -     NaN      NaN  
46        -       -          NaN        -     NaN      NaN  
47        -       -          NaN        -     NaN      NaN  
48        -       -          NaN        -     0.0      0.0  
49        -       -          NaN        -     NaN      0.0  
50        -       -      50000.0     0.02     NaN      0.0  
51   171981   16.76   40683981.0    16.97     NaN      0.0  
52        -       -   56768592.0    23.78     1.0      1.0  
53        -       -   11438000.0     4.79     1.0      1.0  
54        -       -    3469117.0     1.45     1.0      1.0  
55        -       -          NaN        -     NaN      NaN  
56   171981   16.76  126372222.0    52.72     NaN      NaN  
57        -       -          NaN        -    99.0     99.0  
58   854455   83.24  113338277.0    47.28    99.0     99.0  
59  1026436  100.00  239710499.0  100.00%     NaN      NaN

In [588]:
df[df['소속회사명']=='클린메디']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3              성명    보통주주식수  \
791  금융회사  클린메디    동일인측          동일인          동일인               -         -   
792  금융회사  클린메디    동일인측           친족     배우자/혈족1촌               -         -   
793  금융회사  클린메디    동일인측           친족       혈족2~4촌               -         -   
794  금융회사  클린메디    동일인측           친족       혈족5~6촌               -         -   
795  금융회사  클린메디    동일인측           친족       인척4촌이내               -         -   
796  금융회사  클린메디    동일인측           친족         친족합계               /         -   
797  금융회사  클린메디    동일인측        비영리법인        비영리법인               -         -   
798  금융회사  클린메디    동일인측        등기된임원        등기된임원               -         -   
799  금융회사  클린메디    동일인측         자기주식         자기주식               -         -   
800  금융회사  클린메디    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  인프라이니셔티브1호사모투자  23550000   
801  금융회사  클린메디    동일인측     기타동일인관련자     기타동일인관련자               -         -   
802  금융회사  클린메디    동일인측       동일인측합계       동일인측합계               /         -   
803  금융회사  클린메디    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주               -  23550000   
804  금융회사  클린메디    기타주주           기타           기타               -         -   
805  금융회사  클린메디      총계           총계           총계               /  23550000   

     보통주지분율 우선주주식수 우선주지분율       합계주식수   합계지분율  ownCat  ownCat2  
791       -      -      -         NaN       -     0.0      0.0  
792       -      -      -         NaN       -     NaN      NaN  
793       -      -      -         NaN       -     NaN      NaN  
794       -      -      -         NaN       -     NaN      NaN  
795       -      -      -         NaN       -     NaN      NaN  
796       -      -      -         NaN       -     0.0      0.0  
797       -      -      -         NaN       -     NaN      0.0  
798       -      -      -         NaN       -     NaN      0.0  
799       -      -      -         NaN       -     NaN      0.0  
800  100.00      -      -  23550000.0  100.00     1.0      1.0  
801       -      -      -         NaN       -     NaN      NaN  
802       -      -      -         NaN       -     NaN      NaN  
803  100.00      -      -  23550000.0  100.00    99.0     99.0  
804       -      -      -         NaN       -    99.0     99.0  
805  100.00      -      -  23550000.0  100.00     NaN      NaN

In [589]:
df.loc[803, '합계주식수'] = 0

In [590]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
동양      0.941753
천안기업    1.108728
Name: own2, dtype: float64

In [591]:
df[df['소속회사명']=='천안기업']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3    성명   보통주주식수  보통주지분율  \
521  비금융회사  천안기업    동일인측          동일인          동일인   유경선   189422   11.56   
522  비금융회사  천안기업    동일인측           친족     배우자/혈족1촌     -        -       -   
523  비금융회사  천안기업    동일인측           친족       혈족2~4촌   유창수   123957    7.56   
524  비금융회사  천안기업    동일인측           친족       혈족5~6촌     -        -       -   
525  비금융회사  천안기업    동일인측           친족       인척4촌이내     -        -       -   
526  비금융회사  천안기업    동일인측           친족         친족합계     /   302157   19.12   
527  비금융회사  천안기업    동일인측        비영리법인        비영리법인     -        -       -   
528  비금융회사  천안기업    동일인측        등기된임원        등기된임원     -        -       -   
529  비금융회사  천안기업    동일인측         자기주식         자기주식     -        -       -   
530  비금융회사  천안기업    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  유진기업  1325569   80.88   
531  비금융회사  천안기업    동일인측     기타동일인관련자     기타동일인관련자     -        -       -   
532  비금융회사  천안기업    동일인측       동일인측합계       동일인측합계     /  1638948  100.00   
533  비금융회사  천안기업    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주     -        -       -   
534  비금융회사  천안기업    기타주주           기타           기타     -        -       -   
535  비금융회사  천안기업      총계           총계           총계     /  1638948  100.00   

    우선주주식수 우선주지분율      합계주식수   합계지분율  ownCat  ownCat2  
521      -      -   189422.0   11.56     0.0      0.0  
522      -      -        NaN       -     NaN      NaN  
523      -      -   123957.0    7.56     NaN      NaN  
524      -      -        NaN       -     NaN      NaN  
525      -      -        NaN       -     NaN      NaN  
526      -      -   302157.0   19.12     0.0      0.0  
527      -      -        NaN       -     NaN      0.0  
528      -      -        NaN       -     NaN      0.0  
529      -      -        NaN       -     NaN      0.0  
530      -      -  1325569.0   80.88     1.0      1.0  
531      -      -        NaN       -     NaN      NaN  
532      -      -  1638948.0  100.00     NaN      NaN  
533      -      -        NaN       -    99.0     99.0  
534      -      -        NaN       -    99.0     99.0  
535      -      -  1638948.0  100.00     NaN      NaN

In [593]:
df.loc[526, '합계주식수'] = 123957.0

In [594]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
동양    0.941753
Name: own2, dtype: float64

In [595]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/유진.xlsx", index=False)

# 하이트진로

In [598]:
df = pd.read_excel('../data/ownership-status/raw/2021/하이트진로홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [599]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [600]:
dfOwn

firmname  type       own      own2    ownername
0        강원물류   0.0  0.000000  0.000000          NaN
1        강원물류   1.0  1.000000  1.000000        하이트진로
2        강원물류  99.0  0.000000  0.000000          NaN
3   농업회사법인팜컬쳐   0.0  0.000000  0.100000          NaN
4   농업회사법인팜컬쳐   1.0  0.900000  0.900000        하이트진로
5   농업회사법인팜컬쳐  99.0  0.000000  0.000000          NaN
6       대우컴바인   0.0  1.000000  1.000000          NaN
7       대우컴바인   1.0       NaN       NaN            -
8       대우컴바인  99.0  0.000000  0.000000          NaN
9       대우패키지   0.0  1.000000  1.000000          NaN
10      대우패키지   1.0       NaN       NaN            -
11      대우패키지  99.0  0.000000  0.000000          NaN
12       대우화학   0.0  1.000000  1.000000          NaN
13       대우화학   1.0       NaN       NaN            -
14       대우화학  99.0  0.000000  0.000000          NaN
15       블루헤런   0.0  0.000000  0.000000          NaN
16       블루헤런   1.0  1.000000  1.000000      하이트진로산업
17       블루헤런  99.0  0.000000  0.000000          NaN
18      서영이앤티   0.0  0.999115  0.999115          NaN
19      서영이앤티   1.0       NaN       NaN            -
20      서영이앤티  99.0  0.000885  0.000885          NaN
21         송정   0.0  1.000000  1.000000          NaN
22         송정   1.0       NaN       NaN            -
23         송정  99.0  0.000000  0.000000          NaN
24       수양물류   0.0  0.000000  0.000000          NaN
25       수양물류   1.0  1.000000  1.000000        하이트진로
26       수양물류  99.0  0.000000  0.000000          NaN
27         연암   0.0  1.000000  1.000000          NaN
28         연암   1.0       NaN       NaN            -
29         연암  99.0  0.000000  0.000000          NaN
30       진로소주   0.0  0.000000  0.000000          NaN
31       진로소주   1.0  1.000000  1.000000     하이트진로홀딩스
32       진로소주  99.0  0.000000  0.000000          NaN
33       진로양조   0.0  0.000000  0.000000          NaN
34       진로양조   1.0  1.000000  1.000000        하이트진로
35       진로양조  99.0  0.000000  0.000000          NaN
36       천주물류   0.0  0.000000  0.000000          NaN
37       천주물류   1.0  1.000000  1.000000        하이트진로
38       천주물류  99.0  0.000000  0.000000          NaN
39     평암농산법인   0.0  0.000000  1.000000          NaN
40     평암농산법인   1.0       NaN       NaN            -
41     평암농산법인  99.0  0.000000  0.000000          NaN
42      하이트진로   0.0  0.025762  0.046968          NaN
43      하이트진로   1.0  0.502680  0.502680     하이트진로홀딩스
44      하이트진로   1.0  0.000412  0.000412        서영이앤티
45      하이트진로   1.0  0.003481  0.003481  JinroInc.해외
46      하이트진로  99.0  0.446459  0.446459          NaN
47    하이트진로산업   0.0  0.000000  0.000000          NaN
48    하이트진로산업   1.0  1.000000  1.000000        하이트진로
49    하이트진로산업  99.0  0.000000  0.000000          NaN
50    하이트진로음료   0.0  0.000000  0.000000          NaN
51    하이트진로음료   1.0  1.000000  1.000000        하이트진로
52    하이트진로음료  99.0  0.000000  0.000000          NaN
53   하이트진로홀딩스   0.0  0.289612  0.416853          NaN
54   하이트진로홀딩스   1.0  0.271646  0.271646        서영이앤티
55   하이트진로홀딩스   1.0  0.036744  0.036744  JinroInc.해외
56   하이트진로홀딩스  99.0  0.274757  0.274757          NaN

In [601]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/유진.xlsx", index=False)

# 삼양

In [602]:
df = pd.read_excel('../data/ownership-status/raw/2021/삼양홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [603]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼양에프앤비    2.0
삼양패키징     4.0
Name: own2, dtype: float64

In [604]:
df[df['소속회사명']=='삼양에프앤비']

금융회사   소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3     성명   보통주주식수  보통주지분율  \
158  비금융회사  삼양에프앤비    동일인측          동일인          동일인      -        -       -   
159  비금융회사  삼양에프앤비    동일인측           친족       혈족2~4촌      -        -       -   
160  비금융회사  삼양에프앤비    동일인측           친족       인척4촌이내      -        -       -   
161  비금융회사  삼양에프앤비    동일인측           친족         친족합계      -        -       -   
162  비금융회사  삼양에프앤비    동일인측        비영리법인        비영리법인      -        -       -   
163  비금융회사  삼양에프앤비    동일인측         자기주식         자기주식      -        -       -   
164  비금융회사  삼양에프앤비    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  삼양홀딩스  1876836  100.00   
165  비금융회사  삼양에프앤비    동일인측     기타동일인관련자     기타동일인관련자      -        -       -   
166  비금융회사  삼양에프앤비    동일인측       동일인측합계       동일인측합계      /  1876836  100.00   
167  비금융회사  삼양에프앤비    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주      -        -       -   
168  비금융회사  삼양에프앤비    기타주주           기타           기타      /  1876836  100.00   
169  비금융회사  삼양에프앤비      총계           총계           총계    NaN  1876836  100.00   

    우선주주식수 우선주지분율      합계주식수   합계지분율  ownCat  ownCat2  
158      -      -        NaN       -     0.0      0.0  
159      -      -        NaN       -     NaN      NaN  
160      -      -        NaN       -     NaN      NaN  
161      -      -        NaN       -     0.0      0.0  
162      -      -        NaN       -     NaN      0.0  
163      -      -        NaN       -     NaN      0.0  
164      -      -  1876836.0  100.00     1.0      1.0  
165      -      -        NaN       -     NaN      NaN  
166      -      -  1876836.0  100.00     NaN      NaN  
167      -      -        NaN       -    99.0     99.0  
168      -      -  1876836.0  100.00    99.0     99.0  
169      -      -  1876836.0  100.00     NaN      NaN

In [605]:
df.loc[168, '합계주식수'] = 0

In [606]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼양패키징    4.0
Name: own2, dtype: float64

In [607]:
df[df['소속회사명'] == '삼양패키징']

금융회사  소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3   성명    보통주주식수  보통주지분율  \
55  비금융회사  삼양패키징    동일인측          동일인          동일인    -         -       -   
56  비금융회사  삼양패키징    동일인측           친족     배우자/혈족1촌    -         -       -   
57  비금융회사  삼양패키징    동일인측           친족       혈족2~4촌    -         -       -   
58  비금융회사  삼양패키징    동일인측           친족       혈족5~6촌    -         -       -   
59  비금융회사  삼양패키징    동일인측           친족       인척4촌이내    -         -       -   
60  비금융회사  삼양패키징    동일인측           친족         친족합계    /         -       -   
61  비금융회사  삼양패키징    동일인측        비영리법인        비영리법인    -         -       -   
62  비금융회사  삼양패키징    동일인측        등기된임원        등기된임원    -         -       -   
63  비금융회사  삼양패키징    동일인측         자기주식         자기주식    -         -       -   
64  비금융회사  삼양패키징    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  삼양사   9378467   66.00   
65  비금융회사  삼양패키징    동일인측     기타동일인관련자     기타동일인관련자    -         -       -   
66  비금융회사  삼양패키징    동일인측       동일인측합계       동일인측합계    /   9378467   66.00   
67  비금융회사  삼양패키징    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주    -         -       -   
68  비금융회사  삼양패키징    기타주주           기타           기타    /   4831337   34.00   
69  비금융회사  삼양패키징    기타주주           총계           총계    /  14209804  100.00   
70  비금융회사  삼양패키징      총계           총계           총계    /  14209804  100.00   

   우선주주식수 우선주지분율       합계주식수   합계지분율  ownCat  ownCat2  
55      -      -         NaN       -     0.0      0.0  
56      -      -         NaN       -     NaN      NaN  
57      -      -         NaN       -     NaN      NaN  
58      -      -         NaN       -     NaN      NaN  
59      -      -         NaN       -     NaN      NaN  
60      -      -         NaN       -     0.0      0.0  
61      -      -         NaN       -     NaN      0.0  
62      -      -         NaN       -     NaN      0.0  
63      -      -         NaN       -     NaN      0.0  
64      -      -   9378467.0   66.00     1.0      1.0  
65      -      -         NaN       -     NaN      NaN  
66      -      -   9378467.0   66.00     NaN      NaN  
67      -      -         NaN       -    99.0     99.0  
68      -      -   4831337.0   34.00    99.0     99.0  
69      -      -  14209804.0  100.00    99.0     99.0  
70      -      -  14209804.0  100.00     NaN      NaN

In [610]:
df.loc[69, '합계주식수'] = 0
df.loc[70, '합계주식수'] = 14209804.0

In [611]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼양패키징   NaN
Name: own2, dtype: float64

In [612]:
dfOwn[dfOwn['firmname'] == '삼양패키징']

firmname  type   own  own2 ownername
15    삼양패키징   0.0  0.00  0.00       NaN
16    삼양패키징   1.0   inf   inf       삼양사
17    삼양패키징   1.0  0.66  0.66       삼양사
18    삼양패키징  99.0   NaN   NaN       NaN

In [613]:
dfOwn.loc[18, 'own'] = 0.34
dfOwn.loc[18, 'own2'] = 0.34

In [616]:
dfOwn = dfOwn.drop(16)

In [617]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/삼양.xlsx", index=False)

# 대방건설

In [618]:
df = pd.read_excel('../data/ownership-status/raw/2021/대방건설.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [619]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [620]:
dfOwn

firmname  type       own      own2 ownername
0       노블랜드   1.0  1.000000  1.000000      대방건설
1   대덕하우징씨스템   0.0  1.000000  1.000000       NaN
2     대방개발기업   1.0  1.000000  1.000000      대방건설
3       대방건설   0.0  1.000000  1.000000       NaN
4     대방건설동탄   1.0  0.950000  0.950000      대방건설
5     대방건설동탄  99.0  0.050000  0.050000       NaN
6       대방덕은   1.0  0.950000  0.950000      대방건설
7       대방덕은  99.0  0.050000  0.050000       NaN
8     대방디엠시티   0.0       NaN  0.033333       NaN
9     대방디엠시티   1.0  0.966667  0.966667      대방건설
10    대방산업개발   0.0  1.000000  1.000000       NaN
11  대방산업개발동탄   1.0  0.900000  0.900000    대방산업개발
12  대방산업개발동탄   1.0  0.050000  0.050000      대방건설
13  대방산업개발동탄  99.0  0.050000  0.050000       NaN
14   대방이노베이션   1.0  1.000000  1.000000      대방건설
15     대방이엔씨   1.0  1.000000  1.000000      대방건설
16  대방일산디엠시티   1.0  0.050000  0.050000      노블랜드
17  대방일산디엠시티   1.0  0.010000  0.010000      대방건설
18  대방일산디엠시티   1.0  0.445000  0.445000      디비일산
19  대방일산디엠시티   1.0  0.445000  0.445000     디케이일산
20  대방일산디엠시티  99.0  0.050000  0.050000       NaN
21      대방주택   1.0  1.000000  1.000000      대방건설
22      대방토건   1.0  1.000000  1.000000      대방건설
23     대방하우징   1.0  1.000000  1.000000      대방건설
24      디비개발   1.0  1.000000  1.000000      대방건설
25      디비건설   1.0  1.000000  1.000000      대방건설
26    디비산업개발   1.0  1.000000  1.000000      대방건설
27      디비일산   0.0       NaN  1.000000       NaN
28    디비종합건설   1.0  1.000000  1.000000      대방건설
29      디비주택   1.0  1.000000  1.000000      대방건설
30     디아이개발   1.0  1.000000  1.000000    대방산업개발
31     디아이건설   1.0  1.000000  1.000000    대방산업개발
32     디아이산업   1.0  1.000000  1.000000    대방산업개발
33   디아이주택개발   1.0  1.000000  1.000000    대방산업개발
34    디아이하우징   1.0  1.000000  1.000000    대방산업개발
35      디엠개발   1.0  1.000000  1.000000      대방건설
36      디엠건설   1.0  1.000000  1.000000      대방건설
37    디엠산업개발   1.0  1.000000  1.000000      대방건설
38     디엠이엔씨   1.0  1.000000  1.000000      대방건설
39      디엠주택   1.0  1.000000  1.000000      대방건설
40     디엠하우징   1.0  1.000000  1.000000      대방건설
41     디케이일산   0.0       NaN  1.000000       NaN
42    선남대방씨씨   1.0  1.000000  1.000000      대방건설
43      엔비건설   1.0  1.000000  1.000000      대방건설
44       엘리움   1.0  1.000000  1.000000    대방산업개발
45     엘리움개발   1.0  1.000000  1.000000    대방산업개발
46     엘리움건설   1.0  1.000000  1.000000    대방산업개발
47     엘리움주택   1.0  1.000000  1.000000    대방산업개발
48   엘리움주택개발   1.0  1.000000  1.000000    대방산업개발
49    엘리움하우징   1.0  1.000000  1.000000    대방산업개발
50   지유인터내셔날   0.0  1.000000  1.000000       NaN

In [621]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/대방건설.xlsx", index=False)

# 현대해상화재보험

In [623]:
df = pd.read_excel('../data/ownership-status/raw/2021/현대해상.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [624]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
밀텍    1000.0
Name: own2, dtype: float64

In [625]:
df[df['소속회사명'] == '밀텍']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3   성명  보통주주식수  보통주지분율  \
409  비금융회사    밀텍    동일인측          동일인          동일인    -       -       -   
410  비금융회사    밀텍    동일인측           친족     배우자/혈족1촌    -       -       -   
411  비금융회사    밀텍    동일인측           친족     배우자/혈족1촌    -       -       -   
412  비금융회사    밀텍    동일인측           친족       혈족2~4촌    -       -       -   
413  비금융회사    밀텍    동일인측           친족       혈족2~4촌    -       -       -   
414  비금융회사    밀텍    동일인측           친족       혈족5~6촌    -       -       -   
415  비금융회사    밀텍    동일인측           친족       혈족5~6촌    -       -       -   
416  비금융회사    밀텍    동일인측           친족       인척4촌이내  이성태   50000   25.00   
417  비금융회사    밀텍    동일인측           친족       인척4촌이내  정희숙   24000   12.00   
418  비금융회사    밀텍    동일인측           친족         친족합계    /   74000   37.00   
419  비금융회사    밀텍    동일인측        비영리법인        비영리법인    -       -       -   
420  비금융회사    밀텍    동일인측        비영리법인        비영리법인    -       -       -   
421  비금융회사    밀텍    동일인측        등기된임원        등기된임원  김완용   30600   15.30   
422  비금융회사    밀텍    동일인측        등기된임원        등기된임원  이선윤   27000   13.50   
423  비금융회사    밀텍    동일인측         자기주식         자기주식   밀텍   41000   20.50   
424  비금융회사    밀텍    동일인측  계열회사(국내+해외)  계열회사(국내+해외)    -       -       -   
425  비금융회사    밀텍    동일인측  계열회사(국내+해외)  계열회사(국내+해외)    -       -       -   
426  비금융회사    밀텍    동일인측     기타동일인관련자     기타동일인관련자    -       -       -   
427  비금융회사    밀텍    동일인측       동일인측합계       동일인측합계    /  172600   86.30   
428  비금융회사    밀텍    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주    -       -       -   
429  비금융회사    밀텍    기타주주           기타           기타    /   27400   13.70   
430  비금융회사    밀텍      총계           총계           총계    /  200000  100.00   

    우선주주식수 우선주지분율     합계주식수   합계지분율  ownCat  ownCat2  
409      -      -       NaN       -     0.0      0.0  
410      -      -       NaN       -     NaN      NaN  
411      -      -       NaN       -     NaN      NaN  
412      -      -       NaN       -     NaN      NaN  
413      -      -       NaN       -     NaN      NaN  
414      -      -       NaN       -     NaN      NaN  
415      -      -       NaN       -     NaN      NaN  
416      -      -   50000.0   25.00     NaN      NaN  
417      -      -   24000.0   12.00     NaN      NaN  
418      -      -   74000.0   37.00     0.0      0.0  
419      -      -       NaN       -     NaN      0.0  
420      -      -       NaN       -     NaN      0.0  
421      -      -   30600.0   15.30     NaN      0.0  
422      -      -   27000.0   13.50     NaN      0.0  
423      -      -   41000.0   20.50     NaN      0.0  
424      -      -       NaN       -     1.0      1.0  
425      -      -       NaN       -     1.0      1.0  
426      -      -       NaN       -     NaN      NaN  
427      -      -  172600.0   86.30     NaN      NaN  
428      -      -       NaN       -    99.0     99.0  
429      -      -   27400.0   13.70    99.0     99.0  
430      -      -     200.0  100.00     NaN      NaN

In [626]:
df.loc[430, '합계주식수'] = 200000

In [627]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [628]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/현대해상화재보험.xlsx", index=False)

# 엠디엠

In [629]:
df = pd.read_excel('../data/ownership-status/raw/2021/엠디엠.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [630]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
비오에스마케팅     2.0
알에스에스테크윈    2.0
엠앤케이        2.0
Name: own2, dtype: float64

In [631]:
df[df['소속회사명'] == '비오에스마케팅']

금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3  성명 보통주주식수 보통주지분율  \
202  비금융회사  비오에스마케팅    동일인측          동일인          동일인   -      -      -   
203  비금융회사  비오에스마케팅    동일인측           친족     배우자/혈족1촌   -      -      -   
204  비금융회사  비오에스마케팅    동일인측           친족       혈족2~4촌   -      -      -   
205  비금융회사  비오에스마케팅    동일인측           친족       혈족5~6촌   -      -      -   
206  비금융회사  비오에스마케팅    동일인측           친족       인척4촌이내  김영  10000   100%   
207  비금융회사  비오에스마케팅    동일인측           친족         친족합계   -  10000   100%   
208  비금융회사  비오에스마케팅    동일인측        비영리법인        비영리법인   -      -      -   
209  비금융회사  비오에스마케팅    동일인측        등기된임원        등기된임원   -      -      -   
210  비금융회사  비오에스마케팅    동일인측         자기주식         자기주식   -      -      -   
211  비금융회사  비오에스마케팅    동일인측  계열회사(국내+해외)  계열회사(국내+해외)   -      -      -   
212  비금융회사  비오에스마케팅    동일인측     기타동일인관련자     기타동일인관련자   -      -      -   
213  비금융회사  비오에스마케팅    동일인측       동일인측합계       동일인측합계   /  10000   100%   
214  비금융회사  비오에스마케팅    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주   -      -      -   
215  비금융회사  비오에스마케팅    기타주주           기타           기타   /      -      -   
216  비금융회사  비오에스마케팅    기타주주           총계           총계   /  10000   100%   

    우선주주식수 우선주지분율    합계주식수 합계지분율  ownCat  ownCat2  
202      -      -      NaN     -     0.0      0.0  
203      -      -      NaN     -     NaN      NaN  
204      -      -      NaN     -     NaN      NaN  
205      -      -      NaN     -     NaN      NaN  
206      -      -  10000.0  100%     NaN      NaN  
207      -      -  10000.0  100%     0.0      0.0  
208      -      -      NaN     -     NaN      0.0  
209      -      -      NaN     -     NaN      0.0  
210      -      -      NaN     -     NaN      0.0  
211      -      -      NaN     -     1.0      1.0  
212      -      -      NaN     -     NaN      NaN  
213      -      -  10000.0  100%     NaN      NaN  
214      -      -      NaN     -    99.0     99.0  
215      -      -      NaN     -    99.0     99.0  
216      -      -  10000.0  100%    99.0     99.0

In [632]:
df.loc[216, '동일인과의관계'] = '총계'

In [633]:
df[df['소속회사명'] == '알에스에스테크윈']

금융회사     소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3   성명  보통주주식수  보통주지분율  \
233  비금융회사  알에스에스테크윈    동일인측          동일인          동일인    -       -       -   
234  비금융회사  알에스에스테크윈    동일인측           친족     배우자/혈족1촌    -       -       -   
235  비금융회사  알에스에스테크윈    동일인측           친족       혈족2~4촌  문양현  103400  51.70%   
236  비금융회사  알에스에스테크윈    동일인측           친족       혈족5~6촌  문동석   12120   6.06%   
237  비금융회사  알에스에스테크윈    동일인측           친족       인척4촌이내  이월순   15000   7.50%   
238  비금융회사  알에스에스테크윈    동일인측           친족         친족합계    -  130520  65.26%   
239  비금융회사  알에스에스테크윈    동일인측        비영리법인        비영리법인    -       -       -   
240  비금융회사  알에스에스테크윈    동일인측        등기된임원        등기된임원    -       -       -   
241  비금융회사  알에스에스테크윈    동일인측         자기주식         자기주식    -       -       -   
242  비금융회사  알에스에스테크윈    동일인측  계열회사(국내+해외)  계열회사(국내+해외)    -       -       -   
243  비금융회사  알에스에스테크윈    동일인측     기타동일인관련자     기타동일인관련자    -       -       -   
244  비금융회사  알에스에스테크윈    동일인측       동일인측합계       동일인측합계    /  130520  65.26%   
245  비금융회사  알에스에스테크윈    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주    -       -       -   
246  비금융회사  알에스에스테크윈    기타주주           기타           기타    /   69480  34.74%   
247  비금융회사  알에스에스테크윈    기타주주           총계           총계    /  200000    100%   

    우선주주식수 우선주지분율     합계주식수   합계지분율  ownCat  ownCat2  
233      -      -       NaN       -     0.0      0.0  
234      -      -       NaN       -     NaN      NaN  
235      -      -  103400.0  51.70%     NaN      NaN  
236      -      -   12120.0   6.06%     NaN      NaN  
237      -      -   15000.0   7.50%     NaN      NaN  
238      -      -  130520.0  65.26%     0.0      0.0  
239      -      -       NaN       -     NaN      0.0  
240      -      -       NaN       -     NaN      0.0  
241      -      -       NaN       -     NaN      0.0  
242      -      -       NaN       -     1.0      1.0  
243      -      -       NaN       -     NaN      NaN  
244      -      -  130520.0  65.26%     NaN      NaN  
245      -      -       NaN       -    99.0     99.0  
246      -      -   69480.0  34.74%    99.0     99.0  
247      -      -  200000.0    100%    99.0     99.0

In [634]:
df.loc[247, '동일인과의관계'] = '총계'

In [635]:
df[df['소속회사명'] == '엠앤케이']

금융회사 소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3   성명 보통주주식수  보통주지분율 우선주주식수  \
217  비금융회사  엠앤케이    동일인측          동일인          동일인    -      -       -      -   
218  비금융회사  엠앤케이    동일인측           친족     배우자/혈족1촌    -      -       -      -   
219  비금융회사  엠앤케이    동일인측           친족       혈족2~4촌  문순애  24000  40.00%      -   
220  비금융회사  엠앤케이    동일인측           친족       혈족2~4촌  권미희  12000  20.00%      -   
221  비금융회사  엠앤케이    동일인측           친족       혈족5~6촌    -      -       -      -   
222  비금융회사  엠앤케이    동일인측           친족       인척4촌이내   김영   6000  10.00%      -   
223  비금융회사  엠앤케이    동일인측           친족         친족합계    -  42000  70.00%      -   
224  비금융회사  엠앤케이    동일인측        비영리법인        비영리법인    -      -       -      -   
225  비금융회사  엠앤케이    동일인측        등기된임원        등기된임원    -      -       -      -   
226  비금융회사  엠앤케이    동일인측         자기주식         자기주식    -      -       -      -   
227  비금융회사  엠앤케이    동일인측  계열회사(국내+해외)  계열회사(국내+해외)    -      -       -      -   
228  비금융회사  엠앤케이    동일인측     기타동일인관련자     기타동일인관련자    -      -       -      -   
229  비금융회사  엠앤케이    동일인측       동일인측합계       동일인측합계    /  42000  70.00%      -   
230  비금융회사  엠앤케이    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주    -      -       -      -   
231  비금융회사  엠앤케이    기타주주           기타           기타    /  18000  30.00%      -   
232  비금융회사  엠앤케이    기타주주           총계           총계    /  60000    100%      -   

    우선주지분율    합계주식수   합계지분율  ownCat  ownCat2  
217      -      NaN       -     0.0      0.0  
218      -      NaN       -     NaN      NaN  
219      -  24000.0  40.00%     NaN      NaN  
220      -  12000.0  20.00%     NaN      NaN  
221      -      NaN       -     NaN      NaN  
222      -   6000.0  10.00%     NaN      NaN  
223      -  42000.0  70.00%     0.0      0.0  
224      -      NaN       -     NaN      0.0  
225      -      NaN       -     NaN      0.0  
226      -      NaN       -     NaN      0.0  
227      -      NaN       -     1.0      1.0  
228      -      NaN       -     NaN      NaN  
229      -  42000.0  70.00%     NaN      NaN  
230      -      NaN       -    99.0     99.0  
231      -  18000.0  30.00%    99.0     99.0  
232      -  60000.0    100%    99.0     99.0

In [636]:
df.loc[232, '동일인과의관계'] = '총계'

In [637]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [638]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/엠디엠.xlsx", index=False)

# 아이에스지주

In [639]:
df = pd.read_excel('../data/ownership-status/raw/2021/아이에스지주.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [640]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [641]:
dfOwn

firmname  type   own  own2 ownername
0       골든에코   0.0  0.00  0.00       NaN
1       골든에코   1.0  0.55  0.55     인선이엔티
2       골든에코   1.0   NaN   NaN         -
3       골든에코  99.0  0.45  0.45       NaN
4      나래이앤씨   0.0  0.00  0.00       NaN
..       ...   ...   ...   ...       ...
179    파주비앤알  99.0  0.00  0.00       NaN
180     해니건설   0.0  0.00  0.00       NaN
181     해니건설   1.0  1.00  1.00    아이에스동서
182     해니건설   1.0   NaN   NaN         -
183     해니건설  99.0  0.00  0.00       NaN

[184 rows x 5 columns]

In [642]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/아이에스지주.xlsx", index=False)

# 중앙

In [644]:
df = pd.read_excel('../data/ownership-status/raw/2021/중앙홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [645]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
더기프팅컴퍼니               1.430740
베이스에이스트링창업벤처전문사모투자    0.984085
베이스인베스트먼트             1.000000
Name: own2, dtype: float64

In [646]:
df[df['소속회사명'] == '더기프팅컴퍼니']

금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3  \
88   비금융회사  더기프팅컴퍼니    동일인측          동일인          동일인   
89   비금융회사  더기프팅컴퍼니    동일인측           친족     배우자/혈족1촌   
90   비금융회사  더기프팅컴퍼니    동일인측           친족     배우자/혈족1촌   
91   비금융회사  더기프팅컴퍼니    동일인측           친족       혈족2~4촌   
92   비금융회사  더기프팅컴퍼니    동일인측           친족       혈족2~4촌   
93   비금융회사  더기프팅컴퍼니    동일인측           친족       혈족5~6촌   
94   비금융회사  더기프팅컴퍼니    동일인측           친족       혈족5~6촌   
95   비금융회사  더기프팅컴퍼니    동일인측           친족       인척4촌이내   
96   비금융회사  더기프팅컴퍼니    동일인측           친족       인척4촌이내   
97   비금융회사  더기프팅컴퍼니    동일인측           친족         친족합계   
98   비금융회사  더기프팅컴퍼니    동일인측        비영리법인        비영리법인   
99   비금융회사  더기프팅컴퍼니    동일인측        비영리법인        비영리법인   
100  비금융회사  더기프팅컴퍼니    동일인측        등기된임원        등기된임원   
101  비금융회사  더기프팅컴퍼니    동일인측        등기된임원        등기된임원   
102  비금융회사  더기프팅컴퍼니    동일인측         자기주식         자기주식   
103  비금융회사  더기프팅컴퍼니    동일인측  계열회사(국내+해외)  계열회사(국내+해외)   
104  비금융회사  더기프팅컴퍼니    동일인측  계열회사(국내+해외)  계열회사(국내+해외)   
105  비금융회사  더기프팅컴퍼니    동일인측     기타동일인관련자     기타동일인관련자   
106  비금융회사  더기프팅컴퍼니    동일인측       동일인측합계       동일인측합계   
107  비금융회사  더기프팅컴퍼니    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주   
108  비금융회사  더기프팅컴퍼니    기타주주           기타           기타   
109  비금융회사  더기프팅컴퍼니      총계           총계           총계   

                             성명 보통주주식수  보통주지분율 우선주주식수  우선주지분율    합계주식수  \
88                            -      -       -      -       -      NaN   
89                            -      -       -      -       -      NaN   
90                            -      -       -      -       -      NaN   
91                            -      -       -      -       -      NaN   
92                            -      -       -      -       -      NaN   
93                            -      -       -      -       -      NaN   
94                            -      -       -      -       -      NaN   
95                            -      -       -      -       -      NaN   
96                            -      -       -      -       -      NaN   
97                            /      -       -      -       -      NaN   
98                            -      -       -      -       -      NaN   
99                            -      -       -      -       -      NaN   
100                           -      -       -      -       -      NaN   
101                           -      -       -      -       -      NaN   
102                           -      -       -      -       -      NaN   
103                           -      -       -      -       -      NaN   
104                           -      -       -      -       -      NaN   
105                           -      -       -      -       -      NaN   
106                           /      -       -      -       -      NaN   
107  THEENCORECOMPANYSGPTE.LTD.  16543  43.07%      -       -  16543.0   
108                           /   8492  22.12%  13371  34.81%  38406.0   
109                           /  25035  65.19%  13371  34.81%  38406.0   

      합계지분율  ownCat  ownCat2  
88        -     0.0      0.0  
89        -     NaN      NaN  
90        -     NaN      NaN  
91        -     NaN      NaN  
92        -     NaN      NaN  
93        -     NaN      NaN  
94        -     NaN      NaN  
95        -     NaN      NaN  
96        -     NaN      NaN  
97        -     0.0      0.0  
98        -     NaN      0.0  
99        -     NaN      0.0  
100       -     NaN      0.0  
101       -     NaN      0.0  
102       -     NaN      0.0  
103       -     1.0      1.0  
104       -     1.0      1.0  
105       -     NaN      NaN  
106       -     NaN      NaN  
107  43.07%    99.0     99.0  
108  56.93%    99.0     99.0  
109    100%     NaN      NaN

In [647]:
df.loc[109, '합계주식수'] = 38406.0 + 16543.0

In [648]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
베이스에이스트링창업벤처전문사모투자    0.984085
베이스인베스트먼트             1.000000
Name: own2, dtype: float64

In [649]:
df[df['소속회사명'] == '베이스에이스트링창업벤처전문사모투자']

금융회사               소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3          성명  \
1157  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측          동일인          동일인           -   
1158  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측           친족     배우자/혈족1촌           -   
1159  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측           친족     배우자/혈족1촌           -   
1160  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측           친족       혈족2~4촌           -   
1161  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측           친족       혈족2~4촌           -   
1162  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측           친족       혈족5~6촌           -   
1163  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측           친족       혈족5~6촌           -   
1164  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측           친족       인척4촌이내         신현성   
1165  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측           친족       인척4촌이내           -   
1166  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측           친족         친족합계           /   
1167  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측        비영리법인        비영리법인           -   
1168  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측        비영리법인        비영리법인           -   
1169  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측        등기된임원        등기된임원           -   
1170  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측        등기된임원        등기된임원           -   
1171  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측         자기주식         자기주식           -   
1172  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측  계열회사(국내+해외)  계열회사(국내+해외)  제이에이치앤드컴퍼니   
1173  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측  계열회사(국내+해외)  계열회사(국내+해외)   베이스인베스트먼트   
1174  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측     기타동일인관련자     기타동일인관련자           -   
1175  금융회사  베이스에이스트링창업벤처전문사모투자    동일인측       동일인측합계       동일인측합계           /   
1176  금융회사  베이스에이스트링창업벤처전문사모투자    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주     케이큐브홀딩스   
1177  금융회사  베이스에이스트링창업벤처전문사모투자    기타주주           기타           기타           /   
1178  금융회사  베이스에이스트링창업벤처전문사모투자      총계           총계           총계           /   

       보통주주식수  보통주지분율 우선주주식수 우선주지분율      합계주식수   합계지분율  ownCat  ownCat2  
1157        -       -      -      -        NaN       -     0.0      0.0  
1158        -       -      -      -        NaN       -     NaN      NaN  
1159        -       -      -      -        NaN       -     NaN      NaN  
1160        -       -      -      -        NaN       -     NaN      NaN  
1161        -       -      -      -        NaN       -     NaN      NaN  
1162        -       -      -      -        NaN       -     NaN      NaN  
1163        -       -      -      -        NaN       -     NaN      NaN  
1164    45000   1.59%      -      -    45000.0   1.59%     NaN      NaN  
1165        -       -      -      -        NaN       -     NaN      NaN  
1166    45000   1.59%      -      -    45000.0   1.59%     0.0      0.0  
1167        -       -      -      -        NaN       -     NaN      0.0  
1168        -       -      -      -        NaN       -     NaN      0.0  
1169        -       -      -      -        NaN       -     NaN      0.0  
1170        -       -      -      -        NaN       -     NaN      0.0  
1171        -       -      -      -        NaN       -     NaN      0.0  
1172    45000   1.59%      -      -    45000.0   1.59%     1.0      1.0  
1173     7500   0.27%      -      -     7500.0   0.27%     1.0      1.0  
1174        -       -      -      -        NaN       -     NaN      NaN  
1175   142500   5.04%      -      -   142500.0   5.04%     NaN      NaN  
1176   375000  13.26%      -      -   375000.0  13.26%    99.0     99.0  
1177  2310000  81.70%      -      -  2310000.0  81.70%    99.0     99.0  
1178  2827500    100%      -   100%  2827500.0    100%     NaN      NaN

In [650]:
dfOwn = dfOwn[dfOwn.ownername != '-']
dfOwn.to_excel("../data/ownership-input/2021/중앙.xlsx", index=False)